In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.optimize import minimize_scalar
import plotly.graph_objects as go
from IPython.display import display, Markdown

from src.rent_vs_buy import RentVsBuy, rent_vs_buy_breakeven_objective_closure

In [2]:
# Helper functions.
def render_plotly_html(fig: go.Figure) -> None:
    """Display a Plotly figure in markdown with HTML."""
    # Show the figure if in a IPython display.
    fig.show()
    # Render the content for html output.
    display(
        Markdown(
            fig.to_html(
                include_plotlyjs="cdn",
            )
        )
    )

## Setup
Convert all annualized rates to monthly for consistency.

In [3]:
# We will reuse these kwargs later on.
kwargs = dict(
    home_price=915_000,
    years=35,
    mortgage_rate=0.06342,
    downpayment=0.2,
    length_of_mortgage=30,
    home_price_growth_rate=0.03,
    rent_growth_rate=0.025,
    investment_return_rate=0.04,
    inflation_rate=0.02,
    filing_jointly=True,
    property_tax_rate=0.0077,
    marginal_tax_rate=0.02,
    costs_of_buying_home=0.00,
    costs_of_selling_home=0.06,
    maintenance_rate=0.005,
    home_owners_insurance_rate=0.0042,
    monthly_utilities=0,
    monthly_common_fees=358,
    monthly_rent=3700,
    security_deposit=1,
    brokers_fee=0.00,
    renters_insurance_rate=0.0132,
)
# Create our RentVsBuy calculator.
rent_vs_buy = RentVsBuy().calculate(**kwargs)

## Calculations
**High level overview**

First, model the cash flows over the life of the home. These cashflows should incorporate any inflation or appreciation that occurs with time.  For this we need:
- Value of the home
- Liabilities of the home
- Assets the home
- Liabilities of renting
- Assets of renting

Next, we can calculate the opportunity cost of either renting or buying. This is the difference in money (i.e. $8000 monthly house payment - $3000 monthly rent payment) that we can invest for the remaining time of the home, appreciated at market rates.

### Counter variables
These are helper arrays that just keep track of time.

In [4]:
pd.DataFrame(
    {
        "Annual Periods": rent_vs_buy.annual_per.flatten(),
        "Monthly Periods": rent_vs_buy.per.flatten(),
        "Period Inverse": rent_vs_buy.per_inv.flatten(),
    }
)

Annual Periods  Monthly Periods  Period Inverse
0                 0                1             419
1                 0                2             418
2                 0                3             417
3                 0                4             416
4                 0                5             415
..              ...              ...             ...
415              34              416               4
416              34              417               3
417              34              418               2
418              34              419               1
419              34              420               0

[420 rows x 3 columns]

### Home Value
How much will the home be worth when we sell it?

In [5]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Home Value": rent_vs_buy.home_value.flatten(),
                "Initial Home Price": rent_vs_buy.home_value.flatten()[0],
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="13146a4c-3cc2-459f-be6d-f3bb100b438d" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("13146a4c-3cc2-459f-be6d-f3bb100b438d")) {                    Plotly.newPlot(                        "13146a4c-3cc2-459f-be6d-f3bb100b438d",                        [{"hovertemplate":"variable=Home Value\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Home Value","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Home Value","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[917256.6368416579,919518.8391585454,921786.620676626,924059.9951557147,926338.9763895622,928623.5782059385,930913.8144667164,933209.6990679555,935511.2459399875,937818.4690474994,940131.3823896194,942450.000000001,944774.3359469086,947104.4043333029,949440.2192969257,951781.7950103871,954129.1456812501,956482.2855521177,958841.2289007189,961205.9900399952,963576.5833181881,965953.0231189255,968335.3238613091,970723.500000002,973117.5660253167,975517.5364633028,977923.4258758343,980335.2488606997,982753.0200516887,985176.7541186822,987606.4657677414,990042.1697411961,992483.8808177348,994931.6138124942,997385.3835771492,999845.2050000031,1002311.0930060773,1004783.0625572029,1007261.1286521106,1009745.3063265217,1012235.6106532402,1014732.0567422438,1017234.6597407747,1019743.4348334329,1022258.3972422678,1024779.5622268701,1027306.9450844649,1029840.5611500042,1032380.4257962607,1034926.5544339202,1037478.962511675,1040037.6655163185,1042602.6789728386,1045174.0184445122,1047751.6995329991,1050335.7378784372,1052926.149159537,1055522.9490936773,1058126.1534369998,1060735.7779845055,1063351.8385701498,1065974.3510669388,1068603.3313870262,1071238.795481809,1073880.7593420248,1076529.2389978487,1079184.25051899,1081845.8100147913,1084513.9336343242,1087188.6375664887,1089869.938040111,1092557.8513240418,1095252.3937272553,1097953.581598948,1100661.4313286382,1103375.9593462644,1106097.1821222869,1108825.1161677851,1111559.7780345608,1114301.1843152363,1117049.351643355,1119804.2966934845,1122566.0361813153,1125334.586863764,1128109.9655390743,1130892.1890469177,1133681.2742684986,1136477.2381266537,1139280.0975859566,1142089.86965282,1144906.571375599,1147730.2198446945,1150560.8321926568,1153398.4255942903,1156243.0172667562,1159094.6244696784,1161953.2645052476,1164818.9547183265,1167691.7124965545,1170571.5552704544,1173458.5005135364,1176352.5657424058,1179253.7685168681,1182162.1264400366,1185077.6571584379,1188000.3783621201,1190930.30778476,1193867.4632037699,1196811.8624404063,1199763.5233598775,1202722.4638714525,1205688.7019285692,1208662.2555289438,1211643.1427146792,1214631.3815723755,1217626.9902332388,1220629.9868731922,1223640.389712985,1226658.2170183042,1229683.4870998843,1232716.2183136197,1235756.4290606752,1238804.1377875975,1241859.3629864275,1244922.1231948135,1247992.436996121,1251070.323019548,1254155.7999402375,1257248.8864793894,1260349.6014043759,1263457.9635288545,1266573.991712882,1269697.7048630295,1272829.1219324968,1275968.2619212267,1279115.1438760217,1282269.7868906592,1285432.210106006,1288602.4327101358,1291780.4739384458,1294966.3530737723,1298160.0894465086,1301361.7024347216,1304571.21146427,1307788.6360089218,1311013.995590473,1314247.3097788647,1317488.598192304,1320737.8804973804,1323995.1764091875,1327260.5056914415,1330533.8881566008,1333815.3436659868,1337104.8921299053,1340402.5535077646,1343708.3478081995,1347022.2950891908,1350344.4154581886,1353674.729072232,1357013.2561380742,1360360.0169123032,1363715.0317014644,1367078.320862186,1370449.9048013,1373829.803975968,1377218.0388938037,1380614.630112999,1384019.5982424468,1387432.963941868,1390854.7479219355,1394284.9709444006,1397723.653822218,1401170.8174196738,1404626.4826525098,1408090.670488053,1411563.4019453404,1415044.6980952485,1418534.5800606192,1422033.0690163902,1425540.1861897218,1429055.9528601256,1432580.3903595952,1436113.520072734,1439655.3634368859,1443205.9419422655,1446765.2771320867,1450333.390602696,1453910.304003702,1457496.0390381075,1461090.6174624395,1464694.0610868835,1468306.391775415,1471927.631445931,1475557.8020703846,1479196.9256749174,1482845.0243399942,1486502.1202005348,1490168.2354460508,1493843.3923207785,1497527.6131238148,1501220.920209252,1504923.335986314,1508634.8829194915,1512355.5835286789,1516085.4603893103,1519824.5361324977,1523572.8334451667,1527330.3750701956,1531097.1838065523,1534873.2825094338,1538658.6940904034,1542453.4415175307,1546257.5478155313,1550071.0360659051,1553893.929407078,1557726.2510345408,1561568.0242009915,1565419.2722164742,1569280.0184485232,1573150.286322303,1577030.0993207507,1580919.4809847185,1584818.454913117,1588727.0447630584,1592645.2742499989,1596573.1671478841,1600510.747289292,1604458.0385655786,1608415.0649270227,1612381.85038297,1616358.4190019807,1620344.7949119739,1624341.002300375,1628347.0654142618,1632363.0085605122,1636388.8561059518,1640424.6324775005,1644470.3621623223,1648526.0697079725,1652591.7797225479,1656667.5168748351,1660753.3058944608,1664849.1715720417,1668955.1387593348,1673071.2323693878,1677197.4773766915,1681333.8988173294,1685480.5217891322,1689637.3714518272,1693804.4730271937,1697981.8517992133,1702169.533114226,1706367.5423810817,1710575.9050712965,1714794.646719205,1719023.7929221166,1723263.3693404712,1727513.4016979937,1731773.9157818512,1736044.937442808,1740326.492595384,1744618.6072180113,1748921.3073531915,1753234.6191076546,1757558.5686525162,1761893.1822234371,1766238.4861207828,1770594.506709782,1774961.2704206873,1779338.8037489357,1783727.1332553085,1788126.285566094,1792536.2873732473,1796957.1654345535,1801388.9465737892,1805831.657680886,1810285.3257120936,1814749.977690142,1819225.6407044083,1823712.3419110773,1828210.1085333098,1832718.9678614056,1837238.9472529697,1841770.0741330788,1846312.375994447,1850865.8803975917,1855430.6149710047,1860006.6074113147,1864593.8854834582,1869192.4770208483,1873802.4099255425,1878423.7121684116,1883056.4117893109,1887700.5368972495,1892356.1156705606,1897023.176357073,1901701.747274282,1906391.8568095218,1911093.533420137,1915806.805633656,1920531.702047964,1925268.2513314756,1930016.4822233107,1934776.423533466,1939548.1041429923,1944331.5530041691,1949126.7991406797,1953933.8716477875,1958752.7996925125,1963583.6125138095,1968426.3394227433,1973281.0098026677,1978147.6531094052,1983026.2988714224,1987916.976690012,1992819.7162394717,1997734.5472672842,2002661.4995942963,2007600.603114902,2012551.887797223,2017515.3836832899,2022491.1208892257,2027479.1296054274,2032479.44009675,2037492.0827026893,2042517.087837567,2047554.4859907147,2052604.3077266582,2057666.5836853047,2062741.3445821274,2067828.621208351,2072928.444431142,2078040.845193791,2083165.8545159048,2088303.5034935926,2093453.8232996545,2098616.845183772,2103792.600472696,2108981.120570438,2114182.43695846,2119396.581195866,2124623.5849195933,2129863.4798446037,2135116.2977640782,2140382.070549607,2145660.830151384,2150952.6085984022,2156257.4379986464,2161575.3505392876,2166906.378486879,2172250.5541875535,2177607.910067216,2182978.4786317446,2188362.2924671834,2193759.384239944,2199169.7866970026,2204593.5326660974,2210030.6550559276,2215481.186856357,2220945.161138608,2226422.6110554687,2231913.569841488,2237418.0708131823,2242936.1473692353,2248467.832990699,2254013.161241201,2259572.165767145,2265144.880297915,2270731.3386460827,2276331.5747076077,2281945.6224620496,2287573.5159727684,2293215.289387135,2298870.9769367347,2304540.6129375803,2310224.2317903144,2315921.8679804225,2321633.5560784396,2327359.3307401617,2333099.226706855,2338853.2788054673,2344621.5219488386,2350403.991135914,2356200.721451954,2362011.7480687513,2367837.1062448393,2373676.8313257103,2379530.9587440263,2385399.5240198374,2391282.5627607955,2397180.110662369,2403092.203508063,2409018.8771696337,2414960.1676073065,2420916.110869994,2426886.7430955153,2432872.1005108166,2438872.2194321873,2444887.136265484,2450916.8875063495,2456961.5097404355,2463021.0396436215,2469095.513982243,2475184.9696133076,2481289.4434847254,2487408.972635528,2493543.594196096,2499693.345388383,2505858.2635261435,2512038.3860151554,2518233.750353451,2524444.3941315426,2530670.355032651,2536911.670832933,2543168.3794017127,2549440.51870171,2555728.12678927,2562031.2418145966,2568349.902021982,2574684.145750038],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Initial Home Price\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Initial Home Price","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Initial Home Price","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579,917256.6368416579],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Liabilities
What will we need to pay to upkeep the home?

In [6]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Monthly Fees": rent_vs_buy.monthly_fees.flatten(),
                "Insurance": rent_vs_buy.insurance.flatten(),
                "Property Taxes": rent_vs_buy.property_taxes.flatten(),
                "Maintenance": rent_vs_buy.maintenance.flatten(),
                "Closing Costs": rent_vs_buy.buying_closing_costs.flatten(),
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="c3682458-65ec-4606-a94e-13e2e2c5f5df" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("c3682458-65ec-4606-a94e-13e2e2c5f5df")) {                    Plotly.newPlot(                        "c3682458-65ec-4606-a94e-13e2e2c5f5df",                        [{"hovertemplate":"variable=Monthly Fees\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Monthly Fees","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Monthly Fees","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[358.0,358.0,358.0,358.0,358.0,358.0,358.0,358.0,358.0,358.0,358.0,358.0,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,365.16000000000025,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,372.4632000000005,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,379.9124640000008,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,387.510713280001,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,395.26092754560136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,403.1661460965136,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,411.22946901844415,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,419.45405839881334,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,427.8431395667899,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,436.40000235812596,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,445.1280024052888,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,454.03056245339485,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,463.1111737024631,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,472.3733971765127,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,481.82086512004327,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,491.45728242244445,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,501.28642807089363,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,511.31215663231194,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,521.5383997649585,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,531.969167760258,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,542.6085511154635,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,553.4607221377731,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,564.529936580529,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,575.8205353121399,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,587.3369460183832,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,599.0836849387512,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,611.0653586375266,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,623.2866658102777,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,635.7523991264835,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,648.4674471090136,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,661.4367960511944,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,674.6655319722188,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,688.1588426116637,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973,701.9220194638973],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Insurance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,320.4234744779531,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,330.03617871229204,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,339.9372640736611,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,350.13538199587134,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,360.6394434557478,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,371.4586267594207,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,382.60238556220366,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,394.0804571290703,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,405.9028708429427,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,418.0799569682315,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,430.62235567727885,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,443.54102634759766,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,456.84725713802607,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,470.55267485216734,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,484.66925509773284,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,499.20933275066534,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,514.185612733186,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,529.6111811151819,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,545.4995165486381,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,561.8645020450978,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,578.7204371064513,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,596.0820502196453,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,613.9645117262354,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,632.3834470780231,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,651.3549504903644,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,670.8955990050762,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,691.0224669752292,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,711.7531409844868,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,733.105735214022,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,755.0989072704435,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,777.7518744885577,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,801.0844307232152,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,825.1169636449125,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,849.8704725542608,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895,875.3665867308895],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Property Taxes\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Property Taxes","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Property Taxes","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,586.5059979016223,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,604.1011778386716,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,622.2242131738323,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,640.890939569048,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,660.1176677561201,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,679.9211977888045,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,700.3188337224692,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,721.3283987341442,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,742.9682506961692,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,765.2572982170551,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,788.2150171635675,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,811.8614676784754,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,836.2173117088305,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,861.3038310600963,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,887.1429459919001,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,913.7572343716581,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,941.1699514028089,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,969.405049944894,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,998.487201443242,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1028.4418174865402,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1059.2950720111376,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1091.0739241714728,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1123.806141896618,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1157.520326153518,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1192.2459359381246,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1228.0133140162698,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1264.853713436759,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1302.7993248398632,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1341.8833045850604,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1382.1398037226138,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1423.6039978342938,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1466.312117769324,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1510.3014813024051,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1555.610525741479,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725,1602.278841513725],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Maintenance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Maintenance","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Maintenance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,381.3172003612933,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,392.7567163721325,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,404.5394178632968,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,416.6756003991962,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,429.17586841117253,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,442.05114446350825,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,455.31267879741387,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,468.97205916133686,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,483.0412209361774,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,497.53245756426327,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,512.4584312911917,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,527.832184229928,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,543.6671497568263,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,559.9771642495317,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,576.7764791770182,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,594.0797735523295,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,611.9021667589001,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,630.2592317616676,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,649.1670087145184,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,668.6420189759546,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,688.7012795452339,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,709.3623179315915,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,730.6431874695402,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,752.5624830936272,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,775.1393575864367,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,798.3935383140307,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,822.3453444634525,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,847.015704797357,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,872.4261759412785,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,898.5989612195178,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,925.5569300561044,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,953.3236379577884,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,981.9233470965231,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1011.3810475094199,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035,1041.7224789347035],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Closing Costs\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Closing Costs","line":{"color":"#FFA15A","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Closing Costs","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [7]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Principal Payment": rent_vs_buy.ppmt.flatten(),
                "Interest Payment": rent_vs_buy.ipmt.flatten(),
                "Total Payment": rent_vs_buy.pmt.flatten(),
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="38ebebc8-8256-45f4-b8f0-f886bed98d2f" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("38ebebc8-8256-45f4-b8f0-f886bed98d2f")) {                    Plotly.newPlot(                        "38ebebc8-8256-45f4-b8f0-f886bed98d2f",                        [{"hovertemplate":"variable=Principal Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Principal Payment","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Principal Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[706.0391937261957,709.6663490050332,713.3121381720944,716.9766569557833,720.6600015762933,724.3622687481334,728.0835556826655,731.823960090659,735.5835801848548,739.3625146825466,743.1608628081685,746.9787242959055,750.81619939231,754.673388858932,758.5503939749678,762.4473165399181,766.3642588762614,770.3013238321391,774.2586147840589,778.2362356396075,782.2342908401779,786.2528853637127,790.2921247274617,794.352114990751,798.4329627577695,802.5347751803647,806.6576599608597,810.8017253548787,814.9670801741927,819.1538337895727,823.3620961336633,827.5919777038707,831.843589565261,836.1170433534785,840.4124512776762,844.7299261234643,849.0695812558661,853.4315306223025,857.815888755576,862.2227707768848,866.6522923988391,871.104569928506,875.5797202704598,880.0778609298495,884.5991100154888,889.1435862429548,893.7114089377055,898.3026980382128,902.9175740991127,907.5561582943683,912.2185724204533,916.9049388995531,921.6153807827727,926.350021753371,931.1089861300115,935.892398870019,940.7003855726698,945.5330724824817,950.3905864925341,955.2730551477948,960.1806066484764,965.1133698533954,970.0714742833575,975.0550501245621,980.064228232015,985.0991401329693,990.1599180303751,995.2466948063543,1000.3596040256875,1005.4987799393193,1010.6643574878894,1015.8564723052673,1021.075260722123,1026.3208597694966,1031.5934071824067,1036.8930414034598,1042.2199015864876,1047.5741276002,1052.95586003186,1058.3652401909721,1063.8024101129954,1069.2675125630713,1074.7606910397699,1080.282089778866,1085.831853757118,1091.4101286960768,1097.0170610659147,1102.6527980892656,1108.317487745102,1114.0112787726039,1119.7343206750788,1125.486763723883,1131.2687589623606,1137.0804582098185,1142.9220140655107,1148.7935799126403,1154.6953099223929,1160.6273590579808,1166.5898830787132,1172.5830385440872,1178.6069828178947,1184.6618740723611,1190.7478712922925,1196.86513427925,1203.0138236557514,1209.1941008694844,1215.4061281975441,1221.6500687506991,1227.9260864776707,1234.2343461694363,1240.5750134635632,1246.9482548485516,1253.3542376682044,1259.7931301260296,1266.2651012896467,1272.7703210952386,1279.3089603519988,1285.8811907466256,1292.4871848478306,1299.127116110867,1305.8011588820827,1312.5094884035007,1319.2522808174217,1326.0297131710454,1332.8419634211205,1339.6892104386206,1346.5716340134354,1353.489414859097,1360.4427346175207,1367.431775863774,1374.4567221108791,1381.5177578146167,1388.6150683783826,1395.74884015805,1402.9192604668606,1410.12651758035,1417.3708007412865,1424.652300164636,1431.9712070425662,1439.3277135494586,1446.7220128469617,1454.154299089054,1461.6247674271485,1469.133614015218,1476.6810360149375,1484.2672316008707,1491.8923999656681,1499.5567413252957,1507.2604569242972,1515.0037490410764,1522.786820993203,1530.6098771427642,1538.4731229017139,1546.3767647372806,1554.321010177377,1562.306067816061,1570.332147319004,1578.3994594289966,1586.5082159714889,1594.6586298601442,1602.850915102434,1611.0852868052589,1619.3619611805912,1627.6811555511563,1636.0430883561394,1644.447979156916,1652.896048642824,1661.3875186369537,1669.9226121019733,1678.5015531459812,1687.124567028398,1695.7918801658711,1704.503720138227,1713.2603156944447,1722.0618967586624,1730.9086944362102,1739.8009410196842,1748.7388699950461,1757.7227160477496,1766.7527150689075,1775.8291041614775,1784.9521216464982,1794.122007069337,1803.3390012059908,1812.603346069393,1821.9152849157858,1831.2750622510944,1840.6829238373502,1850.1391166991498,1859.643889130131,1869.1974906994965,1878.800172258576,1888.452185947397,1898.153785201317,1907.9052247576728,1917.7067606624705,1927.5586502771107,1937.461152285142,1947.4145266990568,1957.4190348671132,1967.474939480208,1977.5825045787615,1987.741995559657,1997.953679183212,2008.217823580177,2018.5346982587816,2028.904574111802,2039.3277234236843,2049.8044198776843,2060.3349385630663,2070.9195559823083,2081.5585500583843,2092.2522001420402,2103.000787019143,2113.804592918047,2124.663901517009,2135.5789979516303,2146.5501688223526,2157.5777022019706,2168.661887643211,2179.803016186323,2191.0013803667325,2202.2572742227026,2213.5709933030844,2224.942834675046,2236.373096931895,2247.8620802009073,2259.4100861512156,2271.0174180017207,2282.6843805290628,2294.4112800756175,2306.1984245575413,2318.0461234728587,2329.9546879095883,2341.924430553907,2353.955665698363,2366.048709250134,2378.203878739313,2390.4214933272474,2402.701873814923,2415.0453426513877,2427.452223942215,2439.922843458011,2452.4575286429763,2465.056608623506,2477.720414216811,2490.4492779396337,2503.2435340169513,2516.103518390775,2529.0295687289563,2542.0220244340594,2555.0812266522616,2568.2075182823355,2581.4012439846256,2594.662750190114,2607.9923851095123,2621.390498742404,2634.857442886438,2648.3935711465588,2661.9992389443028,2675.674803527117,2689.420623977743,2703.2370612236637,2717.124478046548,2731.0832390917967,2745.1137108781268,2759.216261807169,2773.3912621731533,2787.6390841726416,2801.9601019142915,2816.35469142867,2830.8232306781456,2845.3660995668024,2859.98367995041,2874.6763556464657,2889.4445124442536,2904.288538114997,2919.2088224220156,2934.2057571309715,2949.2797360201707,2964.431154890869,2979.6604115776927,2994.9679059590735,3010.3540399677513,3025.819217601326,3041.363844932862,3056.9883301215586,3072.693083423465,3088.4785172022466,3104.345045940015,3120.293086248217,3136.3230568785684,3152.435378734046,3168.6304748799475,3184.908770554996,3201.2706931825023,3217.7166723815963,3234.2471399785,3250.8625300178637,3267.563278774176,3284.3498247632097,3301.2226087535264,3318.1820737780745,3335.2286651458007,3352.362830453354,3369.585019596828,3386.8956847835907,3404.295280544131,3421.7842637440344,3439.363093595932,3457.0322316715965,3474.792141914033,3492.6432906496875,3510.5861466006736,3528.6211808970784,3546.7488670893463,3564.9696811607014,3583.284101539657,3601.6926091125615,3620.195687236238,3638.793821750677,3657.4875009917823,3676.2772158042044,3695.163459554217,3714.146728142685,3733.227520018083,3752.406336189565,3771.6836802401476,3791.0600583399096,3810.535979259297,3830.1119543824752,3849.7884977207577,3869.5661259261024,3889.4453583046757,3909.4267168304978,3929.51072615914,3949.6979136414902,3969.988809337624,3990.3839460307036,4010.8838592409743,4031.489087239823,4052.200171063916,4073.017654529409,4093.942084246204,4114.974009632332,4136.113982928357,4157.362559211886,4178.720296412151,4200.187755324625,4221.765499625813,4243.454095887964,4265.254113594029,4287.166125152566,4309.190705912788,4331.328434179661,4353.57989122909,4375.94566132321,4398.426331725665,4421.022492717098,4443.734737610601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Interest Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Interest Payment","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Interest Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[3760.5244690411155,3756.897313762278,3753.251524595217,3749.587005811528,3745.903661191018,3742.2013940191778,3738.4801070846456,3734.739702676652,3730.9800825824564,3727.2011480847646,3723.4027999591426,3719.5849384714056,3715.747463375001,3711.890273908379,3708.0132687923433,3704.116346227393,3700.1994038910498,3696.262338935172,3692.3050479832523,3688.3274271277037,3684.3293719271333,3680.3107774035984,3676.2715380398495,3672.21154777656,3668.1307000095417,3664.0288875869464,3659.9060028064514,3655.7619374124324,3651.5965825931185,3647.4098289777385,3643.201566633648,3638.9716850634404,3634.72007320205,3630.4466194138327,3626.151211489635,3621.833736643847,3617.494081511445,3613.1321321450087,3608.747774011735,3604.3408919904264,3599.911370368472,3595.459092838805,3590.9839424968513,3586.4858018374616,3581.9645527518223,3577.4200765243563,3572.8522538296056,3568.2609647290983,3563.6460886681984,3559.007504472943,3554.345090346858,3549.658723867758,3544.9482819845384,3540.21364101394,3535.4546766372996,3530.671263897292,3525.8632771946413,3521.0305902848295,3516.173076274777,3511.2906076195163,3506.3830561188347,3501.4502929139157,3496.4921884839537,3491.508612642749,3486.499434535296,3481.464522634342,3476.403744736936,3471.316967960957,3466.2040587416236,3461.064882827992,3455.8993052794217,3450.707190462044,3445.488402045188,3440.2428029978146,3434.9702555849044,3429.6706213638513,3424.3437611808235,3418.989535167111,3413.607802735451,3408.198422576339,3402.7612526543157,3397.29615020424,3391.8029717275413,3386.281572988445,3380.731809010193,3375.1535340712344,3369.5466017013964,3363.9108646780455,3358.246175022209,3352.5523839947073,3346.8293420922323,3341.076899043428,3335.2949038049505,3329.4832045574926,3323.6416487018005,3317.770082854671,3311.8683528449183,3305.9363037093303,3299.973779688598,3293.980624223224,3287.9566799494164,3281.90178869495,3275.8157914750186,3269.698528488061,3263.54983911156,3257.3695618978268,3251.157534569767,3244.913594016612,3238.6375762896405,3232.329316597875,3225.988649303748,3219.6154079187595,3213.2094250991067,3206.7705326412815,3200.2985614776644,3193.7933416720725,3187.2547024153123,3180.6824720206855,3174.0764779194806,3167.436546656444,3160.7625038852284,3154.0541743638105,3147.3113819498894,3140.5339495962658,3133.7216993461907,3126.8744523286905,3119.9920287538757,3113.074247908214,3106.1209281497904,3099.131886903537,3092.106940656432,3085.0459049526944,3077.9485943889285,3070.8148226092612,3063.6444023004506,3056.437145186961,3049.1928620260246,3041.911362602675,3034.592455724745,3027.2359492178525,3019.8416499203495,3012.4093636782573,3004.9388953401626,2997.430048752093,2989.8826267523737,2982.2964311664405,2974.671262801643,2967.0069214420155,2959.303205843014,2951.5599137262348,2943.776841774108,2935.953785624547,2928.0905398655973,2920.1868980300305,2912.242652589934,2904.25759495125,2896.231515448307,2888.1642033383146,2880.0554467958223,2871.905032907167,2863.712747664877,2855.4783759620523,2847.20170158672,2838.882507216155,2830.5205744111718,2822.115683610395,2813.667614124487,2805.1761441303574,2796.641050665338,2788.06210962133,2779.4390957389132,2770.77178260144,2762.059942629084,2753.3033470728665,2744.5017660086487,2735.654968331101,2726.762721747627,2717.824792772265,2708.8409467195615,2699.8109476984037,2690.7345586058336,2681.611541120813,2672.441655697974,2663.2246615613203,2653.960316697918,2644.6483778515253,2635.288600516217,2625.880738929961,2616.4245460681614,2606.9197736371802,2597.3661720678147,2587.7634905087352,2578.111476819914,2568.409877565994,2558.6584380096383,2548.8569021048406,2539.0050124902004,2529.1025104821692,2519.1491360682544,2509.144627900198,2499.088723287103,2488.9811581885497,2478.821667207654,2468.609983584099,2458.345839187134,2448.0289645085295,2437.659088655509,2427.235939343627,2416.759242889627,2406.228724204245,2395.644106785003,2385.005112708927,2374.311462625271,2363.562875748168,2352.759069849264,2341.899761250302,2330.984664815681,2320.0134939449586,2308.9859605653405,2297.9017751241004,2286.760646580988,2275.5622824005786,2264.3063885446086,2252.9926694642268,2241.620828092265,2230.1905658354162,2218.701582566404,2207.1535766160955,2195.5462447655905,2183.8792822382484,2172.1523826916937,2160.36523820977,2148.5175392944525,2136.608974857723,2124.6392322134043,2112.607997068948,2100.514953517177,2088.359784027998,2076.1421694400638,2063.861788952388,2051.5183201159234,2039.1114388250965,2026.6408193093005,2014.1061341243346,2001.5070541438056,1988.8432485505,1976.1143848276774,1963.32012875036,1950.4601443765357,1937.534094038355,1924.5416383332517,1911.4824361150497,1898.3561444849754,1885.1624187826858,1871.9009125771972,1858.5712776577989,1845.173164024907,1831.706219880873,1818.1700916207524,1804.5644238230084,1790.888859240194,1777.1430387895678,1763.3266015436475,1749.439184720763,1735.4804236755144,1721.4499518891841,1707.3474009601425,1693.1724005941576,1678.9245785946694,1664.6035608530196,1650.2089713386413,1635.7404320891658,1621.1975632005085,1606.579982816901,1591.8873071208452,1577.1191503230575,1562.275124652314,1547.3548403452955,1532.3579056363394,1517.2839267471404,1502.1325078764423,1486.9032511896185,1471.5957568082374,1456.2096227995596,1440.7444451659853,1425.199817834449,1409.5753326457527,1393.8705793438462,1378.0851455650647,1362.218616827296,1346.270576519094,1330.2406058887427,1314.1282840332651,1297.9331878873634,1281.6548922123152,1265.292969584809,1248.846990385715,1232.3165227888112,1215.7011327494472,1199.000383993135,1182.2138380041015,1165.3410540137847,1148.3815889892367,1131.3349976215102,1114.2008323139569,1096.978643170483,1079.6679779837207,1062.26838222318,1044.779399023277,1027.2005691713791,1009.5314310957148,991.7715208532782,973.9203721176237,955.9775161666374,937.942481870233,919.8147956779648,901.5939816066099,883.2795612276541,864.8710536547494,846.367975531073,827.7698410166342,809.076161775529,790.286446963107,771.4002032130944,752.416934624626,733.3361427492283,714.1573265777462,694.8799825271637,675.5036044274017,656.0276835080139,636.4517083848361,616.7751650465536,596.997536841209,577.1183044626354,557.1369459368133,537.0529366081709,516.8657491258207,496.5748534296871,476.1797167366074,455.679803526337,435.0745755274879,414.3634917033952,393.5460082379021,372.6215785211071,351.5896531349794,330.44967983895503,309.2011035554253,287.84336635516087,266.37590744268584,244.7981631414979,223.10956687934663,201.30954917328188,179.39753761474532,157.37295685452295,135.23522858765023,112.983771538221,90.61800144410121,68.13733104164699,45.541170050212536,22.828925156710486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Total Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Payment","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Total Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767312,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,4466.563662767311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [8]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Total Cumulative Home Liabilities": rent_vs_buy.total_home_liability.flatten().cumsum(),
                "Cumulative Principal": rent_vs_buy.ppmt.flatten().cumsum(),
                "Cumulative Interest": rent_vs_buy.ipmt.flatten().cumsum(),
                "Cumulative Payment": rent_vs_buy.pmt.flatten().cumsum(),
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="5de07813-51cf-47d1-bd9f-9cf3098a530e" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("5de07813-51cf-47d1-bd9f-9cf3098a530e")) {                    Plotly.newPlot(                        "5de07813-51cf-47d1-bd9f-9cf3098a530e",                        [{"hovertemplate":"variable=Total Cumulative Home Liabilities\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Cumulative Home Liabilities","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Total Cumulative Home Liabilities","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[189112.81033550817,195225.62067101634,201338.4310065245,207451.24134203268,213564.05167754085,219676.86201304902,225789.6723485572,231902.48268406535,238015.29301957352,244128.1033550817,250240.91369058986,256353.72402609803,262512.3417617884,268670.9594974788,274829.57723316917,280988.19496885955,287146.8127045499,293305.4304402403,299464.0481759307,305622.66591162106,311781.28364731144,317939.9013830018,324098.5191186922,330257.1368543826,336462.86461226066,342668.59237013874,348874.3201280168,355080.0478858949,361285.77564377297,367491.50340165105,373697.2311595291,379902.9589174072,386108.6866752853,392314.41443316336,398520.14219104144,404725.8699489195,410980.0479976509,417234.22604638233,423488.40409511374,429742.58214384515,435996.76019257656,442250.938241308,448505.1162900394,454759.2943387708,461013.4723875022,467267.6504362336,473521.828484965,479776.00653369643,486080.01388936676,492384.0212450371,498688.0286007074,504992.03595637775,511296.0433120481,517600.0506677184,523904.05802338873,530208.0653790591,536512.0727347294,542816.0800903997,549120.08744607,555424.0948017404,561779.350361065,568134.6059203896,574489.8614797143,580845.1170390389,587200.3725983636,593555.6281576882,599910.8837170128,606266.1392763375,612621.3948356621,618976.6503949867,625331.9059543114,631687.161513636,638095.1252205819,644503.0889275278,650911.0526344737,657319.0163414195,663726.9800483654,670134.9437553113,676542.9074622572,682950.8711692031,689358.834876149,695766.7985830948,702174.7622900407,708582.7259969866,715044.9000437969,721507.0740906071,727969.2481374174,734431.4221842276,740893.5962310379,747355.7702778481,753817.9443246584,760280.1183714686,766742.2924182789,773204.4664650891,779666.6405118994,786128.8145587096,792646.744622351,799164.6746859924,805682.6047496338,812200.5348132752,818718.4648769166,825236.394940558,831754.3250041994,838272.2550678408,844790.1851314822,851308.1151951236,857826.045258765,864343.9753224064,870919.25183749,877494.5283525736,884069.8048676573,890645.0813827409,897220.3578978245,903795.6344129081,910370.9109279917,916946.1874430754,923521.463958159,930096.7404732426,936672.0169883262,943247.2935034098,949881.5529726673,956515.8124419247,963150.0719111821,969784.3313804396,976418.590849697,983052.8503189544,989687.1097882119,996321.3692574693,1002955.6287267267,1009589.8881959842,1016224.1476652416,1022858.407134499,1029553.3334779276,1036248.2598213562,1042943.1861647847,1049638.1125082134,1056333.038851642,1063027.9651950707,1069722.8915384994,1076417.817881928,1083112.7442253567,1089807.6705687854,1096502.596912214,1103197.5232556427,1109954.849199467,1116712.1751432915,1123469.5010871158,1130226.8270309402,1136984.1529747646,1143741.478918589,1150498.8048624133,1157256.1308062377,1164013.456750062,1170770.7826938864,1177528.1086377108,1184285.4345815352,1191106.9430881666,1197928.4515947981,1204749.9601014296,1211571.468608061,1218392.9771146926,1225214.485621324,1232035.9941279555,1238857.502634587,1245679.0111412185,1252500.51964785,1259322.0281544814,1266143.536661113,1273031.0624013233,1279918.5881415338,1286806.1138817442,1293693.6396219546,1300581.165362165,1307468.6911023755,1314356.216842586,1321243.7425827964,1328131.2683230068,1335018.7940632172,1341906.3198034277,1348793.845543638,1355749.2764122002,1362704.7072807623,1369660.1381493243,1376615.5690178864,1383570.9998864485,1390526.4307550106,1397481.8616235727,1404437.2924921347,1411392.7233606968,1418348.154229259,1425303.585097821,1432259.015966383,1439284.2946424678,1446309.5733185525,1453334.8519946372,1460360.1306707219,1467385.4093468066,1474410.6880228913,1481435.966698976,1488461.2453750607,1495486.5240511454,1502511.8027272301,1509537.0814033148,1516562.3600793995,1523659.4856330594,1530756.6111867193,1537853.7367403791,1544950.862294039,1552047.9878476989,1559145.1134013587,1566242.2389550186,1573339.3645086784,1580436.4900623383,1587533.6156159982,1594630.741169658,1601727.866723318,1608898.896269424,1616069.92581553,1623240.955361636,1630411.984907742,1637583.014453848,1644754.043999954,1651925.07354606,1659096.103092166,1666267.132638272,1673438.162184378,1680609.191730484,1687780.22127659,1695027.2716776298,1702274.3220786697,1709521.3724797096,1716768.4228807495,1724015.4732817893,1731262.5236828292,1738509.574083869,1745756.624484909,1753003.674885949,1760250.7252869888,1767497.7756880287,1774744.8260890685,1782070.075708259,1789395.3253274492,1796720.5749466396,1804045.82456583,1811371.0741850203,1818696.3238042106,1826021.573423401,1833346.8230425913,1840672.0726617817,1847997.322280972,1855322.5719001624,1862647.8215193527,1870053.5120255582,1877459.2025317636,1884864.893037969,1892270.5835441744,1899676.2740503799,1907081.9645565853,1914487.6550627907,1921893.3455689962,1929299.0360752016,1936704.726581407,1944110.4170876124,1951516.1075938179,1959004.5458198152,1966492.9840458126,1973981.42227181,1981469.8604978074,1988958.2987238048,1996446.7369498021,2003935.1751757995,2011423.613401797,2018912.0516277943,2026400.4898537917,2033888.928079789,2041377.3663057864,2048950.9261614594,2056524.4860171324,2064098.0458728054,2071671.6057284784,2079245.1655841514,2086818.7254398244,2094392.2852954974,2101965.8451511706,2109539.4050068436,2117112.9648625166,2124686.5247181896,2132260.0845738626,2139921.209015957,2147582.3334580516,2155243.457900146,2162904.5823422405,2170565.706784335,2178226.8312264294,2185887.955668524,2193549.0801106184,2201210.204552713,2208871.3289948073,2216532.453436902,2224193.5778789963,2231944.7809391175,2239695.983999239,2247447.18705936,2255198.3901194814,2262949.5931796026,2270700.796239724,2278451.999299845,2286203.2023599665,2293954.4054200877,2301705.608480209,2309456.8115403303,2317208.0146004516,2325051.883473033,2332895.7523456146,2340739.621218196,2348583.4900907776,2356427.358963359,2364271.2278359407,2372115.096708522,2379958.9655811037,2387802.834453685,2395646.7033262667,2403490.5721988482,2411334.4410714298,2419273.6382634565,2427212.8354554833,2435152.03264751,2443091.229839537,2451030.4270315636,2458969.6242235904,2466908.821415617,2474848.018607644,2482787.2157996707,2490726.4129916974,2498665.610183724,2506604.807375751,2514642.072920069,2522679.338464387,2530716.6040087054,2538753.8695530235,2546791.1350973416,2554828.4006416597,2562865.666185978,2570902.931730296,2578940.197274614,2586977.4628189323,2595014.7283632504,2603051.9939075685,2611190.147641675,2619328.3013757817,2627466.455109888,2635604.608843995,2643742.7625781014,2651880.916312208,2660019.0700463145,2668157.223780421,2676295.3775145276,2684433.531248634,2692571.684982741,2700709.8387168474,2704485.2189663355,2708260.5992158237,2712035.979465312,2715811.3597148,2719586.739964288,2723362.1202137764,2727137.5004632645,2730912.8807127527,2734688.260962241,2738463.641211729,2742239.021461217,2746014.4017107054,2749896.558693207,2753778.7156757084,2757660.87265821,2761543.0296407114,2765425.186623213,2769307.3436057144,2773189.500588216,2777071.6575707174,2780953.814553219,2784835.9715357204,2788718.128518222,2792600.2855007234,2796592.2928247396,2800584.300148756,2804576.307472772,2808568.314796788,2812560.3221208043,2816552.3294448205,2820544.3367688367,2824536.344092853,2828528.351416869,2832520.3587408853,2836512.3660649015,2840504.3733889177,2844609.3942773347,2848714.4151657517,2852819.4360541687,2856924.4569425858,2861029.477831003,2865134.49871942,2869239.519607837,2873344.540496254,2877449.561384671,2881554.582273088,2885659.603161505,2889764.624049922,2893985.913976565,2898207.203903208,2902428.493829851,2906649.783756494,2910871.073683137,2915092.36360978,2919313.653536423,2923534.943463066,2927756.233389709,2931977.523316352,2936198.813242995,2940420.103169638],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Cumulative Principal\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Cumulative Principal","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Cumulative Principal","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[706.0391937261957,1415.705542731229,2129.0176809033233,2845.9943378591065,3566.6543394354,4291.016608183533,5019.100163866198,5750.924123956857,6486.507704141712,7225.870218824259,7969.031081632427,8716.009805928334,9466.826005320643,10221.499394179575,10980.049788154542,11742.49710469446,12508.86136357072,13279.162687402859,14053.421302186918,14831.657537826526,15613.891828666703,16400.144714030415,17190.436838757876,17984.788953748626,18783.221916506394,19585.756691686758,20392.414351647618,21203.216077002497,22018.18315717669,22837.33699096626,23660.699087099925,24488.291064803794,25320.134654369056,26156.251697722535,26996.66414900021,27841.394075123677,28690.463656379543,29543.895187001846,30401.711075757423,31263.933846534306,32130.586138933144,33001.69070886165,33877.27042913211,34757.348290061964,35641.94740007745,36531.09098632041,37424.80239525811,38323.105093296326,39226.02266739544,40133.57882568981,41045.79739811026,41962.70233700982,42884.31771779259,43810.66773954596,44741.776725675976,45677.669124545995,46618.36951011867,47563.90258260115,48514.29316909368,49469.56622424148,50429.746830889955,51394.86020074335,52364.93167502671,53339.98672515127,54320.05095338328,55305.15009351625,56295.31001154662,57290.556706352974,58290.91631037866,59296.41509031798,60307.079447805874,61322.935920111144,62344.011180833266,63370.33204060276,64401.92544778517,65438.818489188634,66481.03839077512,67528.61251837532,68581.56837840717,69639.93361859815,70703.73602871115,71773.00354127423,72847.764232314,73928.04632209287,75013.87817584998,76105.28830454606,77202.30536561197,78304.95816370125,79413.27565144634,80527.28693021895,81647.02125089403,82772.50801461791,83903.77677358028,85040.8572317901,86183.7792458556,87332.57282576825,88487.26813569064,89647.89549474862,90814.48537782734,91987.06841637143,93165.67539918932,94350.33727326168,95541.08514455397,96737.95027883322,97940.96410248897,99150.15820335846,100365.564331556,101587.2144003067,102815.14048678437,104049.3748329538,105289.94984641737,106536.89810126592,107790.25233893412,109050.04546906015,110316.3105703498,111589.08089144505,112868.38985179705,114154.27104254368,115446.75822739152,116745.88534350239,118051.68650238447,119364.19599078798,120683.4482716054,122009.47798477644,123342.31994819756,124682.00915863618,126028.58079264962,127382.07020750872,128742.51294212624,130109.94471799002,131484.4014401009,132865.9191979155,134254.5342662939,135650.28310645194,137053.2023669188,138463.32888449915,139880.69968524043,141305.35198540505,142737.32319244763,144176.65090599708,145623.37291884405,147077.5272179331,148539.15198536025,150008.28559937546,151484.9666353904,152969.23386699127,154461.12626695694,155960.68300828224,157467.94346520654,158982.94721424763,160505.73403524084,162036.3439123836,163574.81703528532,165121.1938000226,166675.51481019997,168237.82087801603,169808.15302533505,171386.55248476405,172973.06070073554,174567.7193305957,176170.57024569812,177781.65553250338,179401.01749368396,181028.69864923513,182664.74173759128,184309.1897167482,185962.085765391,187623.47328402795,189293.39589612992,190971.89744927592,192659.0220163043,194354.81389647018,196059.3176166084,197772.57793230284,199494.6398290615,201225.5485234977,202965.3494645174,204714.08833451246,206471.8110505602,208238.5637656291,210014.39286979058,211799.34499143707,213593.4669985064,215396.80599971238,217209.40934578178,219031.32463069758,220862.5996929487,222703.28261678605,224553.4217334852,226413.0656226153,228282.2631133148,230161.0632855734,232049.51547152078,233947.6692567221,235855.57448147977,237773.28124214223,239700.83989241935,241638.30104470448,243585.71557140353,245543.13460627064,247510.60954575086,249488.1920503296,251475.93404588927,253473.88772507248,255482.10554865265,257500.64024691144,259529.54482102324,261568.87254444693,263618.6769643246,265679.01190288767,267749.93145886995,269831.49000892835,271923.74220907036,274026.7429960895,276140.54758900753,278265.2114905245,280400.79048847617,282547.3406572985,284704.9183595005,286873.5802471437,289053.38326333003,291244.3846436968,293446.64191791945,295660.21291122254,297885.15574589756,300121.52884282946,302369.3909230304,304628.8010091816,306899.8184271833,309182.50280771236,311476.91408778797,313783.1125123455,316101.15863581834,318431.1133237279,320773.03775428183,323126.9934199802,325493.04212923034,327871.24600796966,330261.6675012969,332664.3693751118,335079.4147177632,337506.8669417054,339946.7897851634,342399.2473138064,344864.3039224299,347342.0243366467,349832.47361458634,352335.7171486033,354851.8206669941,357380.850235723,359922.8722601571,362477.95348680933,365046.1610050917,367627.5622490763,370222.22499926644,372830.217384376,375451.6078831184,378086.46532600484,380734.85889715137,383396.8581360957,386072.5329396228,388761.95356360055,391465.1906248242,394182.31510287075,396913.39834196254,399658.51205284067,402417.72831464786,405191.119576821,407978.75866099366,410780.71876290796,413597.0734543366,416427.89668501477,419273.2627845816,422133.246464532,425007.92282017844,427897.3673326227,430801.6558707377,433720.8646931597,436655.0704502907,439604.3501863109,442568.78134120174,445548.4417527794,448543.40965873847,451553.7636987062,454579.58291630755,457620.9467612404,460677.935091362,463750.62817478547,466839.1066919877,469943.45173792774,473063.74482417596,476200.06788105454,479352.50325978856,482521.1337346685,485706.0425052235,488907.31319840596,492125.0298707876,495359.2770107661,498610.13954078394,501877.7028195581,505162.0526443213,508463.2752530748,511781.4573268529,515116.6859919987,518469.0488224521,521838.63384204893,525225.5295268325,528629.8248073766,532051.6090711206,535490.9721647166,538948.0043963882,542422.7965383022,545915.4398289518,549426.0259755525,552954.6471564496,556501.396023539,560066.3657046998,563649.6498062394,567251.342415352,570871.5381025883,574510.3319243389,578167.8194253307,581844.0966411349,585539.2601006891,589253.4068288318,592986.6343488499,596739.0406850395,600510.7243652797,604301.7844236196,608112.3204028789,611942.4323572614,615792.2208549822,619661.7869809082,623551.2323392129,627460.6590560434,631390.1697822025,635339.8676958439,639309.8565051815,643300.2404512123,647311.1243104533,651342.6133976931,655394.813568757,659467.8312232865,663561.7733075327,667676.747317165,671812.8613000934,675970.2238593053,680148.9441557174,684349.131911042,688570.8974106679,692814.3515065558,697079.6056201499,701366.7717453025,705675.9624512153,710007.2908853949,714360.870776624,718736.8164379472,723135.2427696729,727556.26526239,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006,732000.0000000006],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Cumulative Interest\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Cumulative Interest","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Cumulative Interest","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[3760.5244690411155,7517.421782803393,11270.67330739861,15020.26031321014,18766.163974401155,22508.365368420335,26246.84547550498,29981.58517818163,33712.56526076409,37439.76640884885,41163.169208807994,44882.7541472794,48598.5016106544,52310.39188456278,56018.405153355125,59722.521499582515,63422.720903473564,67118.98324240874,70811.28829039198,74499.61571751969,78183.94508944682,81864.25586685042,85540.52740489026,89212.73895266683,92880.86965267637,96544.89854026331,100204.80454306977,103860.5664804822,107512.16306307532,111159.57289205305,114802.7744586867,118441.74614375015,122076.46621695219,125706.91283636603,129333.06404785566,132954.8977844995,136572.39186601096,140185.52399815596,143794.2717721677,147398.61266415814,150998.5240345266,154593.9831273654,158184.96706986226,161771.45287169973,165353.41742445156,168930.8375009759,172503.6897548055,176071.9507195346,179635.5968082028,183194.60431267574,186748.9494030226,190298.60812689035,193843.5564088749,197383.77004988884,200919.22472652615,204449.89599042345,207975.7592676181,211496.7898579029,215012.9629341777,218524.2535417972,222030.63659791602,225532.08689082993,229028.57907931387,232520.08769195664,236006.58712649194,239488.05164912628,242964.45539386323,246435.7723618242,249901.97642056583,253363.0413033938,256818.94060867323,260269.64779913527,263715.13620118046,267155.3790041783,270590.3492597632,274020.01988112705,277444.3636423079,280863.353177475,284276.96098021045,287685.1594027868,291087.9206554411,294485.2168056454,297877.01977737295,301263.3013503614,304644.0331593716,308019.1866934428,311388.7332951442,314752.64415982226,318110.8903348445,321463.44271883916,324810.27206093137,328151.34895997477,331486.6438637797,334816.1270683372,338139.768717039,341457.5387998937,344769.4071527386,348075.34345644794,351375.3172361365,354669.2978603598,357957.2545403092,361239.15632900415,364514.97212047916,367784.6706489672,371048.2204880788,374305.5900499766,377556.7475845464,380801.661178563,384040.2987548527,387272.62807145057,390498.6167207543,393718.23212867306,396931.4415537722,400138.21208641346,403338.51064789115,406532.3039895632,409719.5586919785,412900.2411639992,416074.31764191866,419241.7541885751,422402.51669246034,425556.5708668241,428703.882248774,431844.4161983703,434978.1378977165,438105.0123500452,441225.00437879906,444338.07862670725,447444.19955485704,450543.3314417606,453635.438382417,456720.4842873697,459798.4328817586,462869.24770436785,465932.8921066683,468989.32925185526,472038.5221138813,475080.433476484,478115.0259322087,481142.26188142656,484162.1035313469,487174.51289502514,490179.4517903653,493176.8818391174,496166.76446586975,499149.0608970362,502123.73215983785,505090.73908127984,508050.04228712287,511001.6022008491,513945.3790426232,516881.33282824775,519809.42336811335,522729.6102661434,525641.8529187334,528546.1105136846,531442.3420291329,534330.5062324712,537210.561679267,540082.4667121741,542946.179459839,545801.657835801,548648.8595373877,551487.7420446039,554318.2626190151,557140.3783026255,559954.04591675,562759.2220608804,565555.8631115457,568343.925221167,571123.364316906,573894.1360995075,576656.1960421365,579409.4993892094,582154.0011552181,584889.6561235492,587616.4188452968,590334.2436380691,593043.0845847887,595742.8955324871,598433.6300910929,601115.2416322137,603787.6832879117,606450.907949473,609104.868266171,611749.5166440225,614384.8052445388,617010.6859834688,619627.110529537,622234.0303031742,624831.396475242,627419.1599657508,629997.2714425707,632565.6813201368,635124.3397581464,637673.1966602512,640212.2016727414,642741.3041832236,645260.4533192919,647769.5979471921,650268.6866704792,652757.6678286678,655236.4894958754,657705.0994794596,660163.4453186467,662611.4742831553,665049.1333718108,667476.3693111544,669893.128554044,672299.3572782482,674695.0013850331,677080.006497742,679454.3179603673,681817.8808361155,684170.6399059647,686512.5396672151,688843.5243320308,691163.5378259758,693472.5237865412,695770.4255616653,698057.1862082463,700332.7484906468,702597.0548791914,704850.0475486557,707091.6683767479,709321.8589425833,711540.5605251497,713747.7141017658,715943.2603465314,718127.1396287696,720299.2920114613,722459.657249671,724608.1747889655,726744.7837638232,728869.4229960366,730982.0309931055,733082.5459466226,735170.9057306506,737247.0479000907,739310.9096890431,741362.4280091589,743401.5394479841,745428.1802672934,747442.2864014177,749443.7934555615,751432.636704112,753408.7510889397,755372.0712176901,757322.5313620666,759260.0654561049,761184.6070944382,763096.0895305532,764994.4456750382,766879.6080938209,768751.509006398,770610.0802840558,772455.2534480807,774286.9596679616,776105.1297595823,777909.6941834054,779700.5830426456,781477.7260814352,783241.0526829789,784990.4918676997,786725.9722913752,788447.4222432644,790154.7696442246,791847.9420448187,793526.8666234133,795191.4701842663,796841.679155605,798477.4195876942,800098.6171508947,801705.1971337117,803297.0844408325,804874.2035911556,806436.4787158079,807983.8335561532,809516.1914617895,811033.4753885367,812535.6078964132,814022.5111476028,815494.106904411,816950.3165272105,818391.0609723765,819816.2607902109,821225.8361228567,822619.7067022006,823997.7918477657,825360.010464593,826706.281041112,828036.5216470008,829350.6499310341,830648.5831189214,831930.2380111337,833195.5309807186,834444.3779711042,835676.6944938931,836892.3956266425,838091.3960106357,839273.6098486398,840438.9509026536,841587.3324916429,842718.6674892644,843832.8683215784,844929.8469647488,846009.5149427325,847071.7833249557,848116.562723979,849143.7632931503,850153.294724246,851145.0662450993,852118.986617217,853074.9641333836,854012.9066152538,854932.7214109318,855834.3153925384,856717.5949537661,857582.4660074208,858428.8339829518,859256.6038239684,860065.679985744,860855.966432707,861627.3666359201,862379.7835705448,863113.119713294,863827.2770398717,864522.1570223988,865197.6606268262,865853.6883103342,866490.140018719,867106.9151837656,867703.9127206068,868281.0310250694,868838.1679710062,869375.2209076144,869892.0866567403,870388.66151017,870864.8412269066,871320.521030433,871755.5956059605,872169.9590976639,872563.5051059017,872936.1266844228,873287.7163375578,873618.1660173967,873927.3671209521,874215.2104873073,874481.5863947499,874726.3845578914,874949.4941247707,875150.803673944,875330.2012115588,875487.5741684133,875622.8093970009,875735.7931685392,875826.4111699833,875894.548501025,875940.0896710751,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318,875962.9185962318],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Cumulative Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Cumulative Payment","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Cumulative Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[4466.563662767311,8933.127325534622,13399.690988301933,17866.254651069245,22332.818313836557,26799.38197660387,31265.94563937118,35732.50930213849,40199.0729649058,44665.636627673106,49132.200290440414,53598.76395320772,58065.32761597503,62531.89127874234,66998.45494150966,71465.01860427696,75931.58226704427,80398.14592981158,84864.70959257889,89331.2732553462,93797.8369181135,98264.40058088081,102730.96424364812,107197.52790641543,111664.09156918274,116130.65523195005,120597.21889471736,125063.78255748466,129530.34622025197,133996.90988301928,138463.4735457866,142930.03720855393,147396.60087132125,151863.16453408857,156329.7281968559,160796.29185962322,165262.85552239054,169729.41918515787,174195.9828479252,178662.5465106925,183129.11017345983,187595.67383622716,192062.23749899448,196528.8011617618,200995.36482452913,205461.92848729645,209928.49215006377,214395.0558128311,218861.61947559842,223328.18313836574,227794.74680113306,232261.3104639004,236727.8741266677,241194.43778943503,245661.00145220236,250127.56511496968,254594.128777737,259060.69244050432,263527.2561032716,267993.8197660389,272460.3834288062,276926.9470915735,281393.5107543408,285860.0744171081,290326.6380798754,294793.2017426427,299259.76540540997,303726.32906817726,308192.89273094456,312659.45639371185,317126.02005647914,321592.58371924644,326059.14738201373,330525.711044781,334992.2747075483,339458.8383703156,343925.4020330829,348391.9656958502,352858.5293586175,357325.0930213848,361791.6566841521,366258.2203469194,370724.7840096867,375191.34767245397,379657.91133522126,384124.47499798855,388591.03866075585,393057.60232352314,397524.16598629043,401990.7296490577,406457.293311825,410923.8569745923,415390.4206373596,419856.9843001269,424323.5479628942,428790.1116256615,433256.6752884288,437723.2389511961,442189.8026139634,446656.36627673067,451122.92993949796,455589.49360226525,460056.05726503255,464522.62092779984,468989.18459056714,473455.74825333443,477922.3119161017,482388.875578869,486855.4392416363,491322.0029044036,495788.5665671709,500255.1302299382,504721.6938927055,509188.2575554728,513654.8212182401,518121.38488100737,522587.94854377466,527054.512206542,531521.0758693093,535987.6395320765,540454.2031948438,544920.7668576111,549387.3305203784,553853.8941831457,558320.457845913,562787.0215086803,567253.5851714476,571720.1488342149,576186.7124969822,580653.2761597495,585119.8398225168,589586.4034852841,594052.9671480514,598519.5308108187,602986.094473586,607452.6581363532,611919.2217991205,616385.7854618878,620852.3491246551,625318.9127874224,629785.4764501897,634252.040112957,638718.6037757243,643185.1674384916,647651.7311012589,652118.2947640262,656584.8584267935,661051.4220895608,665517.9857523281,669984.5494150954,674451.1130778627,678917.67674063,683384.2404033972,687850.8040661645,692317.3677289318,696783.9313916991,701250.4950544664,705717.0587172337,710183.622380001,714650.1860427683,719116.7497055356,723583.3133683029,728049.8770310702,732516.4406938375,736983.0043566048,741449.5680193721,745916.1316821394,750382.6953449066,754849.259007674,759315.8226704412,763782.3863332085,768248.9499959758,772715.5136587431,777182.0773215104,781648.6409842777,786115.204647045,790581.7683098123,795048.3319725796,799514.8956353469,803981.4592981142,808448.0229608815,812914.5866236488,817381.150286416,821847.7139491834,826314.2776119506,830780.8412747179,835247.4049374852,839713.9686002525,844180.5322630198,848647.0959257871,853113.6595885544,857580.2232513217,862046.786914089,866513.3505768563,870979.9142396236,875446.4779023909,879913.0415651582,884379.6052279255,888846.1688906928,893312.73255346,897779.2962162273,902245.8598789946,906712.4235417619,911178.9872045292,915645.5508672965,920112.1145300638,924578.6781928311,929045.2418555984,933511.8055183657,937978.369181133,942444.9328439003,946911.4965066676,951378.0601694349,955844.6238322022,960311.1874949695,964777.7511577368,969244.314820504,973710.8784832713,978177.4421460386,982644.0058088059,987110.5694715732,991577.1331343405,996043.6967971078,1000510.2604598751,1004976.8241226424,1009443.3877854097,1013909.951448177,1018376.5151109443,1022843.0787737116,1027309.6424364789,1031776.2060992462,1036242.7697620135,1040709.3334247808,1045175.897087548,1049642.4607503153,1054109.0244130827,1058575.5880758502,1063042.1517386176,1067508.715401385,1071975.2790641524,1076441.8427269198,1080908.4063896872,1085374.9700524546,1089841.533715222,1094308.0973779894,1098774.6610407569,1103241.2247035243,1107707.7883662917,1112174.352029059,1116640.9156918265,1121107.479354594,1125574.0430173613,1130040.6066801287,1134507.1703428961,1138973.7340056635,1143440.297668431,1147906.8613311984,1152373.4249939658,1156839.9886567332,1161306.5523195006,1165773.115982268,1170239.6796450354,1174706.2433078028,1179172.8069705702,1183639.3706333376,1188105.934296105,1192572.4979588725,1197039.0616216399,1201505.6252844073,1205972.1889471747,1210438.752609942,1214905.3162727095,1219371.879935477,1223838.4435982443,1228305.0072610117,1232771.5709237792,1237238.1345865466,1241704.698249314,1246171.2619120814,1250637.8255748488,1255104.3892376162,1259570.9529003836,1264037.516563151,1268504.0802259184,1272970.6438886859,1277437.2075514533,1281903.7712142207,1286370.334876988,1290836.8985397555,1295303.462202523,1299770.0258652903,1304236.5895280577,1308703.1531908251,1313169.7168535925,1317636.28051636,1322102.8441791274,1326569.4078418948,1331035.9715046622,1335502.5351674296,1339969.098830197,1344435.6624929644,1348902.2261557318,1353368.7898184992,1357835.3534812666,1362301.917144034,1366768.4808068015,1371235.0444695689,1375701.6081323363,1380168.1717951037,1384634.735457871,1389101.2991206385,1393567.862783406,1398034.4264461733,1402500.9901089408,1406967.5537717082,1411434.1174344756,1415900.681097243,1420367.2447600104,1424833.8084227778,1429300.3720855452,1433766.9357483126,1438233.49941108,1442700.0630738474,1447166.6267366149,1451633.1903993823,1456099.7540621497,1460566.317724917,1465032.8813876845,1469499.445050452,1473966.0087132193,1478432.5723759867,1482899.1360387541,1487365.6997015215,1491832.263364289,1496298.8270270564,1500765.3906898238,1505231.9543525912,1509698.5180153586,1514165.081678126,1518631.6453408934,1523098.2090036608,1527564.7726664282,1532031.3363291956,1536497.899991963,1540964.4636547305,1545431.0273174979,1549897.5909802653,1554364.1546430327,1558830.7183058,1563297.2819685675,1567763.845631335,1572230.4092941023,1576696.9729568698,1581163.5366196372,1585630.1002824046,1590096.663945172,1594563.2276079394,1599029.7912707068,1603496.3549334742,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416,1607962.9185962416],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Assets
What money will the home bring in (besides the inherent value of the home)?

In [9]:
render_plotly_html(
    px.line(
        pd.DataFrame({"Total Home Assets": rent_vs_buy.total_home_assets.flatten()})
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="7520d45d-69c3-49a8-aefe-70351613fa62" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("7520d45d-69c3-49a8-aefe-70351613fa62")) {                    Plotly.newPlot(                        "7520d45d-69c3-49a8-aefe-70351613fa62",                        [{"hovertemplate":"variable=Total Home Assets\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Home Assets","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Total Home Assets","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.35156180859524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.583099184857026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.739627301468335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.8171320039811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.81135941892811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.717801048015964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.531677922935145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.247923761330593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.861167060707318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.365712063039034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.755518518585236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.02418017289216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.16490189613219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1704753688134266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Rent Liabilities
What will renting cost us?

In [10]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Rent Payment": rent_vs_buy.rent.flatten(),
                "Renters Insurance": rent_vs_buy.renters_insurance.flatten(),
                "Security Deposit": rent_vs_buy.security_deposit_cost.flatten(),
                "Brokers Fee": rent_vs_buy.brokers_fee_cost.flatten(),
                "Total Rent Liability": rent_vs_buy.total_rent_liability.flatten(),
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="80493c85-2886-4fdd-9f4b-7aa46174aaf7" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("80493c85-2886-4fdd-9f4b-7aa46174aaf7")) {                    Plotly.newPlot(                        "80493c85-2886-4fdd-9f4b-7aa46174aaf7",                        [{"hovertemplate":"variable=Rent Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rent Payment","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Rent Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[3700.0,3700.0,3700.0,3700.0,3700.0,3700.0,3700.0,3700.0,3700.0,3700.0,3700.0,3700.0,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3792.4999999999945,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3887.312499999989,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,3984.495312499983,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4084.107695312477,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4186.210387695283,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4290.86564738766,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4398.137288572345,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4508.090720786647,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4620.792988806307,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4736.312813526458,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4854.720633864613,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,4976.0886497112215,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5100.490865953995,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5228.003137602837,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5358.703216042902,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5492.6707964439665,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5629.987566355057,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5770.737255513926,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,5915.005686901765,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6062.880829074302,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6214.4528498011505,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6369.8141710461705,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6529.0595253223155,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6692.286013455364,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,6859.593163791739,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7031.082992886522,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7206.860067708675,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7387.031569401382,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7571.707358636406,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7761.000042602305,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,7955.025043667351,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8153.900669759025,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8357.74818650299,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552,8566.691891165552],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Renters Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Renters Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Renters Insurance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,48.839999999999996,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,50.06099999999993,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,51.31252499999986,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,52.59533812499978,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,53.9102215781247,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,55.25797711757774,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,56.63942654551711,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,58.05541220915495,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,59.506797514383734,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,60.994467452243256,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,62.519329138549246,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,64.08231236701289,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,65.68437017618812,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,67.32647943059274,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,69.00964141635745,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,70.7348824517663,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,72.50325451306036,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,74.31583587588676,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,76.17373177278382,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,78.0780750671033,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,80.03002694378078,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,82.03077761737518,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,84.08154705780944,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,86.18358573425456,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,88.33817537761081,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,90.54662976205096,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,92.8102955061021,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,95.13055289375451,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,97.50881671609824,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,99.94653713400055,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,102.44520056235042,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,105.00633057640904,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,107.63148884081913,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,110.32227606183946,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529,113.08033296338529],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Security Deposit\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Security Deposit","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Security Deposit","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[3700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Brokers Fee\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Brokers Fee","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Brokers Fee","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Total Rent Liability\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Rent Liability","line":{"color":"#FFA15A","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Total Rent Liability","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[3749.84,3749.84,3749.84,3749.84,3749.84,3749.84,3749.84,3749.84,3749.84,3749.84,3749.84,3749.84,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3843.5609999999947,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,3939.625024999989,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4038.0906506249826,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4139.017916890602,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4242.468364812861,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4348.505073933177,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4457.1927007815,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4568.5975183010305,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4682.78745625855,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4799.832142665007,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,4919.802946231625,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5042.77301988741,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5168.8173453845875,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5298.012779019195,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5430.438098494668,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5566.174050957027,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5705.303402230944,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5847.91098728671,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,5994.083761968868,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6143.910856018083,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6297.483627418525,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6454.89571810398,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6616.24311105657,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6781.624188832975,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,6951.13979355379,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7124.8932883926245,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7302.99062060243,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7485.54038611748,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7672.653895770406,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,7864.445243164656,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8061.03137424376,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8262.532158599844,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8469.070462564829,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938,8680.772224128938],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Opportunity Costs
This is where we consider counterfactuals; if we didn't buy the house, how much could we earn in the market after we pay the rent? This goes both ways, depending on whether buying or renting is cheaper.

In [11]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Home Cumulative Opportunity Cost": rent_vs_buy.home_cumulative_opportunity.flatten(),
                "Rental Cumulative Opportunity Cost": rent_vs_buy.rental_cumulative_opportunity.flatten(),
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="139cdd25-952b-4942-8955-e6fa68636277" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("139cdd25-952b-4942-8955-e6fa68636277")) {                    Plotly.newPlot(                        "139cdd25-952b-4942-8955-e6fa68636277",                        [{"hovertemplate":"variable=Home Cumulative Opportunity Cost\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Home Cumulative Opportunity Cost","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Home Cumulative Opportunity Cost","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[729071.9852133185,738335.7230309402,747569.2327404964,756772.6129780383,765945.9620577618,775089.3779730584,784202.9583975609,793286.8006861879,802341.0018761832,811365.6586881524,820360.8675270964,829207.7667401676,837963.1614577912,846689.9868200541,855388.3360504021,864058.3020680877,872699.9774891626,881313.4546274666,889898.8254956147,898456.1818059791,906985.6149716694,915487.2161075089,923961.076031008,932299.3548434861,940539.9835433384,948753.7225988839,956940.6597525751,965100.8824605555,973234.4778935945,981341.5329380181,989422.1341966372,997476.3679896729,1005504.3203556784,1013506.0770524583,1021481.7235579842,1029334.0328291195,1037082.8300640495,1044806.3425289647,1052504.6527295208,1060177.8429021523,1067825.9950149518,1075449.1907685453,1083047.511596965,1090621.038668519,1098169.8528866584,1105694.0348908424,1113193.6650573984,1120581.7388698573,1127860.707629429,1135115.9246962604,1142347.467573488,1149555.4135113508,1156739.8395080161,1163900.8223104016,1171038.4384149956,1178152.7640686736,1185243.8752695133,1192311.8477676066,1199356.7570658682,1206301.4487169927,1213131.6969874054,1219939.6577657822,1226725.4037774885,1233489.0075105827,1240230.5412165907,1246950.076911277,1253647.6863754147,1260323.4411555526,1266977.4125647787,1273609.671683482,1280220.2893601123,1286741.6059601654,1293143.3818677752,1299524.2684132555,1305884.3337597933,1312223.6458481557,1318542.272397415,1324840.280905672,1331117.738650778,1337374.7126910521,1343611.2698659985,1349827.4767970205,1356023.3998881318,1362140.535619249,1368133.2612123066,1374106.4321978826,1380060.1123837836,1385994.3653696072,1391909.2545474223,1397804.8431024451,1403681.1940137157,1409538.3700547693,1415376.4337943078,1421195.4475968676,1426995.4736234862,1432726.8417109891,1438329.1454321006,1443913.168514702,1449478.9706095655,1455026.6111728197,1460556.1494665844,1466067.6445596039,1471561.1553278773,1477036.7404552882,1482494.4584342313,1487934.3675662375,1493356.525962596,1498719.7894083343,1503949.5373081407,1509162.2202404595,1514357.893889259,1519536.6137568073,1524698.4351642653,1529843.413252278,1534971.6029815632,1540083.059133499,1545177.8363107082,1550255.9889376427,1555317.5712611636,1560329.6723866398,1565203.998127255,1570062.4186632826,1574904.9858943182,1579731.7515506053,1584542.7671935891,1589338.084216467,1594117.753844737,1598881.8271367457,1603630.3549842332,1608363.388112877,1613080.9770828339,1617758.1659280304,1622293.4996229198,1626814.0342637189,1631319.81814059,1635810.8993861228,1640287.3259758467,1644749.1457287446,1649196.406307763,1653629.1552203218,1658047.4398188216,1662451.3073011488,1666840.804711181,1671198.6663332384,1675410.7622684157,1679609.1138928605,1683793.7660550473,1687964.7634571074,1692122.1506553069,1696265.9720605216,1700396.2719387128,1704513.094411399,1708616.4834561276,1712706.4829069448,1716783.136454864,1720836.6171690635,1724740.58044876,1728631.8048722597,1732510.3320071816,1736376.2032855067,1740229.4600040214,1744070.1433247582,1747898.2942754354,1751713.9537498958,1755517.1625085429,1759307.961178777,1763086.3902554289,1766849.822859296,1770460.135017592,1774058.6665201504,1777645.455807927,1781220.5411964424,1784783.9608761917,1788335.7529130518,1791875.9552486886,1795404.6057009622,1798921.7419643311,1802427.4016102543,1805921.6220875932,1809404.440723011,1812734.9843413245,1816054.6602047002,1819363.503775246,1822661.550399355,1825948.8353080826,1829225.393617524,1832491.260329188,1835746.4703303718,1838991.058394533,1842225.0591816618,1845448.5072386505,1848661.4369996632,1851725.5193684553,1854779.6034606565,1857823.7219012172,1860857.9072086306,1863882.1917952797,1866896.6079677849,1869901.1879273476,1872895.9637700953,1875880.9674874234,1878856.2309663382,1881821.7859897963,1884777.664237045,1887588.0400986187,1890389.2455424927,1893181.310492266,1895964.2647738955,1898738.1381160142,1901502.9601502486,1904258.7604115359,1907005.5683384389,1909743.4132734605,1912472.3244633577,1915192.3310594533,1917903.462117948,1920472.3555258065,1923032.8664870684,1925585.0223541376,1928128.8503901663,1930664.377769345,1933191.631577193,1935710.6388108488,1938221.4263793568,1940724.021103956,1943218.4497183666,1945704.7388690745,1948182.9151156167,1950522.040444474,1952853.5330731608,1955177.4179076157,1957493.719772508,1959802.4634115023,1962103.6734875236,1964397.37458302,1966683.5912002262,1968962.347761424,1971233.6686092042,1973497.5780067265,1975754.1001379783,1977874.6822772066,1979988.3448352322,1982095.1103910462,1984195.0014499621,1986288.0404438581,1988374.2497314156,1990453.651598358,1992526.2682576897,1994592.121849932,1996651.2344433612,1998703.6280342434,2000749.3245470691,2002662.118309102,2004568.6705153305,2006469.0015323095,2008363.1316601362,2010251.0811326678,2012132.8701177374,2014008.518717369,2015878.0469679926,2017741.4748406583,2019598.8222412493,2021450.1090106948,2023295.3549251822,2025010.6636991827,2026720.3753222267,2028424.508058139,2030123.0801111483,2031816.1096260822,2033503.6146885604,2035185.6133251882,2036862.1235037493,2038533.1631333972,2040198.7500648468,2041858.9020905653,2043513.636944962,2045041.3306248921,2046564.0393526836,2048081.779394521,2049594.5669635118,2051102.418219859,2052605.3492710341,2054103.3761719489,2055596.514925127,2057084.781480875,2058568.191737453,2060046.7615412434,2061520.5066869215,2062870.038902035,2064215.1675160923,2065555.9068982967,2066892.2713709634,2068224.2752096732,2069551.932643425,2070875.2578547876,2072194.2649800512,2073508.9681093786,2074819.3812869554,2076125.5185111405,2077427.3937346148,2078607.8184093067,2079784.3912905506,2080957.1249469696,2082126.0319061745,2083291.124654898,2084452.4156391276,2085609.9172642392,2086763.6418951289,2087913.6018563458,2089059.8094322227,2090202.2768670085,2091341.0163649982,2092361.0036339173,2093377.6626258495,2094391.0042011538,2095401.0391847508,2096407.778366239,2097411.23250001,2098411.412305363,2099408.3284666194,2100401.991633237,2101392.412419924,2102379.601406752,2103363.56913927,2104231.4206425943,2105096.4402966644,2105958.6373419673,2106818.0209888374,2107674.600417555,2108528.3847784447,2109379.3831919734,2110227.6047488474,2111073.0585101093,2111915.7535072346,2112755.6987422295,2113592.903187725,2114316.566770545,2115037.8689975697,2115756.8175740377,2116473.4201800465,2117187.684470632,2117899.6180758523,2118609.2286008666,2119316.5236260197,2120021.5107069197,2120724.1973745218,2121424.591135206,2122122.69947086,2122709.7833096944,2123294.9514602753,2123878.210173603,2124459.565680281,2125039.0241905814,2125616.5918945125,2126192.274961884,2126766.0795423733,2127338.011765591,2127908.0777411456,2128476.283558711,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089,2129042.635288089],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Rental Cumulative Opportunity Cost\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rental Cumulative Opportunity Cost","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Rental Cumulative Opportunity Cost","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,4958.739170965457,9901.29769147647,14827.728359923942,19738.083802414592,24632.41647333314,29510.77865590264,34373.222462742975,39219.79983642758,44050.56255003827,48865.562207718336,53664.850245223795,58448.47793047283,63321.21782637178,68178.05769238558,73019.04941122414,77844.24469630106,82653.69509228613,87447.45197565592,92225.5665552426,96988.08987278103,101735.07280345389,106466.56605643526,111182.62017543225,115883.285539225,120671.37024337775,125443.83115239366,130200.71924761278,134942.08534402013,139667.98009078848,144378.4539718195,149073.55730628295,153753.34024915428,158417.85279175034,163067.14476226346,167701.26582629373,172320.26548737957,177025.01936683743,181714.42136442105,186388.52157420194,191047.36992679173,195691.0161898756,200319.50996874395,204932.9007068222,209531.2376861991,214114.57002815313,218682.94669367716,223236.41648400156,227775.0280411155,232397.75602200732,237005.39977623872,241598.00852450682,246175.63132689885,250738.3170834161,255286.11453449636,259819.07226153457,264337.23868740187,268840.66207696265,273329.39053759037,277803.47201968136,282262.95431716705],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

## Advantage of Buying a Home instead of Renting
Here we hypothesis selling the home and "cashing out".

In [12]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {"Buy vs. Rent": rent_vs_buy.buy_vs_rent.flatten(), "Break Even": 0}
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="73e99634-bea4-47a5-a16f-d26982e79db6" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("73e99634-bea4-47a5-a16f-d26982e79db6")) {                    Plotly.newPlot(                        "73e99634-bea4-47a5-a16f-d26982e79db6",                        [{"hovertemplate":"variable=Buy vs. Rent\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Buy vs. Rent","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Buy vs. Rent","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[-598144.7073884339,-604572.3086791763,-610960.7916235647,-617310.2231938073,-623620.6699121379,-629892.1978512928,-636124.8726349813,-642318.7594383529,-648473.9229884533,-654590.4275646788,-660668.3369992217,-666588.7569342384,-672408.4596623766,-678190.3473525699,-683934.4801231375,-689640.9176536293,-695309.7191852168,-700940.9435210731,-706534.649026752,-712090.8936305571,-717609.7348239059,-723091.2296616884,-728535.4347626197,-733834.4758897356,-739026.2495630343,-744181.4816316925,-749300.2250776433,-754382.5324544953,-759428.4558878305,-764438.0470754906,-769411.3572878604,-774348.4373681447,-779249.3377326387,-784114.1083709912,-788942.7988464637,-793638.1460539929,-798219.9389819573,-802766.3685381922,-807277.4807207794,-811753.3211086877,-816193.9348619728,-820599.3667219745,-824969.6610115046,-829304.86163503,-833605.0120788491,-837870.1554112642,-842100.3342827433,-846208.5062955569,-850197.0847135484,-854151.3847026855,-858071.4454144032,-861957.3055890015,-865809.0035557554,-869626.5772330142,-873410.0641283004,-877159.5013383967,-880874.9255494301,-884556.3730369487,-888203.8796659948,-891740.251187316,-895151.2219005746,-898528.9075621164,-901873.3405986573,-905184.5530325309,-908462.576481704,-911707.4421597829,-914919.1808760175,-918097.8230352958,-921243.3986381353,-924355.9372806646,-927435.4681546021,-930414.2897954549,-933262.120583322,-936077.5696696416,-938860.6628630882,-941611.4255734786,-944329.8828116902,-947016.0591895788,-949669.9789198837,-952291.6658161319,-954881.1432925335,-957438.4343638709,-959963.5616453814,-962397.9776452179,-964696.0154297267,-966962.4861892338,-969197.4092057828,-971400.8033668515,-973572.6871651767,-975713.0786985754,-977821.9956697586,-979899.4553861385,-981945.4747596302,-983960.0703064447,-985943.2581468798,-987845.3218837231,-989605.8086614773,-991335.4555847265,-993034.2754849769,-994702.2808022212,-996339.4835846708,-997945.8954884807,-999521.5277774672,-1001066.3913228206,-1002580.4966028108,-1004063.8537024861,-1005516.4723133658,-1006897.1595964839,-1008131.2461273742,-1009335.1334492209,-1010508.8280036764,-1011652.3358426862,-1012765.662628124,-1013848.8136314193,-1014901.7937331804,-1015924.6074228093,-1016917.2587981104,-1017879.7515648932,-1018812.0890365661,-1019681.3091692461,-1020399.0664100678,-1021087.17935546,-1021745.6481023711,-1022374.472358587,-1022973.6514422669,-1023543.1842814772,-1024083.0694137122,-1024593.3049854137,-1025073.8887514808,-1025524.8180747738,-1025946.0899256098,-1026312.6945200058,-1026523.1227853782,-1026704.3765407199,-1026856.4495677182,-1026979.3352581633,-1027073.0266133866,-1027137.5162436939,-1027172.7963677876,-1027178.8588121858,-1027155.6950106317,-1027103.2960034977,-1027021.6524371824,-1026893.4419574491,-1026604.4777846388,-1026286.7241708245,-1025940.1685959576,-1025564.7981480595,-1025160.5995225628,-1024727.5590216377,-1024265.6625535169,-1023774.8956318108,-1023255.2433748147,-1022706.6905048112,-1022129.2213473653,-1021512.94935134,-1020731.4700395855,-1019921.5018567984,-1019083.025978548,-1018216.0231851212,-1017320.4738607584,-1016396.3579928782,-1015443.6551712967,-1014462.3445874387,-1013452.4050335417,-1012413.8149018533,-1011346.5521838189,-1010247.9272273681,-1008979.7530161062,-1007683.306024255,-1006358.5611038861,-1005005.4927070872,-1003624.0748850899,-1002214.2812873897,-1000776.0851608573,-999309.4593488445,-997814.37629028,-996290.8080187598,-994738.7261616262,-993158.1019390435,-991407.995783016,-989629.7482752437,-987823.3279083697,-985988.7027689698,-984125.8405365713,-982234.708482665,-980315.2734697053,-978367.5019501058,-976391.3599652268,-974386.8131443535,-972353.8267036652,-970292.3654451997,-968064.0303377593,-965807.6002577728,-963523.0372853151,-961210.3030868939,-958869.3589143577,-956500.1656037926,-954102.6835744124,-951676.8728274388,-949222.6929449756,-946740.1030888733,-944229.061999585,-941689.5279950144,-938985.6017840145,-936253.5413310335,-933493.3022327861,-930704.8396634646,-927888.1083735303,-925043.0626884932,-922169.6565076867,-919267.8433030316,-916337.5761177908,-913378.8075653175,-910391.4898277924,-907375.5746549553,-904197.6222883983,-900991.4099462841,-897756.8866530254,-894494.0010000576,-891202.7011445094,-887882.9348078609,-884534.6492745993,-881157.7913908593,-877752.3075630597,-874318.143756526,-870855.2454941096,-867363.5578547912,-863712.0609858893,-860032.0909054566,-856323.5899579658,-852586.500042907,-848820.7626133314,-845026.3186743897,-841203.1087818567,-837351.0730406514,-833470.1511033433,-829560.2821686517,-825621.4049799354,-821653.4578236716,-817527.8016970984,-813373.3703527339,-809190.0993281635,-804977.9237023811,-800736.7780942037,-796466.5966606787,-792167.3130954821,-787838.860627302,-783481.1720182204,-779094.1795620762,-774677.8150828257,-770232.0099328873,-765630.4674657935,-760999.7574992354,-756339.8086169905,-751650.5489289707,-746931.9060695067,-742183.8071956185,-737406.1789852777,-732598.9476356576,-727762.0388613746,-722895.3778927182,-717998.8894738704,-713072.4978611162,-707992.2108238549,-702882.271979355,-697742.6027899552,-692573.1242271322,-687373.7567696427,-682144.4204016528,-676885.0346108614,-671595.5183866061,-666275.790217963,-660925.7680918344,-655545.3694910251,-650134.5113923063,-644571.4685847864,-638978.1956307902,-633354.6066820866,-627700.6153807882,-622016.1348573463,-616301.0777285337,-610555.3560954188,-604778.8815413266,-598971.5651297874,-593133.3174024762,-587264.0483771369,-581363.6675455,-575312.6798555965,-569230.7873785673,-563117.8967441106,-556973.9140516021,-550798.7448679325,-544592.2942253419,-538354.4666192371,-532085.1660059965,-525784.295800766,-519451.75887523894,-513087.4575554279,-506691.29361942015,-500145.96583989915,-493568.9597858889,-486960.1743302045,-480319.5077927178,-473646.85793803306,-466942.1219731597,-460205.1965451678,-453435.9777388321,-446634.36107426346,-439800.2415045253,-432933.5134132416,-426034.0706121884,-418986.7698821246,-411906.9157111598,-404794.398956093,-397649.1098961083,-390470.93823029194,-383259.77307513217,-376015.50296200323,-368738.01583464164,-361427.1990466055,-354082.939358721,-346705.12293651537,-339293.63534763595,-331735.45607323456,-324143.74338050163,-316518.37982565607,-308859.2473604989,-301166.2273297487,-293439.20046836603,-285678.0468988619,-277882.6461285916,-270052.8770470377,-262188.61792307114,-254289.74640220916,-246356.13950384757,-238276.86460195575,-230162.96804805356,-222014.32378661376,-213830.8051286405,-205612.28474882455,-197358.6346826777,-189069.72632365278,-180745.43042025366,-172385.61707312427,-163990.15573212714,-155558.9151934029,-147091.76359641505,-138479.82189172204,-129832.05827194103,-121148.33773074695,-112428.52459694771,-103672.48253143928,-94880.07452414464,-86051.16289094184,-77185.60927056684,-68283.27462151228,-59344.01921890071,-50367.702651345404,-41354.183817796875,-31246.641724982765,-21142.581945613492,-11041.920363773126,-944.5729585927911,9149.54419650184,19240.514934543055,29328.422995770816,39413.35202837922,49495.38558926713,59574.60714478092,69651.10007146187,79724.94765678653,89900.95456218952,100074.140724828,110244.59028446116,120412.38729110779,130577.61570579465,140740.35940130055,150900.70216290606,161058.72768913629,171214.51959250448,181368.16140025528,191519.73655510275,201669.32841597358,211919.77813991578,222168.07573445374,232414.30540569173,242658.55127531616,252900.89738134714,263141.4276788784,273380.2260408215,283617.3762586452,293852.96204311773,304087.06702504307,314319.77475599945,324551.1687090751,334882.15855891677,345211.67234258866,355539.79437566875,365866.6088946718,376192.2000577962,386516.6519456599,396840.04856204195,407162.47383461986,417484.0116157066,427804.7456829846,438124.7597402432,448444.13741810713,458861.88844849356,469278.8473423966,479695.09856866207,490110.7265224606,500525.8155260198,510940.44982936466,521354.7136110561,531768.6909789206,542182.4659707877,552596.1225552224,563009.7446322558,573423.4160341141],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Break Even\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Break Even","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Break Even","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

## Optimize

In addition to visualizing the cost curves, we can also use `scipy.optimize.minimize_scalar` to answer questions.

In [13]:
minimize_scalar(
    fun=rent_vs_buy_breakeven_objective_closure(scalar="mortgage_rate", **kwargs),
    bounds=[0.0001, 0.1],
)

 message: Solution found.
 success: True
  status: 0
     fun: 6.899501220788807
       x: 0.07842162955910047
     nit: 17
    nfev: 17

The listed rent price is $3700, but lets see what rent price we would break even if we were to buy.

In [14]:
minimize_scalar(
    fun=rent_vs_buy_breakeven_objective_closure(scalar="monthly_rent", **kwargs),
    # Exclude PMI for now.
    bounds=[0, 5_000],
)

 message: Solution found.
 success: True
  status: 0
     fun: 0.0238104616291821
       x: 3258.2686230126023
     nit: 27
    nfev: 27

## Vectorize
Since `numpy-financial` is a vectorized library, we can actually see the breakdown for a range of values.

In [15]:
kwargs.update({"home_price": np.linspace(100_000, 2_000_000)})
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Break Even": RentVsBuy().calculate(**kwargs).value,
                "Home Price": np.linspace(100_000, 2_000_000),
            }
        ),
        x="Home Price",
        y="Break Even",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="e7883296-e768-4b26-9506-3203abc277ce" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("e7883296-e768-4b26-9506-3203abc277ce")) {                    Plotly.newPlot(                        "e7883296-e768-4b26-9506-3203abc277ce",                        [{"hovertemplate":"Home Price=%{x}\u003cbr\u003eBreak Even=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"","orientation":"v","showlegend":false,"x":[100000.0,138775.51020408163,177551.02040816325,216326.5306122449,255102.04081632654,293877.55102040817,332653.0612244898,371428.5714285714,410204.0816326531,448979.59183673473,487755.10204081633,526530.612244898,565306.1224489796,604081.6326530613,642857.1428571428,681632.6530612245,720408.1632653062,759183.6734693878,797959.1836734695,836734.693877551,875510.2040816327,914285.7142857143,953061.224489796,991836.7346938776,1030612.2448979593,1069387.7551020407,1108163.2653061226,1146938.775510204,1185714.2857142857,1224489.7959183673,1263265.306122449,1302040.8163265307,1340816.3265306123,1379591.836734694,1418367.3469387756,1457142.8571428573,1495918.367346939,1534693.8775510206,1573469.387755102,1612244.8979591837,1651020.4081632653,1689795.918367347,1728571.4285714286,1767346.9387755103,1806122.448979592,1844897.9591836736,1883673.4693877553,1922448.979591837,1961224.4897959186,2000000.0],"xaxis":"x","y":[3962386.574075157,3801111.056748586,3639835.5394220115,3478560.022095442,3317284.5047688684,3156008.987442293,2994733.470115723,2833457.952789152,2672182.43546258,2510906.918136005,2349631.400809435,2188355.8834828613,2027098.1303236396,1865862.0544387663,1704644.0678519392,1543439.1603051196,1382246.154089294,1221062.0375521427,1059885.6601208497,898715.9146909877,737551.7349886652,576392.0930442894,415235.9967725435,254084.30306386063,92935.46487936657,-68211.55789467227,-229370.77806219878,-390522.4717708798,-551705.1708216253,-712908.1430264446,-874070.3516947916,-1035300.9899815703,-1196469.0653709504,-1357637.14076033,-1518904.1690306813,-1680078.533352654,-1841252.8976746276,-2002559.726933336,-2163740.8151576202,-2324921.9033818897,-2486272.0968483584,-2647460.357304098,-2808648.6177598303,-2969836.8782155598,-3131241.794669836,-3292437.689668975,-3453633.58466811,-3614829.4796672463,-3776025.374666379,-3937221.269665518],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Break Even"}},"legend":{"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

Or even a 2-d Surface:

In [16]:
x, y = np.meshgrid(
    np.linspace(100_000, 2_000_000, 100), np.linspace(0.03, 0.1, 100), indexing="ij"
)
x = x.flatten()
y = y.flatten()
kwargs.update({"home_price": x, "mortgage_rate": y})
render_plotly_html(
    px.imshow(
        RentVsBuy().calculate(**kwargs).value.reshape((100, 100)).T,
        aspect="auto",
        labels=dict(x="Home Price", y="Mortgage Rate", color="Break Even"),
        x=np.linspace(100_000, 2_000_000, 100),
        y=np.linspace(0.03, 0.1, 100),
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.29.1.min.js"></script>                <div id="41b06a8a-710d-4568-b0ce-1c995ddb0efe" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("41b06a8a-710d-4568-b0ce-1c995ddb0efe")) {                    Plotly.newPlot(                        "41b06a8a-710d-4568-b0ce-1c995ddb0efe",                        [{"coloraxis":"coloraxis","name":"0","x":[100000.0,119191.9191919192,138383.8383838384,157575.75757575757,176767.67676767678,195959.59595959596,215151.51515151514,234343.43434343435,253535.35353535353,272727.2727272727,291919.1919191919,311111.1111111111,330303.0303030303,349494.9494949495,368686.8686868687,387878.78787878784,407070.70707070705,426262.62626262626,445454.5454545454,464646.4646464646,483838.38383838383,503030.303030303,522222.2222222222,541414.1414141414,560606.0606060605,579797.9797979798,598989.898989899,618181.8181818181,637373.7373737374,656565.6565656565,675757.5757575757,694949.494949495,714141.4141414141,733333.3333333333,752525.2525252525,771717.1717171717,790909.0909090908,810101.0101010101,829292.9292929292,848484.8484848484,867676.7676767677,886868.6868686868,906060.606060606,925252.5252525252,944444.4444444444,963636.3636363635,982828.2828282828,1002020.202020202,1021212.1212121211,1040404.0404040404,1059595.9595959596,1078787.878787879,1097979.797979798,1117171.717171717,1136363.6363636362,1155555.5555555555,1174747.4747474748,1193939.3939393938,1213131.313131313,1232323.2323232323,1251515.1515151514,1270707.0707070706,1289898.98989899,1309090.909090909,1328282.8282828282,1347474.7474747475,1366666.6666666665,1385858.5858585858,1405050.505050505,1424242.424242424,1443434.3434343433,1462626.2626262626,1481818.1818181816,1501010.101010101,1520202.0202020202,1539393.9393939392,1558585.8585858585,1577777.7777777778,1596969.6969696968,1616161.616161616,1635353.5353535353,1654545.4545454544,1673737.3737373736,1692929.292929293,1712121.212121212,1731313.1313131312,1750505.0505050505,1769696.9696969695,1788888.8888888888,1808080.808080808,1827272.727272727,1846464.6464646463,1865656.5656565656,1884848.4848484846,1904040.404040404,1923232.3232323232,1942424.2424242422,1961616.1616161615,1980808.0808080807,2000000.0],"y":[0.03,0.030707070707070707,0.031414141414141415,0.03212121212121212,0.032828282828282825,0.033535353535353536,0.03424242424242424,0.03494949494949495,0.03565656565656566,0.03636363636363636,0.037070707070707073,0.03777777777777778,0.03848484848484848,0.039191919191919194,0.0398989898989899,0.040606060606060604,0.041313131313131315,0.04202020202020202,0.042727272727272725,0.043434343434343436,0.04414141414141414,0.044848484848484846,0.04555555555555556,0.04626262626262627,0.04696969696969697,0.04767676767676768,0.04838383838383839,0.04909090909090909,0.0497979797979798,0.05050505050505051,0.051212121212121216,0.05191919191919192,0.05262626262626263,0.05333333333333334,0.05404040404040404,0.05474747474747475,0.05545454545454546,0.05616161616161616,0.056868686868686874,0.05757575757575758,0.05828282828282828,0.058989898989898995,0.0596969696969697,0.060404040404040404,0.061111111111111116,0.06181818181818182,0.06252525252525254,0.06323232323232324,0.06393939393939393,0.06464646464646465,0.06535353535353536,0.06606060606060607,0.06676767676767678,0.06747474747474748,0.06818181818181818,0.06888888888888889,0.0695959595959596,0.07030303030303031,0.07101010101010102,0.07171717171717172,0.07242424242424243,0.07313131313131313,0.07383838383838384,0.07454545454545455,0.07525252525252527,0.07595959595959596,0.07666666666666667,0.07737373737373737,0.07808080808080808,0.0787878787878788,0.07949494949494951,0.08020202020202022,0.08090909090909092,0.08161616161616161,0.08232323232323233,0.08303030303030304,0.08373737373737375,0.08444444444444446,0.08515151515151516,0.08585858585858586,0.08656565656565657,0.08727272727272728,0.08797979797979799,0.0886868686868687,0.0893939393939394,0.09010101010101011,0.09080808080808081,0.09151515151515152,0.09222222222222223,0.09292929292929293,0.09363636363636364,0.09434343434343435,0.09505050505050507,0.09575757575757576,0.09646464646464648,0.09717171717171719,0.09787878787878788,0.0985858585858586,0.09929292929292931,0.1],"z":[[4089622.824276359,4034218.666749295,3978814.509222229,3923410.3516951646,3868006.194168105,3812602.036641035,3757197.879113972,3701793.7215869124,3646389.5640598475,3590985.4065327826,3535581.249005722,3480177.091478658,3424772.9339515935,3369368.7764245295,3313964.6188974665,3258560.4613704006,3203156.3038433385,3147752.1463162783,3092347.988789214,3036943.831262147,2981539.6737350873,2926135.5162080238,2870731.3586809575,2815327.201153896,2759923.043626832,2704518.8860997655,2649114.728572704,2593710.571045641,2538306.413518576,2482902.2559915134,2427498.0984644503,2372093.940937387,2316689.783410323,2261285.625883259,2205881.4683561954,2150477.310829131,2095073.1533020695,2039668.9957750048,1984264.8382479413,1928860.6807208783,1873456.5231938146,1818052.3656667517,1762648.2081396882,1707244.050612624,1651839.8930855608,1596435.7355584966,1541031.5780314328,1485627.4205043684,1430223.2629773053,1374819.1054502437,1319414.9479231783,1264010.790396114,1208606.632869054,1153204.3359727717,1097802.998659663,1042402.2401165171,987003.5562101766,931604.8723038356,876207.6856476655,820811.4953175033,765415.3049873346,710020.9563348903,654627.1064531873,599233.2565714861,543841.0619879728,488449.4058667505,433057.74974552356,377667.07253653,322277.46967455465,266887.8668125784,211498.2639505975,156110.43078927323,100722.74662887072,45335.0624684752,-10052.524004059378,-65438.418310756795,-120824.31261745654,-176210.2069241628,-231596.10123086255,-286980.61246378254,-342364.84027964715,-397749.06809550896,-453133.2959113652,-508517.219640255,-563899.8990591662,-619282.578478069,-674665.2578969728,-730047.9373158794,-785430.4857055079,-840811.7297600526,-896192.9738145862,-951574.217869143,-1006955.4619236682,-1062336.7059782166,-1117717.3681304567,-1173097.284991596,-1228477.2018527389,-1283857.1187138828,-1339237.0355750294,-1394616.9524361715],[4087149.1661246032,4031270.26612397,3975391.36612334,3919512.466122707,3863633.566122077,3807754.666121444,3751875.76612081,3695996.8661201806,3640117.9661195492,3584239.0661189137,3528360.1661182833,3472481.266117649,3416602.3661170155,3360723.4661163846,3304844.566115755,3248965.666115119,3193086.766114489,3137207.866113857,3081328.966113225,3025450.066112593,2969571.1661119587,2913692.2661113283,2857813.366110697,2801934.4661100637,2746055.566109431,2690176.6661087982,2634297.7661081674,2578418.866107534,2522539.9661069014,2466661.06610627,2410782.1661056383,2354903.266105005,2299024.3661043732,2243145.466103741,2187266.5661031096,2131387.666102478,2075508.766101846,2019629.8661012128,1963750.9661005808,1907872.0660999473,1851993.166099316,1796114.266098684,1740235.3660980505,1684356.4660974187,1628477.5660967855,1572598.666096155,1516719.766095522,1460840.8660948914,1404961.966094258,1349083.066093628,1293204.1660929949,1237325.2660923605,1181449.1754680173,1125573.1615601159,1069698.782605975,1013825.4847556986,957952.3398644715,902081.595104021,846210.8503435617,790340.8919663276,734472.544118633,678604.1962709418,622736.711972611,566870.6113855275,511004.51079844777,455138.83655839507,399274.83984909113,343410.8431397844,287546.8464304721,231684.3305397951,175822.30034901202,119960.27015823824,64098.239967461675,8238.04435625486,-47622.15088784229,-103482.34613195062,-159342.5413760543,-215201.19716410432,-271059.68347312696,-326918.16978214495,-382776.65609115735,-438634.4980231561,-494491.3960667169,-550348.2941102898,-606205.1921538515,-662062.090197416,-717918.3199934624,-773773.745309134,-829629.1706248112,-885484.5959404884,-941340.0212561609,-997195.4465718316,-1053049.5490779532,-1108903.6122729992,-1164757.6754680509,-1220611.7386631006,-1276465.8018581495,-1332319.8650532002,-1388172.8387562446,-1444025.6457016086],[4084664.9990550783,4028309.3397178696,3971953.680380666,3915598.0210434557,3859242.3617062494,3802886.7023690413,3746531.0430318303,3690175.3836946245,3633819.7243574196,3577464.0650202096,3521108.4056830024,3464752.746345793,3408397.087008584,3352041.4276713775,3295685.7683341703,3239330.10899696,3182974.449659755,3126618.7903225487,3070263.1309853396,3013907.4716481315,2957551.8123109248,2901196.1529737148,2844840.493636509,2788484.834299302,2732129.1749620955,2675773.5156248864,2619417.8562876787,2563062.1969504706,2506706.537613263,2450350.8782760547,2393995.218938847,2337639.5596016394,2281283.9002644313,2224928.2409272236,2168572.5815900164,2112216.9222528087,2055861.262915601,1999505.6035783929,1943149.9442411861,1886794.284903977,1830438.6255667703,1774082.9662295617,1717727.3068923552,1661371.6475551464,1605015.9882179392,1548660.3288807312,1492304.6695435224,1435949.010206314,1379593.350869108,1323237.6915319008,1266882.704949278,1210529.9975539027,1154177.2901585314,1097827.1483913925,1041477.2197009511,985128.475454675,928781.1593880719,872433.843321471,816088.488532701,759743.6258853846,703398.7632380747,647056.0632572728,590713.5014327164,534370.9396081651,478030.2091091457,421689.8018606631,365349.3946121838,309009.99493409693,252671.60211668164,196333.20929925703,139994.81648183707,83658.03325120406,27321.520582875703,-29014.992085449398,-85351.5047537731,-141686.48162010685,-198021.2427876657,-254356.00395521987,-310690.7651227731,-367024.6900158962,-423357.82291791216,-479690.95581992436,-536024.0887219459,-592357.2216239581,-648689.2860295158,-705020.9087001672,-761352.5313708112,-817684.1540414598,-874015.7767121056,-930346.8586160578,-986677.0840923078,-1043007.3095685467,-1099337.5350447986,-1155667.7605210394,-1211997.98599728,-1268327.6129453043,-1324656.5494628698,-1380985.4859804465,-1437314.4224980306,-1493643.3590156091],[4082170.389482186,4025335.966691602,3968501.5439010127,3911667.121110418,3854832.6983198347,3797998.275529239,3741163.8527386445,3684329.429948058,3627495.007157469,3570660.5843668785,3513826.1615762888,3456991.738785698,3400157.315995107,3343322.8932045163,3286488.4704139275,3229654.0476233345,3172819.6248327475,3115985.2020421596,3059150.779251567,3002316.3564609773,2945481.9336703867,2888647.510879796,2831813.088089207,2774978.665298615,2718144.242508027,2661309.819717434,2604475.396926846,2547640.9741362534,2490806.551345663,2433972.128555074,2377137.705764484,2320303.282973894,2263468.8601833023,2206634.4373927126,2149800.0146021224,2092965.591811533,2036131.1690209424,1979296.7462303515,1922462.3234397613,1865627.9006491716,1808793.4778585816,1751959.0550679904,1695124.6322774012,1638290.2094868093,1581455.7866962198,1524621.3639056305,1467786.941115041,1410952.5183244501,1354118.0955338592,1297284.9616541574,1240453.556624685,1183622.6810201108,1126794.1173387554,1069965.553657401,1013139.0879928204,956313.196488841,899487.8207340245,842664.4391969717,785841.0576599105,729018.6802933929,672197.653201913,615376.6261104336,558556.5339141795,501737.71217536833,444918.8904365604,388100.4179688636,331283.65866763424,274466.8993664053,217650.1400651764,160834.59254838945,104019.75870648958,47204.924864576664,-9609.908977331594,-66423.37156644184,-123236.41122211516,-180049.4508777894,-236862.49053345993,-293674.65133506805,-350486.0225951746,-407297.3938552737,-464108.765115384,-520920.136375498,-577730.1768390909,-634540.0002260115,-691349.8236129321,-748159.6469998518,-804969.4703867752,-861778.2998521421,-918586.6908258796,-975395.0817995984,-1032203.4727733191,-1089011.8637470473,-1145820.2547207698,-1202627.3979845084,-1259434.4671405666,-1316241.5362966321,-1373048.6054526893,-1429855.6746087484,-1486662.7437648205,-1543469.19995319],[4079665.4043843234,4022350.22687798,3965035.049371637,3907719.8718652963,3850404.694358954,3793089.516852607,3735774.339346267,3678459.161839924,3621143.984333583,3563828.806827241,3506513.6293209,3449198.4518145532,3391883.2743082084,3334568.0968018696,3277252.919295527,3219937.7417891826,3162622.564282844,3105307.386776499,3047992.209270156,2990677.0317638135,2933361.8542574714,2876046.6767511256,2818731.4992447854,2761416.321738441,2704101.1442321003,2646785.9667257555,2589470.7892194143,2532155.6117130704,2474840.434206727,2417525.2567003863,2360210.0791940438,2302894.901687701,2245579.724181358,2188264.5466750157,2130949.3691686727,2073634.1916623306,2016319.0141559872,1959003.8366496451,1901688.6591433021,1844373.4816369596,1787058.304130618,1729743.1266242745,1672427.949117933,1615112.7716115892,1557797.5941052458,1500482.416598904,1443167.2390925605,1385852.0615862175,1328538.6559139898,1271226.5619580909,1213915.6143856784,1156606.424416429,1099297.3962656995,1041990.9380460302,984684.479826354,927379.4345580051,870075.5428928444,812771.6512276907,755469.8025656771,698168.3190282597,640866.8354908489,583567.448527215,526268.221195885,468968.99386455957,411671.38231868716,354374.26552386675,297077.1487290384,239780.67234142777,182485.52642231528,125190.38050320419,67895.23458409263,10601.13362812344,-46692.17530118814,-103985.48423048761,-161278.79315979686,-218571.33088509366,-275862.93116051424,-333154.5314359367,-390446.1317113573,-447737.7319867704,-505027.87808142323,-562317.8927051034,-619607.9073287798,-676897.9219524646,-734187.9365761308,-791476.6320551252,-848765.178903331,-906053.7257515294,-963342.2725997344,-1020630.8194479356,-1077918.9413729701,-1135206.1333959578,-1192493.3254189482,-1249780.5174419424,-1307067.7094649374,-1364354.9014879195,-1421642.0198023971,-1478927.9652165892,-1536213.9106307896,-1593499.8560449844],[4077150.111287038,4019352.200762023,3961554.2902370095,3903756.379711997,3845958.4691869817,3788160.558661964,3730362.6481369543,3672564.737611941,3614766.827086928,3556968.916561915,3499171.0060369004,3441373.0955118844,3383575.1849868717,3325777.2744618556,3267979.363936845,3210181.4534118283,3152383.542886817,3094585.6323618046,3036787.7218367895,2978989.8113117744,2921191.9007867607,2863393.9902617466,2805596.0797367343,2747798.1692117197,2690000.258686707,2632202.348161691,2574404.4376366767,2516606.5271116635,2458808.61658665,2401010.706061636,2343212.795536623,2285414.885011608,2227616.974486595,2169819.063961582,2112021.153436566,2054223.2429115532,1996425.3323865402,1938627.4218615247,1880829.5113365124,1823031.6008114975,1765233.6902864831,1707435.77976147,1649637.8692364574,1591839.9587114444,1534042.0481864305,1476244.1376614175,1418446.2271364033,1360650.4382179945,1302855.6770026337,1245062.553571371,1187270.7589760195,1129479.74403431,1071690.7407785237,1013901.7375227353,956114.9335459871,898328.5534711662,840542.6896098219,782758.7714029481,724974.8531960743,667191.7893130528,609410.1782410229,551628.5671689897,493847.5896683517,436068.13732257765,378288.6849768027,320509.2326310342,262731.69138502795,204954.25543588353,147176.81948674005,89399.93963872641,31624.38359940611,-26151.172439909074,-83926.72847922333,-141701.77209306043,-199475.57909352332,-257249.38609398995,-315023.19309445564,-372797.00009491574,-430569.36940043233,-488341.5528354617,-546113.7362705013,-603885.9197055446,-661658.0909810066,-719428.7711363649,-777199.4512917111,-834970.1314470582,-892740.8116024034,-950511.4917577654,-1008281.3662777338,-1066050.6584532177,-1123819.950628696,-1181589.2428041818,-1239358.5349796722,-1297127.8271551533,-1354896.476889019,-1412664.4915925488,-1470432.5062960768,-1528200.5209995974,-1585968.5357031347,-1643736.5504066534],[4074624.5782462833,4016341.9694609204,3958059.360675563,3899776.751890206,3841494.1431048396,3783211.5343194795,3724928.9255341203,3666646.31674876,3608363.707963401,3550081.099178038,3491798.490392681,3433515.8816073174,3375233.2728219563,3316950.6640365943,3258668.055251238,3200385.446465875,3142102.8376805186,3083820.2288951576,3025537.620109795,2967255.0113244345,2908972.4025390744,2850689.7937537134,2792407.184968354,2734124.576182993,2675841.9673976344,2617559.358612272,2559276.749826911,2500994.141041551,2442711.532256191,2384428.9234708305,2326146.31468547,2267863.7059001094,2209581.0971147493,2151298.4883293887,2093015.8795440271,2034733.270758668,1976450.661973308,1918168.0531879468,1859885.4444025862,1801602.835617226,1743320.2268318653,1685037.6180465051,1626755.0092611443,1568472.4004757844,1510189.7916904236,1451907.1829050633,1393626.9124310664,1335347.5186843097,1277070.128551688,1218793.764052587,1160518.6787979791,1102245.1652453877,1043971.8463987904,985701.0126913679,927430.1789839468,869160.6387625961,810892.3207200267,752624.002677469,694357.455727323,636091.4958222383,577825.5359171489,519561.24743644707,461297.49453663593,403033.74163682945,344771.0246911333,286509.33381175855,228247.6429323838,169985.95205301,111726.08796657948,53466.32003185339,-4793.447902873624,-63053.114204647485,-121311.09259056672,-179569.07097648643,-237827.0493624024,-296085.0277483249,-354341.7257366916,-412598.04250997864,-470854.3592832694,-529110.676056562,-587366.9499337422,-645621.7277827337,-703876.5056317272,-762131.2834807197,-820386.0613297084,-878640.8391786972,-936894.5500143971,-995147.9065832337,-1053401.2631520536,-1111654.6197208893,-1169907.9762897147,-1228161.3328585532,-1286413.5871921275,-1344665.6352765458,-1402917.6833609575,-1461169.731445374,-1519421.7795297839,-1577673.827614191,-1635925.6583645893,-1694176.5061002225],[4072088.8738315534,4013319.614703972,3954550.3555763895,3895781.096448808,3837011.837321228,3778242.578193648,3719473.3190660616,3660704.0599384825,3601934.8008109033,3543165.5416833204,3484396.2825557394,3425627.0234281565,3366857.7643005736,3308088.5051729954,3249319.2460454176,3190549.98691783,3131780.7277902546,3073011.468662671,3014242.209535089,2955472.9504075074,2896703.691279928,2837934.432152346,2779165.173024765,2720395.9138971847,2661626.654769603,2602857.395642019,2544088.1365144383,2485318.8773868578,2426549.6182592763,2367780.359131695,2309011.1000041147,2250241.8408765327,2191472.581748951,2132703.3226213707,2073934.0634937885,2015164.804366208,1956395.5452386257,1897626.2861110438,1838857.0269834627,1780087.7678558826,1721318.508728301,1662549.249600719,1603779.9904731384,1545010.7313455567,1486241.4722179775,1427474.6351224314,1368708.6567320183,1309944.9222049438,1251182.0356835958,1192420.8096567881,1133660.8337049873,1074901.5597986411,1016144.3203928662,957387.0809870875,898631.8080270775,839877.1381384884,781122.6493984214,722370.3887223639,663618.128046303,604866.2304363614,546116.2251322968,487366.21982823266,428616.21452417085,369868.30208471697,311120.4045261531,252372.50696758786,193625.66290129442,134879.7314353329,76133.79996937374,17387.868503413163,-41356.69482630631,-100100.79611015692,-158844.89739400893,-217588.9986778535,-276332.1186019238,-335074.5200940287,-393816.92158614285,-452559.3230782449,-511301.7245703507,-570042.6087471107,-628783.4355314691,-687524.2623158265,-746265.0891001839,-805005.9158845376,-863745.5346164266,-922484.9066761378,-981224.2787358612,-1039963.6507955752,-1098703.022855294,-1157442.2824957445,-1216180.3149106493,-1274918.3473255485,-1333656.3797404636,-1392394.4121553581,-1451132.4445702601,-1509870.4769851714,-1568607.7647951627,-1627344.5679317983,-1686081.3710684301,-1744818.174205062],[4069543.067109093,4010285.2188125565,3951027.3705160203,3891769.522219481,3832511.673922939,3773253.8256264045,3713995.9773298614,3654738.129033327,3595480.28073679,3536222.4324402516,3476964.5841437136,3417706.735847176,3358448.887550634,3299191.039254099,3239933.190957562,3180675.342661019,3121417.4943644837,3062159.646067947,3002901.7977714073,2943643.9494748684,2884386.1011783313,2825128.2528817924,2765870.404585254,2706612.556288717,2647354.7079921784,2588096.85969564,2528839.011399101,2469581.1631025635,2410323.3148060245,2351065.4665094875,2291807.618212949,2232549.7699164115,2173291.9216198726,2114034.073323334,2054776.225026796,1995518.376730258,1936260.5284337199,1877002.6801371821,1817744.8318406437,1758486.9835441043,1699229.1352475674,1639971.2869510285,1580713.4386544924,1521455.5903579518,1462200.114914354,1402945.6130230075,1343693.4696555103,1284442.1222478899,1225192.698151268,1165944.3209515053,1106697.0416978754,1047451.4577805423,988205.8738632081,928962.8247967167,869719.8643030128,810477.7552744667,751237.2551393136,691996.7550041615,632757.4252936183,573519.2289833156,514281.0326730171,455043.74781422317,395807.7050741259,336571.6623340342,277335.73490964854,218101.70152159873,158867.66813354334,99633.63474549307,40400.29282453982,-18831.869626397733,-78064.03207733901,-137296.19452827377,-196527.81731349137,-255758.24166328833,-314988.6660130769,-374219.0903628608,-433449.51471264847,-492678.618098977,-551907.4318202632,-611136.2455415521,-670365.0592628438,-729593.8729841392,-788821.4589456869,-848048.7843551422,-907276.1097645992,-966503.4351740554,-1025730.7605835181,-1084957.7660316722,-1144183.720489352,-1203409.6749470374,-1262635.62940472,-1321861.5838624034,-1381087.5383200953,-1440313.4927777713,-1499538.2763070026,-1558762.9724080525,-1617987.6685090922,-1677212.364610128,-1736437.0607111715,-1795661.7568122186],[4066987.227625091,4007238.8646801156,3947490.501735127,3887742.138790147,3827993.775845165,3768245.412900181,3708497.049955202,3648748.687010216,3589000.3240652373,3529251.961120252,3469503.598175272,3409755.2352302857,3350006.8722853027,3290258.5093403216,3230510.146395343,3170761.7834503576,3111013.420505376,3051265.0575603927,2991516.694615411,2931768.3316704263,2872019.9687254447,2812271.6057804627,2752523.2428354807,2692774.8798904982,2633026.516945516,2573278.154000531,2513529.791055551,2453781.428110568,2394033.065165585,2334284.7022206024,2274536.339275621,2214787.976330638,2155039.6133856564,2095291.250440673,2035542.8874956905,1975794.5245507087,1916046.161605727,1856297.7986607435,1796549.4357157608,1736801.0727707788,1677052.7098257975,1617304.3468808117,1557555.9839358309,1497809.8118498751,1438064.860947437,1378322.2576350833,1318580.523823757,1258840.8576172306,1199102.1536665205,1139364.831958319,1079628.978060905,1019893.4452458157,960160.2687312919,900427.0922167697,840695.339308755,780964.6743683494,721234.0094279349,661505.2306271242,601776.9180490151,542048.6054708962,482322.0454191128,422595.9323339313,362869.8192487415,303144.77036836883,243420.71000450104,183696.64964063186,123972.5892767692,64250.30105196964,4528.152500266675,-55193.99605143303,-114916.14460313227,-174636.56220335327,-234356.93421581574,-294077.30622828193,-353797.67824074626,-413517.058436892,-473235.7837643735,-532954.5090918457,-592673.2344193319,-652391.9597468004,-712109.5599209685,-771826.7632081797,-831543.966495405,-891261.1697826162,-950978.3730698302,-1010695.1609834135,-1070410.9618666982,-1130126.7627499783,-1189842.5636332585,-1249558.3645165432,-1309274.1653998243,-1368989.7585767433,-1428704.2718773903,-1488418.7851780448,-1548133.2984786965,-1607847.8117793435,-1667562.3250800008,-1727276.8383806534,-1786991.1255999198,-1846705.8649819633],[4064421.4253889755,4004180.635752218,3943939.846115461,3883699.0564786918,3823458.266841934,3763217.4772051675,3702976.6875684075,3642735.8979316438,3582495.108294887,3522254.318658119,3462013.5290213595,3401772.7393845953,3341531.9497478325,3281291.1601110715,3221050.370474312,3160809.5808375473,3100568.7912007873,3040328.0015640236,2980087.211927262,2919846.422290501,2859605.6326537393,2799364.843016977,2739124.053380214,2678883.2637434513,2618642.4741066904,2558401.684469928,2498160.894833166,2437920.1051964033,2377679.315559642,2317438.525922879,2257197.736286117,2196956.9466493553,2136716.157012593,2076475.3673758318,2016234.5777390688,1955993.7881023074,1895752.998465546,1835512.2088287831,1775271.419192021,1715030.6295552591,1654789.839918496,1594549.050281735,1534310.136780856,1474072.8247931774,1413837.7238642883,1353603.6915673385,1293371.7525609047,1233140.8097909372,1172911.4217948182,1112683.3858824135,1052455.9756346992,992230.6711159926,932005.3665972902,871781.9593590037,811559.2176980535,751336.5849170955,691116.2442389256,630895.9035607604,570675.8253164454,510457.7301503341,450239.63498422876,390021.5398181188,329805.37827885244,269589.3793115346,209373.38034420507,149158.2080640397,88944.16190234339,28730.115740650333,-31483.93042104831,-91696.93676920328,-151909.16782651283,-212121.39888381585,-272333.62994112354,-332545.3320607245,-392755.8802423645,-452966.4284240166,-513176.97660566214,-573387.5247873105,-633597.1737061534,-693806.1659793509,-754015.1582525428,-814224.1505257431,-874433.1427989416,-934641.7374087544,-994849.2956820307,-1055056.8539553117,-1115264.4122285992,-1175471.9705018932,-1235679.528775175,-1295886.6873232014,-1356092.9286415167,-1416299.1699598227,-1476505.4112781305,-1536711.6525964392,-1596917.8939147517,-1657124.135233068,-1717329.7527609086,-1777536.6178696891,-1837742.8591879895,-1897949.1005063104],[4061845.7308566426,4001110.6160066053,3940375.5011565643,3879640.3863065266,3818905.271456486,3758170.1566064483,3697435.041756412,3636699.9269063715,3575964.8120563347,3515229.697206293,3454494.5823562583,3393759.4675062145,3333024.352656175,3272289.2378061377,3211554.122956101,3150819.00810606,3090083.8932560226,3029348.7784059835,2968613.6635559443,2907878.5487059066,2847143.433855868,2786408.3190058284,2725673.20415579,2664938.0893057496,2604202.9744557133,2543467.859605672,2482732.7447556346,2421997.629905595,2361262.5150555554,2300527.4002055177,2239792.2853554795,2179057.1705054403,2118322.055655401,2057586.940805363,1996851.8259553234,1936116.7111052854,1875381.5962552465,1814646.4814052065,1753911.3665551688,1693176.2517051296,1632441.1368550896,1571707.4507747737,1510975.8791494162,1450246.2564530259,1389518.0271097282,1328791.7980528085,1268066.7179156712,1207343.253162645,1146621.1367196504,1085899.8452273668,1025180.5142391515,964461.1832509353,903744.12446367,843027.4076835271,782311.2824709625,721597.0153755788,660882.7482802016,600169.3668781724,539457.3914087759,478745.415939386,418034.0238357042,357324.1881491039,296614.3524625059,235904.51677591354,175196.40023114718,114488.55846040137,53780.71668966301,-6926.92160697002,-67632.9095827206,-128338.89755845955,-189044.88553420268,-249750.87350994442,-310455.21088298224,-371159.47965976875,-431863.7484365571,-492568.0172133446,-553271.7368608201,-613974.4157226868,-674677.094584547,-735379.773446423,-796082.4523082823,-856784.86532191,-917486.0784454588,-978187.2915690057,-1038888.5046925638,-1099589.7178161088,-1160290.9309396623,-1220991.6564302947,-1281691.5230814498,-1342391.3897326011,-1403091.2563837469,-1463791.1230348935,-1524490.989686031,-1585190.856337185,-1645889.7957625855,-1706590.5896394728,-1767290.4562906316,-1827990.3229417717,-1888690.1895929296,-1949390.0562440725],[4059260.2149137594,3998028.8899332695,3936797.564952782,3875566.239972291,3814334.9149918016,3753103.590011312,3691872.2650308157,3630640.9400503263,3569409.6150698382,3508178.2900893465,3446946.965108849,3385715.6401283615,3324484.3151478716,3263252.99016738,3202021.6651868913,3140790.3402064005,3079559.0152259083,3018327.6902454193,2957096.3652649275,2895865.0402844367,2834633.715303947,2773402.3903234554,2712171.0653429637,2650939.7403624738,2589708.415381984,2528477.090401491,2467245.7654210017,2406014.4404405104,2344783.115460018,2283551.790479528,2222320.465499038,2161089.1405185475,2099857.8155380557,2038626.4905575658,1977395.1655770736,1916163.8405965841,1854932.5156160928,1793701.190635602,1732469.8656551116,1671238.5406746217,1610008.0645414987,1548780.348330214,1487554.1933208113,1426329.8819741264,1365107.3591416723,1303886.2566923245,1242666.718164977,1181448.6362776076,1120231.4731125424,1059016.2307900453,997800.9884675487,936588.2942333673,875375.7175344964,814164.1223420696,752954.0441099261,691743.9658777853,630535.31007504,569327.5696760011,508119.8292769613,446913.337749531,385707.7808222198,324502.2238948941,263297.1690551117,202093.64726899238,140890.12548287306,79686.60369675932,18484.19558727648,-42717.43358938955,-103919.06276605604,-165120.6919427216,-226321.3671631571,-287521.2406865191,-348721.11420987546,-409920.98773322534,-471120.78745100833,-532319.0369161796,-593517.2863813406,-654715.5358465081,-715913.7853116719,-777112.0158203188,-838308.7676677229,-899505.519515126,-960702.2713625291,-1021899.023209936,-1083095.7750573307,-1144292.0564614339,-1205487.4321751911,-1266682.807888968,-1327878.1836027354,-1389073.5593165085,-1450268.9350302778,-1511464.3107440397,-1572658.551060629,-1633855.0621715859,-1695050.4378853543,-1756245.8135991208,-1817441.189312892,-1878636.5650266586,-1939850.3009209922,-2001047.0527683962],[4056664.9488591733,3994935.542514677,3933206.136170171,3871476.7298256713,3809747.323481168,3748017.917136664,3686288.5107921613,3624559.1044476572,3562829.6981031564,3501100.291758658,3439370.885414153,3377641.4790696492,3315912.0727251484,3254182.666380644,3192453.260036146,3130723.8536916417,3068994.4473471376,3007265.041002638,2945535.6346581355,2883806.2283136314,2822076.8219691296,2760347.4156246292,2698618.009280127,2636888.6029356243,2575159.196591122,2513429.790246618,2451700.3839021176,2389970.977557614,2328241.571213112,2266512.16486861,2204782.7585241087,2143053.3521796064,2081323.9458351035,2019594.539490602,1957865.133146098,1896135.726801597,1834406.3204570953,1772676.9141125912,1710947.5077680906,1649218.2378798565,1587492.5058178743,1525767.821636928,1464045.557012816,1402324.7502881715,1340605.7542637633,1278888.1584801981,1217172.2399158175,1155457.2281183107,1093744.2032766435,1032031.1860008186,970320.8782621676,908610.5705235121,846901.5411543339,785193.7807428199,723486.0203313064,661780.1321860561,600074.7559063588,538369.3796266592,476665.83687473414,414962.6878589601,353259.5388431791,291557.5938397837,229856.52130403975,168155.44876829814,106454.39951839205,44755.25852878904,-16943.88246082235,-78643.02345043235,-140342.04346684925,-202039.39221928595,-263736.7409717273,-325434.0897241691,-387131.4384766072,-448827.65653642174,-510523.34695209377,-572219.0373677788,-633914.7277834602,-695610.4181991294,-757304.922950712,-818999.0837295409,-880693.2445083605,-942387.4052871969,-1004081.5660660192,-1065775.3796647443,-1127468.134506194,-1189160.889347637,-1250853.6441890867,-1312546.3990305243,-1374239.1538719796,-1435931.908713431,-1497623.4162316462,-1559317.4183963146,-1621010.173237768,-1682702.9280792112,-1744395.6829206524,-1806088.4377621077,-1867799.8169695772,-1929493.9777484052,-1991188.1385272322,-2052882.2993060602],[4054060.0043882583,3991830.659205908,3929601.3140235534,3867371.9688411974,3805142.623658843,3742913.278476488,3680683.933294132,3618454.588111779,3556225.2429294284,3493995.897747068,3431766.5525647155,3369537.207382358,3307307.8622000054,3245078.517017652,3182849.171835299,3120619.826652941,3058390.4814705863,2996161.136288233,2933931.791105878,2871702.445923523,2809473.100741172,2747243.755558815,2685014.410376463,2622785.065194106,2560555.720011752,2498326.3748293966,2436097.029647042,2373867.6844646866,2311638.339282333,2249408.9940999793,2187179.6489176243,2124950.3037352697,2062720.9585529151,2000491.6133705606,1938262.268188206,1876032.923005852,1813803.5778234967,1751574.2326411407,1689344.8874587873,1627118.5737253893,1564893.3772796544,1502671.3018631737,1440450.2348177324,1378231.4877137223,1316013.864808317,1253798.2520915428,1191583.4282520886,1129370.763462376,1067158.1317028524,1004948.2355579745,942738.3394130943,880529.9251043024,818322.6252239011,756115.3253435004,693910.2602980784,631705.3909382131,569500.5873435983,507297.98914149636,445095.3909393982,382892.7927372968,320692.02508863714,258491.54481725767,196291.06454588147,134091.340132948,71892.83046500152,9694.320797055494,-52504.188870891,-114701.71882454585,-176898.39954352425,-239095.08026250498,-301291.760981489,-363488.0050951112,-425682.99306494,-487877.981034765,-550072.9690045919,-612267.9569744272,-674462.2070398452,-736655.6332154898,-798849.0593911344,-861042.4855667809,-923235.911742419,-985429.2210413907,-1047621.2113346951,-1109813.2016279912,-1172005.1919212863,-1234197.1822145851,-1296389.172507871,-1358581.1628011763,-1420771.8910635756,-1482962.566537207,-1545157.133681071,-1607349.1239743615,-1669541.114267663,-1731733.1045609582,-1793943.8782014186,-1856137.304377065,-1918330.7305527022,-1980524.1567283487,-2042717.5829039933,-2104911.009079634],[4051445.4535764726,3988714.325915087,3925983.1982537033,3863252.0705923173,3800520.942930931,3737789.815269547,3675058.6876081643,3612327.55994678,3549596.4322853973,3486865.3046240094,3424134.176962626,3361403.0493012397,3298671.9216398546,3235940.793978471,3173209.6663170876,3110478.5386557016,3047747.4109943165,2985016.2833329313,2922285.155671549,2859554.028010162,2796822.900348779,2734091.7726873937,2671360.645026009,2608629.517364624,2545898.3897032416,2483167.2620418537,2420436.13438047,2357705.0067190854,2294973.8790577007,2232242.751396316,2169511.623734932,2106780.496073547,2044049.3684121622,1981318.240750778,1918587.113089392,1855855.9854280076,1793124.8577666245,1730393.7301052394,1667664.722279433,1604937.3998330592,1542213.3144243425,1479490.0243940027,1416769.6825371147,1354050.076338062,1291332.9258237386,1228616.3401613848,1165902.1918237347,1103188.0434860862,1040476.673993771,977765.3459038145,915055.6094060494,852346.9265933372,789638.2765562357,726932.0707412027,664225.8649261696,601520.1026124852,538816.2119490816,476112.32128567295,413408.6524009914,350706.92122922326,288005.1900574458,225303.45888566412,162603.14602879528,99903.42463675467,37203.703244709875,-25496.018147334456,-88193.96550832596,-150891.82111322554,-213589.67671812978,-276287.5323230317,-338984.0459458688,-401680.17425691243,-464376.3025679542,-527072.4308789903,-589768.3868355462,-652462.9210582329,-715157.4552809121,-777851.9895035904,-840546.5237262668,-903241.0579489525,-965934.3476281324,-1028627.4158835784,-1091320.4841390317,-1154013.552394485,-1216706.6206499245,-1279399.6889053788,-1342091.5786048397,-1404783.304125376,-1467478.8936717184,-1530171.9619271643,-1592865.0301826075,-1655558.0984380543,-1718270.0028438112,-1780964.5370664839,-1843659.0712891743,-1906353.6055118507,-1969048.139734529,-2031742.6739572147,-2094475.6386550097,-2157171.7669660393],[4048821.3688627896,3985586.62898363,3922351.889104469,3859117.1492253044,3795882.4093461437,3732647.669466977,3669412.929587818,3606178.189708654,3542943.449829493,3479708.70995033,3416473.9700711686,3353239.2301920047,3290004.4903128427,3226769.7504336797,3163535.0105545195,3100300.2706753556,3037065.530796194,2973830.7909170333,2910596.051037873,2847361.311158709,2784126.5712795486,2720891.8314003856,2657657.0915212235,2594422.3516420606,2531187.611762898,2467952.871883735,2404718.132004574,2341483.39212541,2278248.6522462494,2215013.9123670864,2151779.1724879253,2088544.432608763,2025309.6927296002,1962074.9528504382,1898840.2129712757,1835605.473092114,1772370.7332129516,1709137.0693215402,1645906.2000763128,1582677.7403520222,1519450.5262039471,1456226.5121284465,1393002.9802320749,1329782.4621750668,1266562.178612582,1203344.7170259045,1140127.2554392284,1076912.4497768614,1013697.8601003806,950484.8777540552,887272.9824415045,824061.2602929897,760851.888542112,697642.5167912301,634433.8841165472,571226.8716095625,508019.8591025807,444813.50116399117,381608.68981814245,318403.87847227696,255199.06712642452,191996.26597222174,128793.50369978976,65590.74142736057,2388.5798393534496,-60812.279682949185,-124013.1392052481,-187213.99872754933,-250414.43224125542,-313613.52979415655,-376812.62734705815,-440011.7248999644,-503210.8224528646,-566408.8054032903,-629606.2764382269,-692803.747473171,-756001.2185081057,-819198.6895430498,-882395.3330552224,-945591.3078994174,-1008787.2827436225,-1071983.2575878277,-1135179.2324320227,-1198375.2072762297,-1261570.1861773748,-1324764.7902310388,-1387963.1318088388,-1451159.1066530421,-1514355.0814972408,-1577551.0563414423,-1640747.0311856503,-1703963.2840322256,-1767160.7550671678,-1830358.2261021072,-1893555.6971370466,-1956753.1681719841,-2019989.1637225468,-2083188.2612754554,-2146387.3588283574,-2209586.4563812586],[4046187.8230333864,3982447.6551667647,3918707.487300146,3854967.3194335243,3791227.1515668975,3727486.9837002703,3663746.815833653,3600006.64796703,3536266.480100405,3472526.312233781,3408786.1443671607,3345045.976500537,3281305.808633912,3217565.6407672893,3153825.472900666,3090085.3050340437,3026345.1371674207,2962604.969300798,2898864.801434176,2835124.633567551,2771384.46570093,2707644.297834307,2643904.1299676844,2580163.962101062,2516423.7942344397,2452683.626367815,2388943.458501192,2325203.290634569,2261463.122767946,2197722.9549013227,2133982.7870347,2070242.619168078,2006502.4513014536,1942762.2834348308,1879022.115568208,1815281.947701585,1751541.779834962,1687805.447984613,1624070.6725741713,1560338.1583571676,1496608.0972912044,1432879.090579564,1369153.009754998,1305427.308376852,1241704.5154195402,1177981.9248481952,1114261.7669393483,1050542.0999989754,986823.9618073793,923107.0383900623,859390.3401051974,795675.9868987994,731961.6336924005,668248.2325866874,604536.282813245,540824.3330398118,477113.39383289125,413403.6869970467,349693.9801611919,285984.71587411826,222277.09752118448,158569.47916824976,94861.86081532389,31155.470068123657,-32550.208448447753,-96255.88696502429,-159961.56548160128,-223666.06668141857,-287369.94842373114,-351073.83016604837,-414777.7119083563,-478481.25039338786,-542183.4730541147,-605885.6957148397,-669587.9183755713,-733290.1410362953,-796992.0629383074,-860692.7590474905,-924393.4551566718,-988094.1512658438,-1051794.847375026,-1115495.5434842026,-1179195.5263477704,-1242894.8234716142,-1306597.6318117417,-1370298.3279209174,-1433999.0240300987,-1497699.7201392753,-1561400.4162484445,-1625121.2582864864,-1688823.480947218,-1752525.703607941,-1816227.926268668,-1879930.1489294088,-1943670.873723846,-2007374.7554661501,-2071078.6372084618,-2134782.5189507734,-2198486.400693089,-2262190.2824353976],[4043544.889205315,3979297.4916141164,3915050.0940229157,3850802.696431713,3786555.2988405083,3722307.9012493077,3658060.503658105,3593813.106066905,3529565.7084756987,3465318.3108844985,3401070.9132932997,3336823.515702093,3272576.118110891,3208328.7205196903,3144081.322928488,3079833.9253372857,3015586.5277460837,2951339.1301548807,2887091.7325636805,2822844.334972477,2758596.937381276,2694349.5397900743,2630102.1421988728,2565854.7446076698,2501607.3470164686,2437359.9494252643,2373112.5518340636,2308865.1542428606,2244617.7566516595,2180370.359060456,2116122.9614692545,2051875.5638780533,1987628.1662868499,1923380.7686956483,1859133.3711044462,1794885.9735132442,1730641.1692465113,1666398.1508255962,1602158.5347590323,1537920.5057661538,1473684.4569515386,1409450.6642417677,1345217.97779422,1280987.7105564491,1216758.189293775,1152530.7768475097,1088304.2309934988,1024079.0406927567,959855.2875918611,895631.7228312688,831410.5866742632,767189.4505172558,702969.3965526759,638750.7081128708,574532.019673063,510314.62058785604,446098.2169674444,381881.8133470332,317666.26210462814,253451.98649080656,189237.71087699058,125023.43526316527,60810.9481823761,-3401.35038716672,-67613.64895670908,-131825.91111532738,-196036.37797791278,-260246.84484050423,-324457.31170310033,-388667.7785656918,-452877.102152423,-517085.87723720074,-581294.6523219915,-645503.4274067711,-709712.2024915516,-773919.7572047105,-838126.9752411479,-902334.1932775797,-966541.4113140171,-1030748.6293504443,-1094955.517860731,-1159161.3085790407,-1223367.0992973577,-1287577.5014961828,-1351784.7195326164,-1415991.9375690427,-1480199.1556054773,-1544426.2785937395,-1608635.0536785284,-1672843.8287632996,-1737052.603848082,-1801261.378932871,-1865508.5164052788,-1929718.9832678651,-1993929.4501304645,-2058139.9169930546,-2122350.3838556334,-2186560.850718231,-2250831.8003212577,-2315044.098890803],[4040892.6408102643,3976136.225850313,3911379.8108903607,3846623.395930413,3781866.980970462,3717110.566010511,3652354.1510505616,3587597.7360906117,3522841.321130661,3458084.9061707114,3393328.4912107633,3328572.0762508097,3263815.661290858,3199059.24633091,3134302.8313709632,3069546.416411009,3004790.0014510592,2940033.5864911093,2875277.1715311594,2810520.756571208,2745764.3416112596,2681007.926651309,2616251.511691359,2551495.0967314085,2486738.681771459,2421982.2668115064,2357225.8518515583,2292469.436891607,2227713.0219316576,2162956.606971707,2098200.1920117573,2033443.7770518064,1968687.362091857,1903930.9471319062,1839174.5321719553,1774419.3358489498,1709666.9875889327,1644917.6555975745,1580169.7517790738,1515424.873179769,1450681.4679238822,1385940.008749273,1321200.3727247957,1256462.1331535466,1191725.5777307365,1126990.3654013854,1062256.2405272443,997523.8702505408,932791.5624274786,868061.855909788,803332.1493920949,738603.5718502309,673876.3574275575,609149.143004884,544423.4191030818,479698.5314853387,414973.64386759,350249.94641351234,285527.2264386127,220804.50646371255,156082.05511425622,91361.34951166948,26640.64390908787,-38080.061693503056,-102800.136138923,-167518.9749755324,-232237.81381214783,-296956.65264875814,-361675.32845343696,-426392.4426848334,-491109.556916235,-555826.6711476361,-620543.785379028,-685260.3976320522,-749975.9241836313,-814691.4507352086,-879406.9772867933,-944122.5038383696,-1008838.0303899385,-1073552.2572975755,-1138266.3280614875,-1202984.6100446722,-1267700.1365962503,-1332415.6631478192,-1397131.1896994086,-1461846.7162509803,-1526583.3846186008,-1591300.4988500085,-1656017.613081403,-1720734.7273127995,-1785451.8415442016,-1850208.7847274411,-1914927.6235640552,-1979646.4624006609,-2044365.301237274,-2109084.14007389,-2173802.9789104965,-2238584.052181421,-2303304.7577839997,-2368025.463386598],[4038231.151578391,3972963.9457557625,3907696.73993313,3842429.5341104963,3777162.328287861,3711895.1224652273,3646627.9166425974,3581360.710819965,3516093.5049973335,3450826.299174701,3385559.093352067,3320291.8875294337,3255024.6817068015,3189757.4758841693,3124490.270061536,3059223.064238905,2993955.8584162714,2928688.6525936387,2863421.44677101,2798154.240948375,2732887.0351257436,2667619.8293031096,2602352.6234804792,2537085.417657845,2471818.211835214,2406551.0060125813,2341283.8001899486,2276016.594367316,2210749.3885446833,2145482.1827220507,2080214.9768994194,2014947.7710767859,1949680.565254153,1884413.3594315206,1819146.1536088889,1753882.791822444,1688621.467052408,1623362.2983715963,1558106.3136911574,1492851.409261052,1427599.4038496413,1362348.518503806,1297099.7871534515,1231852.214230009,1166606.562008985,1101361.633960483,1036118.8731596442,970876.1123588055,905635.909357874,840395.8592117731,775156.9011477251,709919.3875664445,644681.8739851597,579445.9740641937,514210.8293754994,448975.6846867972,383741.9954681592,318509.0581695442,253276.1208709227,188043.84229105804,122812.95681135077,57582.07133164536,-7648.814148067497,-72878.53532801056,-138107.51885789912,-203336.50238779,-268565.48591767903,-333793.6511664493,-399020.8771339576,-464248.1031014682,-529475.3290689779,-594702.5550364908,-659928.4913378316,-725154.0988607025,-790379.706383571,-855605.313906448,-920830.9214293193,-986055.7892426588,-1051279.9123727707,-1116504.0355028585,-1181733.3515208075,-1246958.9590436798,-1312184.566566551,-1377410.1740894252,-1442656.4926141044,-1507883.718581616,-1573110.9445491228,-1638338.1705166334,-1703565.3964841375,-1768832.1071051434,-1834061.090635037,-1899290.0741649289,-1964519.057694817,-2029748.0412247004,-2094977.024754594,-2160268.034978451,-2225498.9204581548,-2290729.8059378657,-2355960.691417573,-2421191.5768972887],[4035560.495522327,3969780.7395475246,3904000.983572718,3838221.227597912,3772441.4716231055,3706661.7156483,3640881.9596734936,3575102.2036986873,3509322.447723883,3443542.6917490764,3377762.935774271,3311983.179799464,3246203.4238246568,3180423.667849851,3114643.911875046,3048864.15590024,2983084.3999254326,2917304.643950629,2851524.887975823,2785745.132001016,2719965.3760262104,2654185.620051404,2588405.8640765995,2522626.108101792,2456846.352126987,2391066.5961521785,2325286.8401773744,2259507.084202568,2193727.328227762,2127947.5722529553,2062167.8162781503,1996388.0603033449,1930608.3043285382,1864828.5483537319,1799051.063269494,1733275.860884529,1667504.2697925603,1601734.6986057512,1535966.8540417575,1470202.1115089024,1404438.1103380865,1338677.1279390666,1272916.6769660043,1207158.8256366018,1141401.2397366262,1075646.3292614892,1009891.4187863504,944138.8080349052,878386.6551892937,812635.4734926708,746885.9017720781,681136.3300514859,615388.4167055236,549641.2560656769,483894.09542582696,418148.58265408454,352403.66926093725,286658.75586779043,220914.8237889898,155171.99977834662,89429.17576769507,23686.35175705701,-42054.83689033007,-107795.72364399256,-173536.61039764294,-239277.4971513045,-305016.96715005254,-370756.06325582555,-436495.1593615995,-502234.255467372,-567972.9750292846,-633710.4217935866,-699447.86855789,-765185.315322197,-830922.7620864986,-896660.132720938,-962396.0663530212,-1028131.9999851054,-1093872.5491437158,-1159609.9959080163,-1225347.4426723206,-1291084.889436625,-1356822.3362009274,-1422581.6009481978,-1488320.6970539587,-1554059.7931597345,-1619798.8892655075,-1685577.0057317745,-1751317.8924854323,-1817058.7792390948,-1882799.6659927554,-1948540.5527464105,-2014281.439500073,-2080084.016583561,-2145826.840594206,-2211569.6646048585,-2277312.4886154905,-2343055.3126261383,-2408798.1366367824,-2474540.9606474265],[4032880.7469212115,3966586.695760332,3900292.644599456,3833998.5934385764,3767704.542277698,3701410.491116815,3635116.439955935,3568822.3887950564,3502528.337634182,3436234.2864732984,3369940.235312422,3303646.1841515405,3237352.1329906615,3171058.0818297844,3104764.0306689045,3038469.979508024,2972175.928347145,2905881.877186267,2839587.8260253877,2773293.7748645074,2706999.7237036303,2640705.67254275,2574411.6213818714,2508117.5702209915,2441823.5190601135,2375529.467899232,2309235.4167383537,2242941.365577475,2176647.3144165957,2110353.263255716,2044059.2120948378,1977765.1609339584,1911471.1097730794,1845177.6246044035,1778887.8359207958,1712600.915462467,1646315.8933395338,1580033.9410529668,1513754.025543214,1447475.6639887418,1381200.0776660582,1314925.0992009512,1248653.10672281,1182381.3113191416,1116112.217225317,1049843.4121750833,983576.5587931809,917310.558424796,851045.3238776252,784781.9492832669,718518.5746889082,652256.8243667795,585995.9031402352,519734.9819136881,453475.8276224262,387217.19360699365,320958.5595915634,254701.16154033458,188444.6545864381,122188.14763253927,55931.711870839354,-10322.822397082113,-76577.35666500917,-142831.89093292924,-209086.2910930151,-275339.001500641,-341591.7119082683,-407844.4223158895,-474097.1327235182,-540348.8133605421,-606599.8433991801,-672850.8734378247,-739101.9034764655,-805352.93351511,-871603.1135458564,-937852.6015797276,-1004102.0896135904,-1070357.0536696734,-1136608.083708318,-1202859.113746957,-1269110.1437856006,-1335382.3680226374,-1401635.0784302661,-1467887.7888378818,-1534140.4992455132,-1600393.2096531289,-1666686.1790952021,-1732940.7133631324,-1799195.24763106,-1865449.7818989828,-1931704.3161668992,-1997958.8504348248,-2064276.5818461236,-2130533.088800014,-2196789.5957539156,-2263046.102707818,-2329302.609661718,-2395559.116615613,-2461904.069056567,-2528162.703072007],[4030191.9803048694,3963381.903227725,3896571.826150577,3829761.749073433,3762951.6719962885,3696141.5949191367,3629331.517841992,3562521.440764848,3495711.3636876997,3428901.286610554,3362091.209533409,3295281.1324562593,3228471.0553791113,3161660.978301966,3094850.901224823,3028040.824147676,2961230.7470705295,2894420.6699933815,2827610.5929162344,2760800.515839088,2693990.438761942,2627180.3616847964,2560370.2846076493,2493560.2075305027,2426750.1304533565,2359940.053376208,2293129.976299063,2226319.8992219167,2159509.82214477,2092699.7450676225,2025889.6679904775,1959079.5909133297,1892269.5138361831,1825462.4938645316,1758658.0314544244,1691856.3746041749,1625057.9174444724,1558260.9847873708,1491466.8418814065,1424674.5036101183,1357883.7983617974,1291095.3009282781,1224307.8307834158,1157522.461045019,1090738.0308164237,1023955.1139125021,957173.5354940835,890392.4328256964,823613.5249175495,756834.6170094004,690057.2200694755,623280.8079135171,556504.3957575578,489729.79585727956,422955.71098305145,356181.62610882195,289408.9634612459,222637.04344146093,155865.1234216774,89093.58454766171,22323.672763145063,-44446.23902135715,-111216.15080586588,-177985.47834733222,-244753.53293364355,-311521.5875199586,-378289.6421062662,-445057.6966925794,-511824.12086704466,-578590.4639275018,-645356.8069879748,-712123.1500484431,-778889.4876628127,-845654.2597140623,-912419.0317653231,-979183.8038165728,-1045954.8653507689,-1112721.2084112437,-1179487.551471701,-1246253.894532171,-1313042.4063146226,-1379810.4609009326,-1446578.515487248,-1513346.5700735543,-1580154.2100650351,-1646924.121849548,-1713694.033634062,-1780463.9454185637,-1847233.8572030645,-1914003.7689875718,-1980836.3172114678,-2047608.2372312602,-2114380.1572510377,-2181152.0772708263,-2247923.997290618,-2314695.917310402,-2381467.837330186,-2448329.938608842,-2515104.0234830678,-2581878.108357299],[4027494.2704380546,3960166.4510632344,3892838.6316884207,3825510.8123135976,3758182.9929387826,3690855.173563957,3623527.3541891426,3556199.534814324,3488871.7154395035,3421543.896064684,3354216.076689866,3286888.257315045,3219560.437940228,3152232.6185654104,3084904.799190593,3017576.9798157727,2950249.160440955,2882921.341066136,2815593.5216913177,2748265.702316498,2680937.8829416796,2613610.0635668617,2546282.2441920424,2478954.4248172236,2411626.605442405,2344298.7860675855,2276970.966692767,2209643.1473179483,2142315.32794313,2074987.5085683102,2007659.6891934937,1940331.8698186749,1873005.2045860263,1805681.7780447975,1738362.08680591,1671044.5524565117,1603729.206753423,1536417.132065535,1469106.2177953115,1401798.0693920588,1334491.248757998,1267186.3875491219,1199882.9296552697,1132581.15798771,1065280.3707102544,997981.4980556942,930682.7259945283,863386.5686721469,796090.4113497678,728795.5721118096,661501.9530217159,594208.3339316249,526916.4982614634,459625.2466283296,392333.99499519076,325044.28302844986,257755.23415499274,190466.1852815356,123177.7639190983,55890.7589490409,-11396.246021018364,-78683.25099108927,-145969.27992106648,-213254.39423098182,-280539.50854089716,-347824.6228508097,-415109.3019885896,-482392.673487789,-549676.0449869935,-616959.4164861944,-684242.7879853966,-751525.5046477653,-818807.2760026879,-886089.0473575871,-953376.2739821924,-1020659.6454814011,-1087943.016980593,-1155226.3884797934,-1222531.1088796929,-1289816.2231896063,-1357101.337499517,-1424386.451809438,-1491671.5661193365,-1558997.3603324015,-1626284.3653024621,-1693571.3702725144,-1760858.375242577,-1828145.3802126367,-1895494.3297956092,-1962783.3786690636,-2030072.427542517,-2097361.476415978,-2164650.525289436,-2231939.5741628883,-2299228.6230363455,-2366607.321963393,-2433898.573596524,-2501189.8252296625,-2568481.076862799,-2635772.328495929],[4024787.692304872,3956940.428641865,3889093.164978861,3821245.9013158535,3753398.637652849,3685551.3739898386,3617704.110326835,3549856.8466638327,3482009.5830008266,3414162.31933782,3346315.055674816,3278467.7920118077,3210620.528348802,3142773.2646857975,3074926.0010227943,3007078.737359788,2939231.473696781,2871384.2100337762,2803536.946370772,2735689.6827077647,2667842.4190447596,2599995.1553817554,2532147.8917187518,2464300.628055745,2396453.364392739,2328606.100729733,2260758.8370667268,2192911.5734037226,2125064.3097407166,2057217.0460777106,1989369.7824147062,1921522.5187517009,1853678.832724666,1785837.9926325511,1717999.5595906153,1650164.9569307356,1582332.2767252875,1514501.4362862548,1446673.6831002277,1378846.7352038075,1311022.7809943357,1243199.4365680323,1175378.6215857351,1107558.1711718903,1039740.3024783959,971922.4337848993,904106.9653454963,836291.856890196,768477.7936841976,700665.2660354143,632852.738386632,565041.8907631249,497231.77084955527,429421.65093599167,361613.1188768926,293805.2397390092,225997.36060113413,158190.2913013245,90384.49185226671,22578.69240321312,-45227.10704583628,-113031.59746285994,-180835.4726671949,-248639.34787152614,-316443.2230758681,-384246.2004984785,-452048.30148056336,-519850.40246265335,-587652.5034447429,-655454.6044268254,-723255.4497187724,-791055.9212920787,-858856.3928653849,-926663.0083551835,-994465.1093372721,-1062267.2103193616,-1130069.311301441,-1197893.6007322315,-1265697.475936558,-1333501.3511408903,-1401305.226345227,-1469109.1015495677,-1536954.694924986,-1604760.4943740387,-1672566.2938230913,-1740372.093272142,-1808177.8927211994,-1876046.892673105,-1943854.7718109954,-2011662.6509488681,-2079470.5300867502,-2147278.4092246285,-2215086.288362502,-2282983.145606611,-2350793.2655201657,-2418603.38543374,-2486413.505347303,-2554223.6252608746,-2622033.745174432,-2689843.865088001],[4022072.321093297,3953703.925581604,3885335.530069916,3816967.134558221,3748598.739046527,3680230.3435348375,3611861.9480231428,3543493.5525114536,3475125.1569997678,3406756.7614880726,3338388.3659763806,3270019.9704646883,3201651.5749529963,3133283.1794413067,3064914.7839296153,2996546.388417923,2928177.992906232,2859809.597394542,2791441.20188285,2723072.806371158,2654704.4108594675,2586336.015347776,2517967.619836086,2449599.224324392,2381230.828812701,2312862.4333010083,2244494.037789318,2176125.642277627,2107757.246765935,2039388.8512542425,1971020.4557425524,1902653.5370827033,1834289.7351265391,1765930.1425046641,1697573.0490616115,1629217.962064273,1560866.2342601856,1492516.354397531,1424168.2281585936,1355822.7546838366,1287478.0199626056,1219136.1937784564,1150794.6480296408,1082455.749486941,1014117.1973742151,945780.5441770125,877444.7972910504,809109.7425030856,740776.6190904239,672443.4956777622,604111.8739406099,535781.1986428848,467450.52334515564,399121.4144157078,330793.0180188981,262464.6216220814,194137.1529083373,125810.87210159888,57484.5912948465,-10841.68951189937,-79166.38756097434,-147490.71041724132,-215815.03327351296,-284139.3561297837,-352462.3727443656,-420784.88984153746,-489107.4069387056,-557429.9240358719,-625752.292408898,-694073.1507046311,-762394.0090003647,-830714.8672961025,-899042.5095217219,-967365.0266188933,-1035687.54371606,-1104010.0608132258,-1172355.5532612847,-1240679.8761175573,-1309004.198973827,-1377328.521830095,-1445693.586453571,-1514019.867260322,-1582346.1480670804,-1650672.428873825,-1718998.7096805759,-1787324.9904873138,-1855715.6666791448,-1924044.0630759625,-1992372.459472768,-2060700.85586958,-2129029.2522664014,-2197357.648663209,-2265776.4885639083,-2334107.1638616305,-2402437.839159362,-2470768.51445709,-2539099.1897548204,-2607429.865052535,-2675760.5403502733,-2744212.674955412],[4019348.2321797875,3950457.031725099,3881565.831270414,3812674.6308157276,3743783.4303610376,3674892.229906352,3606001.0294516613,3537109.8289969745,3468218.628542288,3399327.4280875986,3330436.227632912,3261545.0271782232,3192653.826723534,3123762.6262688497,3054871.4258141625,2985980.225359475,2917089.0249047843,2848197.8244500975,2779306.623995411,2710415.4235407226,2641524.2230860367,2572633.022631348,2503741.822176661,2434850.6217219727,2365959.4212672855,2297068.2208125964,2228177.0203579096,2159285.819903222,2090394.6194485342,2021503.4189938465,1952612.2185391593,1883724.8509253561,1814840.6308156196,1745959.0986810364,1677080.9337514313,1608205.529699233,1539331.741980516,1470461.1099504363,1401591.9960457408,1332724.8077632089,1263859.5906756825,1194995.6411688,1126133.5838268118,1057272.6753733007,988413.0785059151,919555.0203487407,850697.2204638631,781841.82853775,712986.436611637,644132.2926665316,575279.3893349511,506426.4860033812,437575.0574634271,368724.47126566805,299873.8850679016,231024.27947840607,162175.8448862643,93327.41029411415,24478.975701969117,-44367.662128740456,-113214.1049448885,-182060.54776104307,-250906.99057719624,-319751.7733092047,-388596.3787054266,-457440.98410165636,-526285.5894978829,-595129.573347799,-663972.4904230288,-732815.4074982451,-801658.3245734759,-870508.6164790103,-939353.221875241,-1008197.8272714652,-1077064.3043710236,-1145910.7471871767,-1214757.1900033234,-1283603.6328194747,-1352450.0756356306,-1421338.1507330975,-1490186.5853252448,-1559035.0199173884,-1627883.4545095423,-1696731.8891016915,-1765643.7008650163,-1834494.2870627819,-1903344.8732605474,-1972195.4594583046,-2041046.04565607,-2109896.631853844,-2178836.7439280692,-2247689.647259621,-2316542.5505912155,-2385395.453922795,-2454248.35725436,-2523101.260585945,-2591954.163917519,-2660807.067249097,-2729783.159693987,-2798638.5516201025],[4016615.5011140956,3947199.8371215505,3877784.1731290035,3808368.5091364556,3738952.845143905,3669537.1811513547,3600121.5171588096,3530705.8531662608,3461290.189173715,3391874.525181168,3322458.861188621,3253043.1971960687,3183627.5332035213,3114211.869210973,3044796.2052184273,2975380.54122588,2905964.877233331,2836549.2132407846,2767133.5492482353,2697717.8852556874,2628302.221263141,2558886.557270593,2489470.8932780456,2420055.229285497,2350639.5652929503,2281223.901300401,2211808.237307854,2142392.5733153056,2072976.9093227577,2003561.2453302098,1934147.115820454,1864736.4137279142,1795329.8074921006,1725926.2234228642,1656524.9104278549,1587126.403013875,1517730.651613173,1448336.1360673222,1378944.8356688817,1309554.472019054,1240166.4602560964,1170779.4939483139,1101394.5707245236,1032010.2854494529,962628.2576710354,893246.2298926166,823866.6546130306,754487.3359128521,685108.9357713461,615732.1462436207,546355.3567158938,476979.87990282616,407605.4458479369,338231.0117930495,268857.54592013545,199485.29959922563,130113.05327831767,60740.80695740692,-8629.48877002811,-77999.70937109692,-147369.92997216992,-216740.06093012495,-286108.41232782556,-355476.76372552617,-424845.1151232249,-494213.4665209218,-563580.7736406084,-632947.407072071,-702314.0405035331,-771686.8721117293,-841055.2235094216,-910423.5749071287,-979791.9263048247,-1049182.7977861352,-1118553.018387218,-1187923.238988285,-1257293.4595893603,-1326663.680190429,-1396076.3657816714,-1465448.6121025775,-1534820.8584234929,-1604193.104744411,-1673565.3510653144,-1743002.0098536033,-1812376.4439085005,-1881750.8779633865,-1951125.3120182771,-2020499.7460731566,-2089874.18012805,-2159339.4348654365,-2228716.224393161,-2298093.0139208864,-2367469.803448613,-2436846.5929763345,-2506223.38250407,-2575600.1720317965,-2645099.280395873,-2714478.59909606,-2783857.9177962383,-2853237.2364964066],[4013874.2036041208,3943932.4320086506,3873990.6604131786,3804048.888817706,3734107.1172222327,3664165.3456267584,3594223.5740312864,3524281.8024358135,3454340.030840343,3384398.2592448685,3314456.4876493984,3244514.7160539213,3174572.94445845,3104631.172862977,3034689.401267508,2964747.629672035,2894805.8580765612,2824864.086481088,2754922.3148856186,2684980.543290143,2615038.7716946723,2545097.0000992,2475155.2285037283,2405213.456908256,2335271.6853127833,2265329.9137173095,2195388.1421218375,2125446.370526365,2055504.598930892,1985562.82733542,1915624.8788235537,1845690.3922776708,1775758.8874006914,1705830.525656391,1635905.536808158,1565982.4130636507,1496061.8081827979,1426143.6924652127,1356226.6359215877,1286312.722896872,1216399.1117086865,1146488.5298075872,1076577.9479064902,1006670.1843704337,936762.5431416873,866856.8011240978,796951.911908621,727047.5357251125,657145.2163567534,587242.8969883993,517341.65735394787,447441.73190204054,377541.8064501332,307642.77097683307,237745.06949857948,167847.36802033195,97949.6665420807,28054.008873494342,-41841.63282419043,-111737.27452187566,-181632.61525625503,-251526.35584584996,-321420.09643543884,-391313.83702503145,-461207.57761461986,-531099.9017499886,-600991.8946036939,-670883.8874574085,-740782.5399729894,-810676.280562581,-880570.0211521694,-950463.7617417537,-1020380.6165917707,-1090276.2582894554,-1160171.8999871463,-1230067.5416848352,-1300004.363022931,-1369902.0645011766,-1439799.765979427,-1509697.4674576884,-1579595.1689359387,-1649492.870414175,-1719455.957107042,-1789355.8825589595,-1859255.808010866,-1929155.733462776,-1999055.6589146787,-2068955.5843665786,-2138947.558377506,-2208849.877745866,-2278752.1971142106,-2348654.5164825693,-2418556.835850929,-2488459.1552192904,-2558361.4745876426,-2628387.6596496385,-2698292.5488651115,-2768197.4380806005,-2838102.3272960708,-2908007.216511553],[4011124.4155010213,3940654.906794857,3870185.3980886876,3799715.8893825193,3729246.380676348,3658776.8719701795,3588307.36326401,3517837.854557843,3447368.345851676,3376898.8371455125,3306429.3284393433,3235959.8197331754,3165490.3110270044,3095020.8023208375,3024551.293614673,2954081.7849085024,2883612.276202336,2813142.7674961644,2742673.2587899966,2672203.750083829,2601734.241377663,2531264.7326714937,2460795.223965326,2390325.715259158,2319856.20655299,2249386.6978468206,2178917.189140654,2108447.680434486,2037978.1717283172,1967509.990423138,1897045.559961386,1826585.3636806621,1756128.4733467104,1685674.131251567,1615222.267852444,1544773.8543319881,1474326.90997136,1403882.4678061332,1333439.989791112,1262998.9361452926,1192560.1421582415,1122122.2722228502,1051686.4187771063,981251.5348174542,910817.9151581584,840385.8262331164,769953.768338948,699524.2900308855,629094.8117228211,558666.1088670082,488239.06302205333,417812.0171771003,347385.7169108172,276960.9313890189,206536.1458672122,136111.36034541158,65688.65079522785,-4734.040769823361,-75156.73233486898,-145578.9677258255,-215999.72615814488,-286420.4845904559,-356841.24302277016,-427262.0014550863,-497681.0222727298,-568100.0030763587,-638518.9838800002,-708945.0351843527,-779365.7936166641,-849786.5520489803,-920207.3104812922,-990651.7226804863,-1061074.414245532,-1131497.1058105836,-1201919.7973756324,-1272384.350090648,-1342809.1356124468,-1413233.921134254,-1483658.70665605,-1554083.4921778487,-1624572.3123265374,-1694999.3581714872,-1765426.404016438,-1835853.4498613896,-1906280.4957063356,-1976707.5415512929,-2047225.3639836265,-2117654.8422916876,-2188084.320599761,-2258513.798907822,-2328943.277215888,-2399372.755523951,-2469802.2338320212,-2540354.442168979,-2610786.531094025,-2681218.620019066,-2751650.708944112,-2822082.7978691505,-2892514.886794202,-2962946.975719247],[4008366.212784385,3937367.3520416874,3866368.4912989945,3795369.6305563045,3724370.769813609,3653371.909070908,3582373.0483282167,3511374.1875855215,3440375.326842827,3369376.4661001316,3298377.6053574374,3227378.7446147404,3156379.8838720475,3085381.023129353,3014382.162386661,2943383.3016439644,2872384.440901272,2801385.580158577,2730386.7194158817,2659387.8586731856,2588388.997930493,2517390.137187798,2446391.276445104,2375392.4157024096,2304393.554959715,2233394.6942170183,2162395.8334743246,2091396.97273163,2020398.1119889356,1949402.7999780818,1878411.2769969702,1807423.0418693242,1736437.965098516,1665456.646375549,1594477.4538364597,1523500.358058889,1452526.5201021864,1381553.7644754776,1310583.7603305702,1239614.940204602,1168648.1674526692,1097682.5221694822,1026718.7807715859,955755.5868066994,884794.6835495019,813833.7802923024,742875.0007074936,671916.7489307784,600958.8966330118,530003.115967588,459047.3353021564,388092.08920551836,317138.60532106925,246185.12143660802,175231.63755214913,104280.200314424,33328.84467693465,-37622.510960548185,-108573.31176789617,-179522.70212944783,-250472.09249099717,-321421.4828525516,-392370.6735535171,-463318.25627169944,-534265.8389898739,-605213.4217080553,-676168.4346603183,-747117.8250218709,-818067.2153834235,-889016.6057449752,-959990.1342478851,-1030941.4898853754,-1101892.845522869,-1172844.2011603545,-1243838.0401369892,-1314791.5240214486,-1385745.0079059051,-1456698.4917903598,-1527651.9756748127,-1598670.3039952368,-1669626.0846606698,-1740581.8653260898,-1811537.6459915126,-1882493.426656955,-1953449.2073223796,-2024496.818584416,-2095455.0703611271,-2166413.3221378373,-2237371.573914554,-2308329.8256912697,-2379288.077467993,-2450246.329244704,-2521328.6727960072,-2592289.5760532096,-2663250.479310409,-2734211.3825676097,-2805172.285824798,-2876133.189082006,-2947094.0923392046,-3018054.9955964014],[4005599.671547538,3934069.8584462632,3862540.0453449837,3791010.232243698,3719480.419142418,3647950.606041138,3576420.7929398557,3504890.9798385743,3433361.166737296,3361831.353636014,3290301.540534733,3218771.7274334505,3147241.9143321672,3075712.1012308896,3004182.288129613,2932652.47502833,2861122.661927048,2789592.8488257676,2718063.0357244867,2646533.222623205,2575003.4095219257,2503473.596420645,2431943.783319364,2360413.970218083,2288884.1571168024,2217354.3440155205,2145824.5309142405,2074294.7178129596,2002765.760688338,1931240.8902622368,1859720.6440528713,1788203.7481762245,1716689.689978763,1645178.0107539431,1573670.2603782667,1502164.224557275,1430660.181051726,1359158.9381915384,1287658.399021959,1216161.1232678888,1144663.9985234265,1073169.7995750827,1001675.6006267427,930184.0312906306,858692.7136722922,787202.9246480942,715714.2994658356,644225.6742835776,572739.5103903813,501253.3950675749,429767.5362864593,358283.7543092803,286799.97233210085,215316.1903549214,143834.36378588155,72352.7444584826,871.1251310813241,-70609.89770483086,-142089.51949589327,-213569.14128694357,-285048.76307800226,-356527.9975761995,-428005.7815885595,-499483.5656009144,-570961.3496132684,-642446.8720332771,-713926.4938243274,-785406.1156153814,-856885.737406441,-928389.9261251497,-999871.5454525594,-1071353.1647799648,-1142834.7841073633,-1214359.4491886888,-1285843.2311658673,-1357327.013143045,-1428810.7951202262,-1500294.577097416,-1571843.9492938174,-1643330.0646166233,-1714816.179939434,-1786302.295262226,-1857788.4105850402,-1929274.525907847,-2000853.4574599136,-2072342.0826421725,-2143830.7078244416,-2215319.333006696,-2286807.9581889473,-2358296.58337121,-2429785.208553456,-2501399.2143896557,-2572890.5320080034,-2644381.8496263316,-2715873.1672446746,-2787364.4848630074,-2858855.802481341,-2930347.120099687,-3001838.4377180357,-3073493.7394287745],[4002824.8679830953,3930762.516823994,3858700.1656648903,3786637.814505788,3714575.4633466857,3642513.1121875797,3570450.7610284765,3498388.409869372,3426326.058710273,3354263.7075511683,3282201.356392063,3210139.005232961,3138076.654073855,3066014.3029147545,2993951.9517556513,2921889.6005965457,2849827.2494374444,2777764.8982783398,2705702.547119236,2633640.195960132,2561577.8448010315,2489515.493641926,2417453.142482823,2345390.791323721,2273328.4401646177,2201266.089005512,2129203.7378464094,2057141.3866873072,1985082.0456856985,1913026.8331002777,1840975.2267758935,1768927.0334947389,1696882.7561183292,1624840.878073255,1552800.9552699975,1480764.725263691,1408729.8728552246,1336697.1815844683,1264666.5717799624,1192637.2074539103,1120610.0522831194,1048583.810286392,976559.3594182956,904536.04202256,832513.5806878686,760492.9967743738,688472.4128608806,616453.8085796372,544435.7733722175,472417.73816479417,400401.98406922445,328386.3188774311,256370.653685641,184356.7903286605,112343.32230061851,40329.85427257698,-31683.033031950705,-103694.47114737704,-175705.9092628071,-247717.34737823065,-319728.2630552333,-391737.833137278,-463747.40321932035,-535756.973301352,-607774.5379553656,-679785.976070792,-751797.4141862234,-823808.8523016456,-895845.2300919807,-967858.6981200213,-1039872.1661480647,-1111885.6341761062,-1183942.6501501873,-1255958.315341995,-1327973.9805337721,-1399989.645725565,-1472005.3109173635,-1544087.2476429949,-1616105.2828504108,-1688123.318057824,-1760141.353265252,-1832159.3884726735,-1904268.6080249203,-1976289.1919384077,-2048309.77585191,-2120330.3597653983,-2192350.943678893,-2264371.527592403,-2336392.1115058856,-2408536.558181192,-2480559.875576929,-2552583.192972659,-2624606.5103683993,-2696629.827764133,-2768653.1451598704,-2840676.462555594,-2912699.779951332,-2984885.7415305423,-3056911.983527271,-3128938.2255240083],[4000041.8783685546,3927445.4180915128,3854848.957814467,3782252.4975374206,3709656.0372603796,3637059.5769833317,3564463.116706285,3491866.6564292414,3419270.1961521977,3346673.73587515,3274077.2755981064,3201480.8153210585,3128884.3550440143,3056287.8947669715,2983691.434489924,2911094.974212881,2838498.513935834,2765902.0536587895,2693305.593381745,2620709.133104697,2548112.6728276536,2475516.2125506094,2402919.7522735633,2330323.2919965186,2257726.8317194725,2185130.371442427,2112533.9111653822,2039937.5714724676,1967346.1538355304,1894759.0273650715,1822175.5434037521,1749595.1985860206,1677017.5129153617,1604443.7148505943,1531872.0409360798,1459302.0533830035,1386735.5109905538,1314169.8873278773,1241606.8634921191,1169044.9422175041,1096485.0863050784,1023926.1002001357,951369.2122398657,878812.4300070887,806258.3166642385,733704.2033213882,661151.5152251204,588599.9895319054,516048.46383869136,443498.8427723069,370949.72386858193,298400.6049648626,225853.07157260925,153306.18445634516,80759.29734008061,8212.917320856359,-64331.907392030116,-136876.73210491193,-209421.55681779282,-281965.77687282534,-354508.70317953033,-427051.62948623765,-499594.5557929487,-572145.6803822131,-644690.5050950921,-717235.3298079749,-789780.1545208553,-862350.2351713632,-934897.1222876254,-1007444.0094038895,-1079990.8965201546,-1152581.7731095287,-1225130.8920132462,-1297680.0109169688,-1370229.129820685,-1442778.2487244066,-1515394.2555712974,-1587945.781264525,-1660497.3069577264,-1733048.8326509353,-1805600.35834416,-1878243.8770212708,-1950797.9903641278,-2023352.1037069857,-2095906.217049825,-2168460.3303926783,-2241014.443735526,-2313568.5570783764,-2386247.633932622,-2458804.5218928913,-2531361.4098531557,-2603918.297813434,-2676475.1857736977,-2749032.073733965,-2821588.9616942443,-2894145.8496545153,-2966866.8447908284,-3039426.7007032586,-3111986.5566156795,-3184546.4125281125],[3997250.7790521896,3924118.6532497853,3850986.52744738,3777854.401644971,3704722.275842565,3631590.150040155,3558458.0242377436,3485325.898435339,3412193.772632931,3339061.646830523,3265929.5210281187,3192797.395225707,3119665.2694232985,3046533.143620895,2973401.0178184872,2900268.892016077,2827136.76621367,2754004.6404112615,2680872.514608854,2607740.3888064446,2534608.2630040385,2461476.1372016296,2388344.0113992225,2315211.885596816,2242079.7597944075,2168947.633991998,2095815.508189591,2022685.5901237018,1949560.1519155086,1876438.6503410628,1803321.0560989815,1730207.3079295508,1657096.2440705989,1583987.4009337737,1510882.0994364107,1437778.673480973,1364677.0111077935,1291578.1466471616,1218479.907009741,1145384.788716508,1072289.7626653668,999197.6133360006,926105.5986668284,853015.6770920935,779926.4782630149,706838.077199101,633751.5050586481,560664.9329181942,487579.82081956137,414495.69234484853,341411.5638701366,268328.7414240837,195246.8794688452,122165.01751359273,49083.530746617354,-23996.2362013394,-97076.0031492915,-170155.7700972492,-243234.90352852456,-316312.7415806805,-389390.5796328401,-462468.4176849965,-535554.6048370246,-608634.3717849767,-681714.138732933,-754793.9056808897,-827899.1879045982,-900981.0498598381,-974062.9118150859,-1047144.77377033,-1120271.0056240885,-1193355.134098799,-1266439.262573517,-1339523.3910482218,-1412672.514874015,-1485759.087014473,-1558845.6591549246,-1631932.231295377,-1705018.8034358313,-1778105.3755762838,-1851284.6795838568,-1924373.8784129368,-1997463.0772420159,-2070552.2760710958,-2143641.4749001833,-2216730.673729251,-2289819.872558333,-2363035.0614461135,-2436127.0761152795,-2509219.0907844603,-2582311.105453642,-2655403.1201228173,-2728495.1347919926,-2801587.149461164,-2874679.164130344,-2947936.670147322,-3021031.696198469,-3094126.722249601,-3167221.748300747,-3240316.7743518874],[3994451.6464389167,3920782.313367301,3847112.980295677,3773443.6472240584,3699774.3141524317,3626104.9810808096,3552435.648009191,3478766.3149375725,3405096.9818659453,3331427.6487943274,3257758.3157227035,3184088.982651077,3110419.6495794575,3036750.316507837,2963080.9834362157,2889411.650364593,2815742.3172929706,2742072.98422135,2668403.6511497274,2594734.3180781044,2521064.985006483,2447395.651934861,2373726.31886324,2300056.985791619,2226387.6527199964,2152718.3196483743,2079048.986576752,2005383.947307634,1931723.4334575315,1858066.8958918366,1784413.810714791,1710763.6778913406,1637117.605075576,1563473.862754119,1489831.9128589407,1416193.44506709,1342556.3298054398,1268921.3327311012,1195288.2194502498,1121656.4816926196,1048026.5762469566,974397.89337824,900770.4379671433,827144.6122471753,753518.8833546089,679895.723456176,606272.563557738,532650.3506318298,459029.67137628235,385408.99212073814,311789.2758479342,238170.89795074984,164552.52005355852,90934.32673522504,17318.076561026275,-56298.17361317482,-129914.42378737591,-203530.06537616998,-277144.3560493882,-350758.6467226106,-424372.93739582924,-497995.6746583767,-571611.9248325741,-645228.1750067733,-718844.4251809726,-792486.3926098687,-866104.7705070544,-939723.1484042373,-1013341.5263014254,-1087004.5929901432,-1160625.2722456949,-1234245.9515012344,-1307866.6307567852,-1381552.7537000123,-1455175.9135984462,-1528799.073496881,-1602422.2333953185,-1676045.3932937495,-1749668.553192189,-1823385.113671665,-1897010.9393916316,-1970636.765111602,-2044262.590831576,-2117888.416551534,-2191514.24227151,-2265264.152845348,-2338892.8357140617,-2412521.518582777,-2486150.201451488,-2559778.8843202144,-2633407.5671889326,-2707036.2500576526,-2780664.932926368,-2854457.357219546,-2928089.0949771786,-3001720.8327348037,-3075352.5704924427,-3148984.308250077,-3222616.0460077096,-3296247.7837653467],[3991644.55697652,3917436.489563633,3843228.4221507446,3769020.3547378546,3694812.2873249627,3620604.2199120726,3546396.1524991808,3472188.0850862935,3397980.0176734026,3323771.9502605093,3249563.8828476216,3175355.8154347334,3101147.7480218397,3026939.680608953,2952731.6131960624,2878523.545783172,2804315.478370282,2730107.4109573904,2655899.3435445013,2581691.276131611,2507483.2087187218,2433275.141305831,2359067.0738929403,2284859.0064800503,2210650.9390671602,2136442.8716542698,2062235.9460901427,1988033.8636238917,1913836.059881154,1839642.8974784731,1765453.2832663893,1691266.6499468957,1617082.514342844,1542901.9744102927,1468723.6143754064,1394546.8141390102,1320373.3460785723,1246200.6221654804,1172030.6457971178,1097861.3736613349,1023694.4957644825,949527.9009742155,875363.9216192313,801199.9422642456,727037.9900351302,652876.7157269944,578715.8064595936,504557.04987093154,430398.2932822611,356240.0926694814,282083.6723835175,207927.25209755963,133770.83181160083,59616.50693064323,-14537.752806208562,-88692.0125430501,-162845.74284034641,-236998.01235673716,-311150.2818731298,-385302.5513895233,-459463.31122728717,-533617.5709641324,-607771.8307009726,-681926.0904378267,-756106.211619922,-830262.6319058882,-904419.0521918451,-978575.4724778091,-1052776.8384910673,-1126935.595079735,-1201094.351668396,-1275253.108257071,-1349477.690592683,-1423638.964900813,-1497800.239208946,-1571961.5135170883,-1646122.7878252147,-1720284.062133357,-1794539.3351602275,-1868703.3145152098,-1942867.2938701967,-2017031.273225177,-2091195.2525801696,-2165359.23193515,-2239648.1316969953,-2313815.0095938463,-2387981.8874906907,-2462148.765387548,-2536315.6432844047,-2610482.521181248,-2684649.3990781056,-2758816.2769749574,-2833148.078094477,-2907318.054462836,-2981488.0308312047,-3055658.0071995733,-3129827.9835679196,-3203997.9599362845,-3278167.936304656,-3352337.912673016],[3988829.5871419455,3914081.272993133,3839332.95884431,3764584.6446954943,3689836.3305466734,3615088.016397855,3540339.702249033,3465591.388100217,3390843.0739514004,3316094.7598025803,3241346.4456537613,3166598.131504941,3091849.8173561217,3017101.5032073036,2942353.189058489,2867604.87490967,2792856.56076085,2718108.24661203,2643359.932463212,2568611.6183143933,2493863.3041655747,2419114.9900167575,2344366.6758679366,2269618.36171912,2194870.0475703,2120121.7334214807,2045376.5803103643,1970636.3191579627,1895900.379689487,1821168.2174002691,1746439.5017219041,1671714.8561626396,1596992.7316604406,1522272.6607369431,1447556.0157021673,1372841.1178780417,1298128.0396897874,1223417.454886914,1148707.7276728004,1074000.6347792454,999294.0496933935,924589.6643453538,849886.0192774434,775183.5917446255,700482.6910397953,625781.7903349632,551083.1583984904,476384.8125876663,401686.55139292916,326990.5769331963,252294.60247345595,177598.628013717,102904.47265023598,28210.691674819216,-46483.089300596155,-121176.47461634083,-195868.23453881638,-270559.99446128635,-345251.754383754,-419951.9941776935,-494645.77515311027,-569339.556128535,-644033.3371039452,-718753.0655031255,-793449.0399628584,-868145.0144225983,-942840.9888823284,-1017582.1036288557,-1092280.4494396867,-1166978.7952505024,-1241677.141061334,-1316441.627990676,-1391142.5286955219,-1465843.4294003537,-1540544.330105191,-1615245.2308100173,-1689946.131514851,-1764741.5580994086,-1839445.2031673193,-1914148.8482352346,-1988852.4933031378,-2063556.1383710438,-2138259.783438958,-2213089.1089982213,-2287795.6940840688,-2362502.279169921,-2437208.8642557664,-2511915.4493416212,-2586622.034427467,-2661328.619513329,-2736035.204599184,-2810907.814675266,-2885617.541889387,-2960327.26910349,-3035036.9963176083,-3109746.7235317295,-3184456.4507458434,-3259166.1779599516,-3333875.9051740766,-3408585.6323881866],[3986006.8134277407,3910716.7548287245,3835426.6962297084,3760136.637630686,3684846.579031667,3609556.5204326436,3534266.461833626,3458976.403234611,3383686.3446355914,3308396.28603657,3233106.227437552,3157816.168838529,3082526.110239511,3007236.0516404905,2931945.993041474,2856655.9344424554,2781365.8758434337,2706075.817244413,2630785.758645395,2555495.700046374,2480205.641447355,2404915.582848337,2329625.5242493176,2254335.465650298,2179045.407051278,2103755.348452258,2028470.4690191012,1953190.5307774993,1877914.9575821543,1802643.1991321724,1727374.7295380253,1652109.0459551844,1576847.2180021629,1501587.6797693744,1426329.7603928093,1351075.3878719446,1275822.0503197869,1200571.0746971858,1125321.4808384308,1050073.6910735217,974827.039866447,899582.2316770146,824337.7570209638,749095.7326002554,673853.7081795428,598613.2690312327,523373.8367755553,448134.40451988485,372896.92728346866,297659.9015298807,222422.87577629602,147187.332994984,71952.53376864502,-3282.265457691159,-78516.85785265919,-153749.60508181574,-228982.35231097369,-304215.09954012884,-379447.84676927654,-454691.0608157292,-529925.8600420621,-605160.6592684006,-680421.4332667193,-755658.4590202989,-830895.4847738869,-906132.5105274622,-981414.8083378859,-1056654.2405935614,-1131893.672849242,-1207133.1051049135,-1282438.926758897,-1357680.951179606,-1432922.9756003204,-1508165.000021033,-1583407.024441732,-1658649.0488624498,-1733986.0549532007,-1809230.8631426329,-1884475.6713320762,-1959720.4795215,-2034965.2877109312,-2110210.09590037,-2185581.268817586,-2260829.0585824866,-2336076.8483474012,-2411324.638112304,-2486572.427877225,-2561820.217642133,-2637068.007407033,-2712315.7971719475,-2787730.633300571,-2862981.6089231754,-2938232.58454578,-3013483.560168378,-3088734.535790964,-3163985.511413575,-3239236.4870361704,-3314487.462658774,-3389738.4382813824,-3465204.453463507],[3983176.312328802,3907343.0262461444,3831509.740163489,3755676.454080835,3679843.167998176,3604009.881915523,3528176.595832865,3452343.3097502114,3376510.0236675562,3300676.7375848996,3224843.451502246,3149010.1654195893,3073176.8793369336,2997343.593254279,2921510.307171626,2845677.021088971,2769843.735006313,2694010.4489236586,2618177.1628410025,2542343.876758347,2466510.590675693,2390677.304593037,2314844.0185103817,2239010.7324277265,2163177.4463450713,2087345.9255773334,2011519.6747698868,1935698.1024353467,1859880.643389267,1784067.4435522663,1708257.927151986,1632451.223369388,1556646.845463732,1480846.0324715998,1405047.1774059278,1329250.0276008332,1253455.8078645533,1177662.145697345,1101871.6684346632,1026081.1911719807,950293.7349205071,874506.2808707992,798721.0248671737,722936.3940797744,647152.5952032851,571370.5939469798,495588.5926906788,419807.9804874095,344028.4209853951,268248.8614833825,192470.38861656422,116693.08879109193,40915.78896562243,-34861.51085985266,-110636.76508813351,-186411.9818617073,-262187.1986352778,-337962.41540885484,-413748.0099872085,-489525.30981268035,-565302.6096381517,-641105.8525407971,-716885.4120428171,-792664.971544832,-868444.5310468492,-944269.4311790648,-1020051.4324353673,-1095833.4336916762,-1171615.434947968,-1247464.0063926354,-1323248.6371800406,-1399033.2679674434,-1474817.8987548444,-1550602.5295422357,-1626387.1603296408,-1702267.1568191955,-1778054.6108688992,-1853842.064918601,-1929629.518968314,-2005416.973018012,-2081204.4270677269,-2157118.8538606763,-2232909.331123367,-2308699.8083860474,-2384490.2856487213,-2460280.76291141,-2536071.240174084,-2611861.71743678,-2687816.95202756,-2763610.658948566,-2839404.3658695687,-2915198.0727905687,-2990991.7797115725,-3066785.486632563,-3142579.1935535762,-3218372.900474583,-3294166.6073955745,-3369960.3143165866,-3445970.41012019,-3521767.55992529],[3980338.160329087,3903960.1784081077,3827582.196487121,3751204.214566141,3674826.232645154,3598448.2507241704,3522070.2688031862,3445692.2868822035,3369314.3049612185,3292936.323040235,3216558.3411192545,3140180.3591982657,3063802.377277286,2987424.3953563003,2911046.4134353194,2834668.4315143363,2758290.4495933508,2681912.4676723676,2605534.485751383,2529156.503830399,2452778.5219094157,2376400.539988432,2300022.5580674484,2223644.5761464653,2147266.5942254807,2070892.3290247428,1994523.3861944156,1918159.1683898065,1841799.1046412466,1765442.6488778603,1689089.4217263197,1612740.2303683728,1536393.3863302479,1460048.417779753,1383707.0224110293,1307366.9657652194,1231029.0593266962,1154693.0494333925,1078358.4165235588,1002025.6256976994,925694.0577168297,849363.6935338273,773034.9883952923,696706.2915853423,620380.2534375419,544054.2152897441,467728.9348344989,391405.37379297614,315081.8127514501,238758.88140260754,162437.61329194298,86116.34518127656,9795.07707061572,-66524.41442703316,-142843.5683218958,-219162.72221674956,-295481.87611160893,-371811.26348271035,-448132.53159337817,-524453.7997040418,-600800.919746853,-677124.4807883836,-753448.0418299083,-829771.6028714404,-906140.509518425,-982466.5476662251,-1058792.5858140215,-1135118.6239618221,-1211511.345206561,-1287840.0503450967,-1364168.755483626,-1440497.460622158,-1516826.1657606931,-1593154.8708992312,-1669579.2536327383,-1745910.8216136089,-1822242.3895944795,-1898573.9575753426,-1974905.5255562114,-2051237.093537083,-2127696.1656894423,-2204030.7985992674,-2280365.431509111,-2356700.0644189436,-2433034.697328776,-2509369.3302386133,-2585703.9631484356,-2662204.169038007,-2738542.0754765244,-2814879.9819150455,-2891217.8883535797,-2967555.794792097,-3043893.7012306303,-3120231.6076691495,-3196569.5141076725,-3272907.420546187,-3349245.3269847147,-3425800.8858185653,-3502142.281187307,-3578483.676556034],[3977492.4338887725,3900568.3024489437,3823644.1710091103,3746720.0395692782,3669795.908129447,3592871.7766896146,3515947.645249782,3439023.5138099515,3362099.3823701213,3285175.250930288,3208251.1194904563,3131326.988050621,3054402.856610792,2977478.7251709606,2900554.5937311286,2823630.4622912984,2746706.330851464,2669782.1994116334,2592858.0679718014,2515933.936531969,2439009.8050921387,2362085.673652306,2285161.542212474,2208237.410772643,2131313.3863556064,2054395.0211317113,1977481.725749189,1900573.4084559495,1823669.286377883,1746768.8077415917,1669871.4443043093,1592976.68999626,1516085.835723993,1439196.9650810063,1362309.87053697,1285425.9686213515,1208542.8428642259,1131662.6008227158,1054782.94183362,977905.8065140732,901028.805425995,824154.5726137953,747280.3398015953,670407.9257080315,593536.3974383213,516664.92973628826,439795.9140232159,362926.89831014303,286057.9943549335,209191.30493071163,132324.61550648697,55457.92608225765,-21407.311866373755,-98271.85580294672,-175136.39973953133,-252000.9436761099,-328875.52103885915,-405742.2104630838,-482608.89988731174,-559501.2902467134,-636370.3059597849,-713239.3216728531,-790108.3373859306,-867022.6396861244,-943894.167955833,-1020765.6962255449,-1097637.2244952451,-1174575.480503431,-1251449.7133156317,-1328323.9461278268,-1405198.1789400335,-1482072.4117522351,-1558946.6445644344,-1635916.7943505142,-1712793.9296700777,-1789671.0649896218,-1866548.200309163,-1943425.3356287116,-2020302.470948251,-2097307.5649206033,-2174187.806962112,-2251068.049003627,-2327948.291045131,-2404828.533086639,-2481708.7751281476,-2558589.0171696497,-2635635.5663302783,-2712519.12583913,-2789402.685347995,-2866286.244856842,-2943169.8043657,-3020053.3638745537,-3096936.9233834026,-3173820.48289227,-3250704.04240112,-3327802.896649195,-3404689.991193222,-3481577.08573726,-3558464.1802812973,-3635351.2748253327],[3974639.2094313586,3897167.4894592995,3819695.7694872376,3742224.0495151705,3664752.3295431123,3587280.6095710485,3509808.889598985,3432337.1696269256,3354865.4496548614,3277393.7296828013,3199922.0097107375,3122450.2897386746,3044978.5697666127,2967506.8497945485,2890035.129822487,2812563.409850425,2735091.6898783618,2657619.969906298,2580148.2499342365,2502676.5299621737,2425204.809990112,2347733.0900180493,2270261.370045987,2192789.650073923,2115319.9219342894,2037855.908489495,1960396.991129748,1882942.5781545453,1805492.1032516446,1728045.7355568716,1650602.940894599,1573162.7799695535,1495724.7619800242,1418290.5293097435,1340857.76428686,1263427.1128142173,1185998.7085831384,1108571.4185833407,1031146.5207976175,953722.3793883182,876300.1692619685,798878.9701085389,721458.6622504189,644040.2052819123,566621.748313413,489205.136762938,411789.22789840354,334373.31903387373,256959.30236785905,179545.75325248577,102132.20413710922,24719.724811324384,-52691.64743790543,-130103.01968714036,-207514.39193637855,-284935.5414397754,-362349.09055514866,-439762.6396705252,-517176.18878589943,-594617.5873405151,-672033.4962050458,-749449.4050695803,-826910.4771196395,-904328.9340881426,-981747.3910566452,-1059165.8480251506,-1136584.304993649,-1214072.207880667,-1291493.4070340986,-1368914.6061875261,-1446335.8053409634,-1523757.0044943923,-1601274.2871184498,-1678698.4285277454,-1756122.5699370392,-1833546.7113463422,-1910970.8527556304,-1988394.9941649232,-2065947.471408341,-2143374.7614081437,-2220802.051407938,-2298229.3414077293,-2375656.631407529,-2453083.921407329,-2530511.211407123,-2608105.460818056,-2685536.1122907056,-2762966.7637633523,-2840397.415235983,-2917828.0667086393,-2995258.7181812814,-3072689.369653926,-3150120.021126576,-3227550.6725992206,-3305197.6635773797,-3382631.8962476514,-3460066.128917929,-3537500.361588221,-3614934.5942584956,-3692368.8269287804],[3971778.563331088,3893757.8304710947,3815737.0976110995,3737716.364751108,3659695.631891117,3581674.899031117,3503654.166171123,3425633.433311133,3347612.7004511417,3269591.9675911437,3191571.2347311503,3113550.501871154,3035529.769011161,2957509.036151169,2879488.303291174,2801467.570431181,2723446.837571187,2645426.104711193,2567405.3718511974,2489384.638991204,2411363.9061312117,2333343.1732712183,2255322.440411225,2177301.70755123,2099284.8503288673,2021273.7636068426,1943267.8229640257,1865266.4309567374,1787269.0155821634,1709275.0288114005,1631284.2842615386,1553297.2936332505,1475312.465398359,1397329.670302262,1319350.1551069878,1241371.6453102499,1163395.88317409,1085421.1350817704,1007448.5634787397,929476.7418331981,851507.1523165959,773537.5627999976,695570.645910695,617603.8363106935,539638.0780307539,461673.85217762226,383709.62632449344,305746.6851731227,227784.85263043735,149823.0200877469,71861.81870991038,-6097.805482760072,-84057.4296754282,-162017.05386809958,-239976.67806077003,-317947.9751683716,-395909.8077110583,-473871.64025374316,-551861.0839130473,-629825.3097661748,-707789.535619305,-785798.7364893416,-863765.5460893428,-941732.3556893463,-1019699.1652893522,-1097665.9748893525,-1175702.1751149958,-1253671.764631595,-1331641.3541481919,-1409610.9436647943,-1487580.5331813963,-1565646.2994213793,-1643618.8710244037,-1721591.442627444,-1799564.0142304748,-1877536.5858335076,-1955509.157436545,-2033610.364408453,-2111586.126544617,-2189561.888680784,-2267537.65081694,-2345513.4129530992,-2423489.1750892643,-2501464.937225423,-2579608.2288888795,-2657587.39656879,-2735566.5642487127,-2813545.731928629,-2891524.899608548,-2969504.067288462,-3047483.234968382,-3125462.4026483046,-3203441.570328215,-3281638.035463501,-3359620.830559601,-3437603.6256557023,-3515586.420751796,-3593569.2158478945,-3671552.010943991,-3749534.8060400914],[3968910.5719005438,3890339.416442771,3811768.2609849977,3733197.10552722,3654625.9500694443,3576054.7946116715,3497483.6391538945,3418912.4836961227,3340341.3282383466,3261770.1727805706,3183199.0173227955,3104627.86186502,3026056.7064072443,2947485.550949471,2868914.3954916974,2790343.240033923,2711772.0845761476,2633200.9291183725,2554629.773660598,2476058.6182028214,2397487.4627450476,2318916.3072872725,2240345.151829498,2161774.0012675202,2083208.9686540815,2004649.414052032,1926094.7249728586,1847544.8741346477,1768999.1089337813,1690456.805649979,1611917.433217139,1533380.7611422148,1454847.7146273039,1376316.3680555434,1297787.2745630976,1219260.613641119,1140734.9698273223,1062212.1135435824,983689.7022781759,905169.8054638719,826650.4161960627,748132.6672965067,669616.0957647641,591099.7147092987,512585.76266150037,434071.81061370345,355558.47375062294,277046.94867418567,198535.42359774932,120024.03373758262,41514.74859915348,-36994.53653927939,-115503.82167770807,-194013.10681614373,-272533.7268608706,-351045.2519373079,-429556.77701374097,-508095.6139598745,-586609.5660076705,-665123.5180554707,-743637.4701032657,-822198.7633383758,-900715.3348701131,-979231.9064018563,-1057748.4779336052,-1136334.3154992047,-1214853.7047670176,-1293373.0940348268,-1371892.4833026393,-1450411.872570449,-1529027.4582149386,-1607549.86948035,-1686072.2807457605,-1764594.692011171,-1843117.1032765796,-1921639.5145419892,-2000290.7827244867,-2078816.4265382802,-2157342.0703520784,-2235867.7141658776,-2314393.3579796627,-2392919.0017934656,-2471444.6456072573,-2550138.3065667544,-2628667.400059209,-2707196.49355165,-2785725.5870440975,-2864254.6805365384,-2942783.7740289886,-3021312.8675214387,-3099841.9610138815,-3178371.054506325,-3257118.316285569,-3335651.083467448,-3414183.8506493326,-3492716.6178312125,-3571249.3850130914,-3649782.152194976,-3728314.919376851,-3806847.6865587374],[3966035.311378373,3886912.3382446263,3807789.3651108798,3728666.3919771328,3649543.4188433844,3570420.4457096383,3491297.4725758904,3412174.4994421457,3333051.5263083978,3253928.5531746494,3174805.5800409038,3095682.606907153,3016559.633773409,2937436.6606396623,2858313.6875059167,2779190.7143721688,2700067.741238419,2620944.7681046734,2541821.7949709264,2462698.8218371775,2383575.848703433,2304452.875569685,2225329.902435938,2146208.7515662103,2067093.7774329437,1987984.3343038678,1908879.8039628395,1829779.594126178,1750683.1369298734,1671590.8549914267,1592501.7998118792,1513415.188190763,1434331.053888606,1355250.110360469,1276170.4119719327,1197093.4915619274,1118017.8605384529,1038944.2147632851,959871.7894441979,880801.2056569422,801731.1582447679,722663.4300994186,643595.7019540702,564529.8916608607,485464.81882930873,406399.7459977544,327337.0171779706,248274.40507698944,169211.79297601245,90151.03406780772,11090.693595662713,-67969.64687648648,-147029.9873486301,-226101.26752888924,-305163.879629869,-384226.4917308483,-463316.05514933355,-542381.1279808888,-621446.2008124501,-700511.2736439994,-779623.4046621993,-858691.1328075458,-937758.860952903,-1016826.5890982486,-1095963.389024389,-1175033.9728116463,-1254104.556598886,-1333175.1403861498,-1412245.7241733978,-1491412.450042075,-1570486.0958172474,-1649559.7415924082,-1728633.3873675782,-1807707.0331427604,-1886780.6789179184,-1965983.3248382928,-2045060.2452482954,-2124137.165658298,-2203214.0860683024,-2282291.0064783143,-2361367.9268883206,-2440444.8472983176,-2519690.1896593506,-2598770.6039454844,-2677851.0182315996,-2756931.4325177083,-2836011.846803833,-2915092.2610899443,-2994172.675376066,-3073253.089662184,-3152333.503948305,-3231632.8699386874,-3310717.0042408416,-3389801.1385429967,-3468885.272845151,-3547969.4071473097,-3627053.5414494667,-3706137.675751636,-3785221.8100537853,-3864305.9443559498],[3963152.857917272,3883476.6866445253,3803800.5153717822,3724124.344099036,3644448.172826288,3564772.00155354,3485095.8302807943,3405419.65900805,3325743.4877353073,3246067.3164625536,3166391.1451898096,3086714.9739170633,3007038.8026443175,2927362.6313715708,2847686.460098824,2768010.288826079,2688334.117553332,2608657.946280585,2528981.775007839,2449305.603735091,2369629.4324623467,2289953.2611896,2210277.089916854,2130604.5573001355,2050938.2604767643,1971277.5479508066,1891621.7958079302,1811970.406115161,1732322.8053918073,1652678.4431481473,1573037.559410557,1493399.9086406834,1413764.2233227687,1334131.1078695315,1254500.4437179347,1174870.8663994384,1095244.3157471982,1015618.0552197499,935994.7465089133,856371.5880374084,776750.683524285,697130.4186854691,617511.1122453907,537893.5386416032,458275.9650378162,378659.97465124726,299044.8956342023,219429.81661716336,139816.0108544398,60203.23525863048,-19409.540337186307,-99022.31593299285,-178645.57846806664,-258260.65748511162,-337875.73650215566,-417490.8155191974,-497134.91820763145,-576752.4918114208,-656370.0654152022,-736034.3484132271,-815654.6132520437,-895274.8780908608,-974895.1429296769,-1054584.2152787033,-1134207.3737502084,-1213830.5322217084,-1293453.6906932043,-1373076.849164708,-1452796.0211310638,-1532422.2816585107,-1612048.5421859557,-1691674.8027134053,-1771301.0632408531,-1850927.323768301,-1930682.6490219356,-2010312.2263404345,-2089941.8036589231,-2169571.380977422,-2249200.9582959237,-2328830.5356144207,-2408460.112932912,-2488258.433885839,-2567891.5493390784,-2647524.6647923226,-2727157.7802455574,-2806790.8956987867,-2886424.011152028,-2966057.126605276,-3045690.2420585062,-3125539.238535894,-3205176.1203837628,-3284813.0022316305,-3364449.8840795048,-3444086.765927378,-3523723.647775243,-3603360.5296231154,-3682997.4114709934,-3762634.293318861,-3842271.1751667345,-3921908.0570146106],[3960263.2875721436,3880032.5522937663,3799801.8170153876,3719571.081737011,3639340.346458633,3559109.611180255,3478878.875901882,3398648.1406235024,3318417.405345128,3238186.6700667473,3157955.9347883724,3077725.199509994,2997494.464231617,2917263.72895324,2837032.993674862,2756802.258396486,2676571.523118108,2596340.787839732,2516110.0525613544,2435879.3172829757,2355648.5820045993,2275417.846726222,2195187.111447845,2114961.9845253453,2034743.276750708,1954529.856451533,1874321.156017798,1794117.220851576,1713917.1113585727,1633720.2714708159,1553526.1667120063,1473335.1714833342,1393147.1004412903,1312960.5246326623,1232776.9246810062,1152594.7324759369,1072414.4915398848,992235.7948768479,912058.6961410448,831882.6410162437,751708.473987438,671534.3069586335,591362.7984379476,511191.35865540337,431020.67828875175,350851.7670447449,270682.8558007353,190514.57952475175,110348.00359468209,30181.427664613817,-49985.148265456315,-130151.72419552505,-210330.61166331917,-290499.52290732367,-370668.43415133515,-450865.918735079,-531037.3585176212,-611208.7983001624,-691426.5325310561,-771600.6995598581,-851774.8665886666,-931949.0336174676,-1012191.6735182409,-1092368.7722540405,-1172545.8709898498,-1252722.9697256503,-1332900.0684614545,-1413172.9774777335,-1493353.2184137898,-1573533.4593498423,-1653713.7002858901,-1733893.9412219482,-1814074.1821580017,-1894383.4734334694,-1974567.0733851232,-2054750.6733367695,-2134934.273288425,-2215117.8732400797,-2295301.473191741,-2375485.0731433844,-2455837.654988642,-2536024.837393432,-2616212.019798221,-2696399.202203011,-2776586.384607803,-2856773.5670125894,-2936960.749417389,-3017147.931822166,-3097551.5565509014,-3177742.551779569,-3257933.547008232,-3338124.54223691,-3418315.5374655817,-3498506.5326942513,-3578697.5279229274,-3658888.5231515933,-3739079.518380264,-3819270.513608929,-3899461.5088376114,-3979932.9260541974],[3957366.676288405,3876580.0257131527,3795793.375137896,3715006.7245626384,3634220.0739873834,3553433.4234121246,3472646.772836868,3391860.122261612,3311073.471686358,3230286.8211110993,3149500.1705358448,3068713.519960586,2987926.869385332,2907140.218810075,2826353.5682348195,2745566.9176595644,2664780.267084307,2583993.616509049,2503206.9659337946,2422420.3153585372,2341633.6647832836,2260847.014208026,2180061.621117327,2099282.8835829636,2018510.1287553655,1937742.7119125698,1856980.0148105975,1776221.444106165,1695466.538768239,1614715.8709225124,1533967.964159918,1453222.298660744,1372479.6980880452,1291739.399771704,1211000.4260289227,1130264.5605950877,1049529.0476851063,968796.6576688131,888064.2676525195,807334.6081629305,726605.1880100628,645877.3673123843,565150.7105056355,484424.0536988918,403699.83173347404,322975.7375113438,242251.6432892047,161529.85434089135,80808.12742443522,86.40050798049197,-80635.32640846632,-161368.82782148523,-242092.9220436262,-322817.0162657583,-403569.17117533134,-484295.827982076,-565022.4847888313,-645749.1415955769,-726524.3747415654,-807253.7948944299,-887983.2150472985,-968712.6352001671,-1049513.0927385138,-1130245.4827548098,-1210977.872771108,-1291710.2627873998,-1372538.1849118322,-1453273.757347534,-1534009.3297832436,-1614744.9022189546,-1695480.4746546512,-1776216.0470903562,-1857080.5761967287,-1937819.5499395085,-2018558.5236822814,-2099297.4974250626,-2180036.4711678503,-2260775.4449106324,-2341514.418653407,-2422422.5288288184,-2503165.1294015124,-2583907.729974212,-2664650.33054692,-2745392.931119619,-2826135.531692311,-2906878.13226502,-2987620.732837716,-3068580.248723209,-3149326.7085983483,-3230073.1684735073,-3310819.6283486355,-3391566.088223777,-3472312.5480989274,-3553059.0079740677,-3633805.4678492164,-3714551.9277243493,-3795298.3875994887,-3876044.8474746356,-3957072.694952541,-4037823.2538782554],[3954463.0998906065,3873119.1972794076,3791775.294668215,3710431.3920570156,3629087.48944582,3547743.5868346193,3466399.684223422,3385055.7816122277,3303711.879001031,3222367.9763898314,3141024.073778637,3059680.171167438,2978336.2685562395,2896992.3659450435,2815648.4633338475,2734304.5607226496,2652960.658111453,2571616.7555002533,2490272.8528890577,2408928.9502778584,2327585.0476666633,2246241.1450554663,2164900.2490978176,2083566.0708703108,2002237.9318080414,1920915.1815377239,1839597.1962104389,1758283.3769069891,1676973.1481123152,1595665.9562555891,1514362.6950145871,1433061.9129007393,1351762.8863200091,1270467.2021717196,1189172.8858651635,1107880.6453809377,1026590.1249019902,945301.1071297829,864013.5022677032,782727.4925960626,701441.9550713627,620158.7449329437,538875.5347945252,457593.8427739255,376313.22935915366,295032.6159443832,213753.58381890692,132475.36979934154,51197.15577977011,-30081.05823979713,-111370.45112947468,-192651.06454423955,-273931.6779590086,-355239.7767281099,-436522.9868665347,-517806.1970049534,-599089.4071433749,-680420.6814935072,-761706.691165146,-842992.7008367879,-924278.7105084341,-1005635.3194031417,-1086924.3371753478,-1168213.354947553,-1249502.3727197587,-1330791.3904919676,-1412178.809630081,-1493471.0501143066,-1574763.2905985317,-1656055.5310827605,-1737347.7715669894,-1818768.7954632803,-1900064.4796115672,-1981360.1637598583,-2062655.8479081523,-2143951.532056437,-2225247.216204726,-2306542.9003530247,-2388007.7914633416,-2469307.1468753396,-2550606.5022873394,-2631905.857699341,-2713205.213111341,-2794504.568523355,-2875803.923935351,-2957103.2793473573,-3038619.9344816403,-3119923.195722647,-3201226.456963647,-3282529.718204652,-3363832.9794456484,-3445136.2406866634,-3526439.501927668,-3607742.76316866,-3689046.0244096788,-3770349.2856506724,-3851652.546891677,-3933238.068088879,-4014545.4770193268,-4095852.885949771],[3951552.634071063,3869650.157211668,3787747.680352277,3705845.203492881,3623942.726633492,3542040.249774095,3460137.772914706,3378235.2960553113,3296332.8191959197,3214430.3423365257,3132527.8654771326,3050625.3886177386,2968722.9117583465,2886820.4348989525,2804917.9580395618,2723015.481180168,2641113.004320775,2559210.527461382,2477308.0506019886,2395405.573742594,2313503.0968832024,2231600.6200238094,2149702.8980380553,2067812.0751191215,1985927.4165857537,1904047.845316727,1822173.1085289852,1740302.7189110937,1658435.9542321546,1576572.2553724456,1494711.0837338006,1412853.6513307663,1330998.2032345198,1249144.4865846394,1167293.8011305977,1085443.6909426774,1003596.7234546354,921749.7781337092,839905.857062682,758061.9359916518,676220.1269878894,594379.0417230581,512538.4939007079,430700.0395902046,348861.58527970174,267023.92894090246,185187.90621163044,103351.883482364,21515.86075309664,-60320.16197617026,-142169.14058331354,-224007.59489382105,-305846.04920432577,-387713.98145491164,-469555.0667197434,-551396.1519845682,-633284.7222158234,-715128.6432868531,-796972.5643578828,-878816.4854289014,-960730.4962583585,-1042577.4637464052,-1124424.4312344403,-1206271.3987224856,-1288118.3662105314,-1370063.3463752703,-1451913.5769399786,-1533763.807504681,-1615614.0380693781,-1697464.2686340911,-1779443.0294609014,-1861296.7461107876,-1943150.4627606678,-2025004.1794105526,-2106857.8960604193,-2188711.6127103036,-2270565.3293601824,-2352588.2392087486,-2434445.671611773,-2516303.1040148083,-2598160.536417844,-2680017.968820882,-2761875.4012239166,-2843732.8336269446,-2925590.2660299754,-3007665.2937232694,-3089526.678528552,-3171388.063333825,-3253249.4481391096,-3335110.8329443773,-3416972.2177496627,-3498833.6025549406,-3580694.9873602204,-3662556.3721655207,-3744417.756970795,-3826279.141776069,-3908423.5653890735,-3990289.1465599826,-4072154.7277308786,-4154020.308901783],[3948635.354378883,3866172.9955583676,3783710.6367378538,3701248.2779173343,3618785.9190968177,3536323.5602763,3453861.20145578,3371398.842635266,3288936.4838147503,3206474.1249942286,3124011.766173713,3041549.407353195,2959087.0485326773,2876624.68971216,2794162.330891644,2711699.9720711266,2629237.613250608,2546775.254430091,2464312.8956095744,2381850.5367890536,2299388.17796854,2216926.1175283366,2134471.207748576,2052022.7486691172,1969580.0690776324,1887142.5247906614,1804709.4970083893,1722280.3907394686,1639855.3295871816,1557433.8172180292,1475014.8557768725,1392598.0387233421,1310184.8634724533,1227772.9751566183,1145363.4469689978,1062955.6650820328,980549.4404063576,898144.8585444712,815741.7186801829,733339.462460543,650939.1947599361,568538.9270593312,486140.81711342745,403743.21468814276,321345.612262859,238950.42866961984,156555.29010685347,74160.15154409083,-8234.987018666696,-90642.39986356022,-173040.00228884676,-255437.60471413145,-337864.01764733857,-420264.2853479474,-502664.5530485511,-585111.6590127274,-667514.7988770227,-749917.9387413156,-832321.0786056011,-914793.7271341784,-997199.9518098487,-1079606.1764855208,-1162012.4011611925,-1244418.6258368748,-1326922.3986595427,-1409331.9268471743,-1491741.4550347924,-1574150.9832224092,-1656560.5114100338,-1739098.2365232063,-1821511.293279849,-1903924.3500364888,-1986337.4067931352,-2068750.4635497741,-2151163.520306413,-2233576.577063051,-2316158.7286854377,-2398575.545738969,-2480992.362792492,-2563409.1798460335,-2645825.9968995573,-2728242.813953086,-2810659.6310066152,-2893076.448060142,-2975711.066874494,-3058131.8829489145,-3140552.699023327,-3222973.515097725,-3305394.331172134,-3387815.1472465442,-3470235.9633209556,-3552656.77939537,-3635077.5954697737,-3717498.4115441833,-3799919.227618601,-3882623.7676160084,-3965048.828768285,-4047473.889920585,-4129898.9510728614,-4212324.0122251455],[3945711.336209079,3862687.802184253,3779664.2681594337,3696640.7341346075,3613617.2001097836,3530593.6660849573,3447570.1320601352,3364546.598035312,3281523.064010493,3198499.529985665,3115475.99596084,3032452.4619360142,2949428.9279111926,2866405.3938863687,2783381.8598615434,2700358.325836723,2617334.7918118956,2534311.257787075,2451287.723762251,2368264.1897374247,2285240.6557126027,2202219.102207285,2119204.7645277963,2036196.9370467218,1953194.942948166,1870198.1324901646,1787205.8813409968,1704217.58898574,1621232.6771995455,1538251.1366152081,1455273.0936835604,1372297.0951581453,1289323.4051509986,1206352.5092593597,1123382.4213589302,1040415.6464824788,957449.0150148936,874485.3634207398,791521.7118265908,708560.5549368176,625599.8119469807,542640.1515405204,459682.10823600413,376724.0649314886,293767.56980118994,210812.02273447858,127856.47566776397,44900.928601054475,-38054.61846566014,-121024.19489560556,-203982.23820012202,-286968.36094118934,-369929.10393101815,-452889.84692085534,-535850.5899106883,-618860.3652740112,-701824.0168681699,-784787.6684623212,-867751.3200564785,-950786.9505523071,-1033753.7254287535,-1116720.5003052033,-1199687.27518166,-1282751.0575281987,-1365721.1764211203,-1448691.2953140321,-1531661.4142069519,-1614631.5330998641,-1697601.6519927927,-1780703.1251133792,-1863676.8151205205,-1946650.5051276768,-2029624.195134819,-2112597.885141962,-2195571.5751491133,-2278714.1768489666,-2361691.6717496887,-2444669.1666504154,-2527646.661551133,-2610624.156451853,-2693601.6513525797,-2776579.146253301,-2859556.6411540257,-2942752.0549390605,-3025733.5955233984,-3108715.136107738,-3191696.6766920798,-3274678.2172764055,-3357659.757860749,-3440641.298445086,-3523622.839029424,-3606604.379613758,-3689585.9201981025,-3772567.4607824367,-3855833.316440478,-3938819.150849823,-4021804.98525918,-4104790.8196685296,-4187776.654077869,-4270762.488487229],[3942780.654791907,3859194.66675773,3775608.6787235513,3692022.6906893705,3608436.7026551897,3524850.7146210107,3441264.7265868345,3357678.7385526504,3274092.750518472,3190506.762484292,3106920.7744501126,3023334.786415932,2939748.798381753,2856162.810347574,2772576.8223133935,2688990.834279214,2605404.8462450346,2521818.8582108538,2438232.870176675,2354646.882142494,2271060.894108315,2187478.5678139804,2103903.5227109445,2020335.04753569,1936772.459898706,1853215.1045279051,1769662.3515861467,1686113.5950591506,1602568.2512102453,1519025.7570985607,1435486.3055851788,1351949.9671858903,1268415.1735775159,1184883.1853254435,1101352.8189306608,1017824.2152674492,934297.3435169782,850771.9016821308,767247.6241009682,683725.1273928764,600202.630684789,516682.81071620714,433163.0481911199,349643.90290301666,266126.66908376385,182609.4352645008,99092.20144524518,15574.967625984456,-67955.52695940714,-151475.28948449157,-234995.05200957833,-318544.83310419414,-402067.32981228223,-485589.8265203661,-569160.6095102015,-652686.0513450471,-736211.4931798894,-819736.9350147322,-903333.6689863959,-986862.2726496123,-1070390.876312822,-1153919.47997603,-1237544.4739846056,-1321076.4622366703,-1404608.4504887518,-1488140.4387408216,-1571672.426992884,-1655204.4152449607,-1738867.3057594434,-1822402.9077309407,-1905938.509702431,-1989474.1116739279,-2073009.7136454294,-2156545.3156169215,-2240249.5610036505,-2323789.0125170276,-2407328.46403041,-2490867.9155437993,-2574407.3670571763,-2657946.81857056,-2741486.270083936,-2825025.721597325,-2908783.1195278894,-2992326.663430456,-3075870.2073330143,-3159413.7512355642,-3242957.2951381328,-3326500.839040691,-3410044.3829432325,-3493587.926845791,-3577131.4707483547,-3660675.0146509185,-3744499.0279852105,-3828046.9144935813,-3911594.801001952,-3995142.6875103265,-4078690.5740187103,-4162238.4605270717,-4245786.347035454,-4329334.233543826],[3939843.385182378,3855693.678738291,3771543.9722942025,3687394.2658501095,3603244.5594060197,3519094.852961932,3434945.1465178407,3350795.440073753,3266645.733629665,3182496.0271855732,3098346.3207414835,3014196.614297392,2930046.907853306,2845897.201409215,2761747.494965126,2677597.788521037,2593448.082076947,2509298.375632858,2425148.6691887677,2340998.9627446774,2256849.2563005886,2172705.103141081,2088568.9386557227,2004439.0110076687,1920314.650100955,1836195.2117120954,1752080.0758446134,1667968.87279537,1583861.8335209405,1499757.670925343,1415655.8358653113,1331557.0576119781,1247460.6752654687,1163365.5533953947,1079273.6468545918,995182.0732216276,911093.5770275258,827005.0808334209,742919.3305229114,658833.8160584483,574749.8077142716,490667.0620168783,406584.31631948054,322503.75757050095,238423.57313844934,154343.3887063982,70263.20427435031,-13829.412167502567,-97912.15786489518,-181994.90356229246,-266106.8430506508,-350192.3575151125,-434277.87197957514,-518410.85827221256,-602499.3544663135,-686587.8506604158,-770676.3468545186,-854835.3759190417,-938927.07256138,-1023018.7692037113,-1107110.4658460408,-1191202.162488372,-1275392.980836778,-1359488.1027068505,-1443583.2245769282,-1527678.3464469924,-1611773.4683170756,-1695998.9595080148,-1780097.7377613499,-1864196.5160146812,-1948295.2942680148,-2032394.072521356,-2116492.850774682,-2200591.629028013,-2284862.591297012,-2368965.263790843,-2453067.9362846725,-2537170.6087785074,-2621273.281272335,-2705375.953766166,-2789478.6262599993,-2873799.1828758614,-2957905.9945059875,-3042012.8061361173,-3126119.6177662537,-3210226.4293963714,-3294333.2410264984,-3378440.05265663,-3462546.864286755,-3546653.675916884,-3630760.487547012,-3715148.1223562257,-3799259.3254054748,-3883370.528454708,-3967481.731503959,-4051592.9345532088,-4135704.137602452,-4219815.340651691,-4303926.54370093,-4388037.746750171],[3936899.6022500624,3852184.927364417,3767470.252478773,3682755.577593128,3598040.9027074813,3513326.227821835,3428611.5529361903,3343896.8780505466,3259182.2031649,3174467.528279254,3089752.85339361,3005038.1785079637,2920323.503622319,2835608.828736673,2750894.1538510285,2666179.478965385,2581464.804079738,2496750.1291940934,2412035.4543084474,2327320.779422801,2242606.665690838,2157900.2815873693,2073200.8944263207,1988507.8009668228,1903820.3256019931,1819137.8186227605,1734459.6545529738,1649785.2305520847,1565113.9648819459,1480446.8214020496,1395782.0909425486,1311119.1729286478,1226459.6675412299,1141801.5075770332,1057145.4681213708,972491.1075124962,887838.3071973578,803186.8194024279,718537.0374989186,633887.5256897062,549240.5472224078,464593.56875510886,379947.7573832311,295303.3728310382,210658.98827884952,126014.60372665571,41370.21917446237,-43288.30204867711,-127935.28051597951,-212582.25898327492,-297260.3453431777,-381910.1272466844,-466559.90915018925,-551259.298160037,-635912.0984751768,-720564.8987903157,-805217.6991054607,-889943.4393588682,-974599.4788145293,-1059255.5182701955,-1143911.5577258584,-1228665.9886938515,-1313325.4940812574,-1397984.9994686749,-1482644.504856096,-1567304.0102435127,-1652093.2707621548,-1736756.4752539578,-1821419.6797457454,-1906082.8842375474,-1990746.0887293443,-2075409.2932211412,-2160072.4977129353,-2244907.4917133655,-2329574.6351932567,-2414241.7786731534,-2498908.92215305,-2583576.065632947,-2668243.2091128407,-2752910.352592732,-2837577.4960726295,-2922466.557243729,-3007137.8866473557,-3091809.2160509797,-3176480.545454601,-3261151.8748582313,-3345823.2042618636,-3430494.533665481,-3515165.8630691133,-3599837.1924727457,-3684789.604157864,-3769465.3738252306,-3854141.1434925813,-3938816.913159946,-4023492.6828272967,-4108168.452494669,-4192844.2221620195,-4277519.991829348,-4362195.761496724,-4446871.53116408],[3933949.3806689777,3848668.5016415063,3763387.622614038,3678106.743586565,3592825.8645590963,3507544.9855316267,3422264.106504156,3336983.227476688,3251702.348449222,3166421.4694217485,3081140.5903942795,2995859.71136681,2910578.8323393427,2825297.953311871,2740017.074284401,2654736.1952569336,2569455.3162294617,2484174.4372019917,2398893.558174522,2313612.6791470516,2228333.9756906386,2143063.0700756833,2057799.2240264895,1972541.7287762065,1887289.903235604,1802043.0922370185,1716800.6648496697,1631562.0127626094,1546326.5487317871,1461093.7050878163,1375864.838724352,1290637.8078191835,1205412.6833371741,1120190.46203688,1034968.8442544101,949750.4640141285,864532.1241355466,779316.6484263241,694101.3637191434,608887.8855512855,523675.439033238,438463.07857076684,353253.2587065846,268043.4388424014,182833.61897821445,97623.79911403544,12400.759924948681,-72811.68659309577,-158024.13311114954,-243266.7680598502,-328482.0527670318,-413697.3374742158,-498961.31392176077,-584179.6538003427,-669397.9936789186,-754616.3335575047,-839906.7469599494,-925128.3647424225,-1010349.9825248923,-1095571.6003073594,-1180890.8021590812,-1266115.9266410954,-1351341.051123097,-1436566.1756051104,-1521791.3000871208,-1607016.4245691258,-1692374.3503603907,-1777603.2167238533,-1862832.083087321,-1948060.9494507844,-2033289.8158142427,-2118518.6821777145,-2203918.8577299537,-2289151.7078768034,-2374384.558023665,-2459617.4081705213,-2544850.258317382,-2630083.1084642345,-2715315.9586111,-2800548.8087579543,-2886003.3807831667,-2971240.463680261,-3056477.546577364,-3141714.629474448,-3226951.712371531,-3312188.7952686297,-3397425.8781657154,-3482662.9610628,-3567900.043959896,-3653418.3733925624,-3738659.945428157,-3823901.5174637353,-3909143.089499332,-3994384.6615349203,-4079626.2335705124,-4164867.805606108,-4250109.377641707,-4335350.9496772885,-4420592.521712878,-4505834.093748473],[3930992.794907786,3845144.4903301867,3759296.1857525883,3673447.881174992,3587599.5765973935,3501751.272019794,3415902.9674421973,3330054.6628645994,3244206.358287004,3158358.0537094036,3072509.749131808,2986661.444554209,2900813.1399766123,2814964.835399016,2729116.530821415,2643268.226243821,2557419.921666222,2471571.6170886243,2385723.312511027,2299875.0079334276,2214030.4924548427,2128193.843197578,2042365.0381907423,1956542.2792766455,1870724.8643748483,1784912.1485296495,1699103.5526634722,1613299.4519621488,1527498.5693940236,1441700.3317457358,1355904.582161503,1270112.2189940123,1184321.169918789,1098532.7525774622,1012745.8175405818,926960.7169362414,841176.9433696035,755394.9347728761,669613.3997827559,583834.2642105706,498055.1286383895,412277.59605235746,326501.11996055534,240724.64386875276,154948.16777694877,69171.69168514246,-16619.68479471281,-102398.8203668911,-188207.15979111847,-273989.16838784516,-359771.1769845751,-445553.18558130134,-531385.9933375483,-617171.093941893,-702956.1945462297,-788812.4289317159,-874600.8462730413,-960389.2636143672,-1046177.6809556931,-1131966.0982970195,-1217854.762267205,-1303646.7271382762,-1389438.6920093545,-1475230.656880429,-1561022.6217515017,-1646946.6532866699,-1732742.4028709047,-1818538.1524551385,-1904333.9020393728,-1990129.651623604,-2075925.4012078438,-2161891.8936069887,-2247691.671816324,-2333491.4500256535,-2419291.2282349756,-2505091.0064443033,-2590890.7846536245,-2676690.562862956,-2762490.341072279,-2848511.5548807504,-2934315.6127046533,-3020119.670528561,-3105923.728352459,-3191727.78617636,-3277531.844000264,-3363335.901824165,-3449139.959648069,-3534944.0174719635,-3620748.0752958683,-3706837.9869057015,-3792646.582771872,-3878455.1786380606,-3964263.774504232,-4050072.370370405,-4135880.966236594,-4221689.562102766,-4307498.157968945,-4393306.753835121,-4479115.349701291,-4564923.945567478],[3928029.919220093,3841612.9819347532,3755196.0446494217,3668779.10736408,3582362.170078744,3495945.232793405,3409528.2955080695,3323111.3582227333,3236694.4209373975,3150277.4836520604,3063860.546366725,2977443.6090813875,2891026.6717960513,2804609.734510716,2718192.7972253775,2631775.859940042,2545358.9226547047,2458941.9853693685,2372525.048084031,2286108.110798694,2199696.9311623303,2113294.501031758,2026898.8402226248,1940509.247293312,1854125.0471985233,1767745.5895493506,1681370.2469416608,1594998.4133490818,1508630.4999259782,1422265.6626659743,1335902.934462216,1249542.9221492026,1163185.347218953,1076828.9233269945,990475.5070872707,904122.4388337841,817772.218753637,731422.2794191048,645074.3224562393,558727.291062858,472380.6911909175,386036.35219094995,299692.0131909875,213347.67419102462,127003.33519104915,40645.10270257015,-45701.92869081255,-132048.9600841957,-218427.053260779,-304776.99259530986,-391126.9319298486,-477526.65322198626,-563879.721475475,-650232.7897289586,-736585.8579824399,-823012.4022960868,-909368.8261880465,-995725.2500799946,-1082081.6739719622,-1168537.4226151118,-1254897.434928128,-1341257.4472411461,-1427617.4595541498,-1513977.4718671716,-1600468.734225349,-1686832.5741365296,-1773196.4140477013,-1859560.2539588786,-1945924.0938700614,-2032287.933781242,-2118821.8638443095,-2205189.7772674253,-2291557.69069052,-2377925.604113627,-2464293.5175367314,-2550661.4309598347,-2637029.3443829305,-2723397.2578060403,-2809986.229867951,-2896358.4698065994,-2982730.709745243,-3069102.949683887,-3155475.1896225335,-3241847.429561178,-3328219.6694998303,-3414591.909438476,-3500964.149377119,-3587336.389315773,-3673994.505942147,-3760371.3328547115,-3846748.1597672794,-3933124.986679841,-4019501.8135924134,-4105878.6405049693,-4192255.4674175447,-4278632.294330103,-4365009.12124266,-4451385.948155232,-4537762.775067791,-4624139.601980358],[3925060.8276350917,3838074.064692032,3751087.301748966,3664100.5388059034,3577113.775862839,3490127.012919774,3403140.249976713,3316153.487033649,3229166.7240905887,3142179.961147523,3055193.198204458,2968206.435261394,2881219.672318332,2794232.90937527,2707246.1464322046,2620259.3834891426,2533272.6205460783,2446285.857603015,2359299.0946599515,2272312.62899628,2185334.4930046685,2098363.9213935453,2011400.1790336408,1924442.5590488783,1837490.3809834158,1750542.9890407957,1663599.750391372,1576660.0535442946,1489723.3067805043,1402790.1836361438,1315859.6840341683,1228930.7632463963,1142005.140009342,1055080.657794774,968158.4291647426,881237.4069812845,794318.3442553494,707399.8331816988,620483.7133934416,533567.5936051873,446653.39720987016,359740.0028133523,272826.608416833,185913.21402031323,98999.81962378835,12070.874875644222,-74845.24491261458,-161791.3457299536,-248710.40845588967,-335629.4711818234,-422597.2289157938,-509519.45754582435,-596441.6861758553,-683363.9148058924,-770358.5712058395,-857284.1944428976,-944209.8176799491,-1031135.440917009,-1118159.3890227107,-1205088.6416318808,-1292017.8942410676,-1378947.1468502348,-1465876.399459415,-1552936.0032721036,-1639869.1264164583,-1726802.249560813,-1813735.3727051867,-1900668.495849547,-1987601.618993904,-2074534.742138259,-2161641.3346956926,-2248578.5762826763,-2335515.8178696474,-2422453.059456626,-2509390.3010436,-2596327.542630582,-2683264.784217556,-2770422.6165929493,-2857364.231631874,-2944305.8466707747,-3031247.4617096893,-3118189.0767485956,-3205130.6917875027,-3292072.3068264145,-3379013.9218653208,-3465955.53690423,-3552897.1519431453,-3640124.5711741643,-3727070.8221453745,-3814017.073116594,-3900963.324087793,-3987909.5750590125,-4074855.8260302087,-4161802.07700143,-4248748.32797263,-4335694.578943842,-4422640.829915054,-4509587.080886248,-4596533.331857462,-4683846.411738574],[3922085.593948204,3834527.8265601797,3746970.0591721595,3659412.2917841366,3571854.524396111,3484296.7570080883,3396738.9896200686,3309181.2222320456,3221623.4548440264,3134065.6874560025,3046507.920067981,2958950.1526799565,2871392.3852919363,2783834.6179039143,2696276.8505158904,2608719.0831278693,2521161.315739846,2433603.548351825,2346045.7809638022,2258489.8578111692,2170942.3354268013,2083402.4430038445,1995869.6631097507,1908343.0905094363,1820821.6286300377,1733304.7747466518,1645792.4620018387,1558283.7241748348,1470777.9640406694,1383274.6025520419,1295774.930946351,1208277.0761381413,1120781.074914086,1033287.7062331634,945795.138650415,858305.2676217235,770815.9030007182,683328.5705731851,595842.1839650276,508356.45794743486,420872.8298133933,333389.20167935034,245905.57354530552,158421.94541126164,70923.86431606673,-16562.52229209151,-104048.90890025115,-191567.10783032048,-279056.4724513283,-366545.8370723268,-454085.9426735481,-541578.5102562956,-629071.077839043,-716634.9507910674,-804130.9520151266,-891626.9532391764,-979122.9544632304,-1066618.9556872877,-1154215.8740129457,-1241715.5456186365,-1329215.2172243306,-1416714.888830008,-1504214.5604357002,-1591847.5154604153,-1679351.1005890113,-1766854.6857176144,-1854358.2708462272,-1941861.855974821,-2029365.4411034225,-2117041.714695216,-2204549.463239743,-2292057.211784268,-2379564.960328789,-2467072.7088733166,-2554580.4574178373,-2642088.2059623646,-2729815.9863517787,-2817328.1553188497,-2904840.3242859244,-2992352.4932529936,-3079864.662220074,-3167376.8311871393,-3254889.000154214,-3342401.169121283,-3429913.3380883625,-3517425.5070554214,-3605223.3121398017,-3692740.166023194,-3780257.0199065795,-3867773.8737899614,-3955290.727673334,-4042807.5815567235,-4130324.435440121,-4217841.28932351,-4305358.143206884,-4392874.997090265,-4480391.850973662,-4567908.70485704,-4655792.746964664,-4743314.55813099],[3919104.2917121667,3830974.3552081333,3742844.418704108,3654714.4822000777,3566584.5456960513,3478454.609192019,3390324.6726879883,3302194.7361839595,3214064.7996799317,3125934.8631759016,3037804.9266718705,2949674.9901678427,2861545.053663814,2773415.117159783,2685285.180655754,2597155.2441517245,2509025.307647695,2420895.371143666,2332765.4346396355,2244638.8884680513,2156520.8220469337,2068411.5103465854,1980308.7312386539,1892211.7843856586,1804119.9945412073,1716032.7098003435,1627949.299914862,1539869.8797290474,1451793.942914683,1363720.4271231177,1275649.171927968,1187581.2205753962,1099514.3306140243,1011450.2596071826,923387.138034259,835326.3071180077,747265.979036771,659208.1612858945,571150.3435350163,483094.65262608766,395039.6265145363,306984.60040298477,218929.5742914374,130874.54817988724,42803.43702975847,-45254.38072111271,-133342.83296078118,-221403.66387703503,-309464.49479328375,-397574.87650231086,-485638.94750916073,-573703.0185160036,-661767.0895228474,-749904.7418555976,-837972.2856024648,-926039.8293493255,-1014107.373096189,-1102274.7183410819,-1190345.9735362316,-1278417.2287313826,-1366488.4839265337,-1454559.7391216913,-1542763.2567626834,-1630838.4685179056,-1718913.6802731236,-1806988.892028334,-1895064.1037835507,-1983139.3155387738,-2071386.3478894052,-2159465.768075218,-2247545.188261035,-2335624.6084468495,-2423704.028632656,-2511783.448818489,-2599862.8690042933,-2687942.289190119,-2776245.5584129216,-2864329.4460249515,-2952413.3336369796,-3040497.2212489946,-3128581.1088610254,-3216664.9964730516,-3304748.884085076,-3392832.771697104,-3480916.6593091357,-3569285.9192082398,-3657374.540745102,-3745463.1622819817,-3833551.7838188633,-3921640.4053557375,-4009729.0268926118,-4097817.6484294925,-4185906.269966375,-4273994.891503234,-4362083.513040122,-4450172.13457699,-4538260.756113865,-4626716.8247766495,-4714810.454630206,-4802904.084483735],[3916116.994228097,3827413.7380049084,3738710.481781717,3650007.22555852,3561303.96933533,3472600.713112134,3383897.4568889425,3295194.2006657477,3206490.944442556,3117787.688219361,3029084.431996166,2940381.1757729715,2851677.91954978,2762974.6633265866,2674271.407103394,2585568.1508802013,2496864.894657006,2408161.638433812,2319458.38221062,2230760.0615568766,2142071.789704074,2053390.844047297,1964716.4882552992,1876048.0166285501,1787384.7485617145,1698726.0267718588,1610071.2155914702,1521419.6993225412,1432771.710358827,1344126.8913297784,1255483.9423503564,1166843.7056027963,1078205.5798731546,989568.8550274456,900934.5381983209,812301.0906419568,723669.5977112213,635039.1985488681,546409.5712017184,457781.99692551605,369154.42264930764,280526.84837310016,191899.27409689408,103256.80357475718,14626.404412408825,-74003.99474994512,-162666.83247804875,-251300.28003441682,-339933.72759078117,-428618.74250386236,-517255.4673495726,-605892.1921952814,-694601.2660959694,-783241.5028435322,-871881.7395910872,-960521.9763386492,-1049260.8202654002,-1137904.809583738,-1226548.798902079,-1315192.788220415,-1403836.7775387568,-1492611.9254437103,-1581259.9144074335,-1669907.9033711394,-1758555.892334851,-1847203.8812985695,-1935851.8702622754,-2024670.7248227983,-2113322.967271923,-2201975.2097210605,-2290627.4521701783,-2379279.6946193064,-2467931.93706844,-2556584.1795175653,-2645236.421966699,-2734111.818771989,-2822768.575682886,-2911425.332593751,-3000082.0895046275,-3088738.8464155067,-3177395.6033263896,-3266052.3602372613,-3354709.1171481432,-3443365.8740590103,-3532307.643500616,-3620969.183368326,-3709630.7232360216,-3798292.2631037254,-3886953.8029714366,-3975615.3428391414,-4064276.8827068536,-4152938.4225745583,-4241599.962442266,-4330261.502309969,-4418923.042177672,-4507584.582045393,-4596613.727380053,-4685280.326613056,-4773946.925846032,-4862613.525079048],[3913123.7745369524,3823846.062009399,3734568.3494818453,3645290.63695429,3556012.924426736,3466735.2118991762,3377457.4993716227,3288179.7868440696,3198902.0743165165,3109624.3617889592,3020346.6492614034,2931068.9367338503,2841791.224206295,2752513.51167874,2663235.799151184,2573958.086623631,2484680.374096075,2395402.66156852,2306124.949040965,2216855.203274857,2127594.101359248,2038340.3847789555,1949093.3155409028,1859852.1826109095,1770616.3000508472,1681385.0052243252,1592157.6570669794,1502933.634417599,1413712.3344064157,1324494.474440543,1235279.1046790292,1146065.0578863453,1056854.5427938113,967644.5824344114,878437.381898589,789230.5071658313,700026.3903275027,610822.2734891712,521620.36855247966,432419.10992751084,343217.8513025413,254016.5926775639,164815.33405260183,75597.57245919341,-13606.544379135128,-102841.82399543654,-192049.02453125874,-281256.22506708,-370513.69868675293,-459724.2137792874,-548934.7288718238,-638145.2439643643,-727430.3521179901,-816644.418336038,-905858.4845540868,-995072.5507721319,-1084387.744637989,-1173605.60460386,-1262823.4645697298,-1352041.3245356046,-1441259.18450147,-1530611.0588594214,-1619832.9616028713,-1709054.8643463114,-1798276.7670897502,-1887498.6698331921,-1976720.5725766383,-2066116.597159394,-2155342.7984819957,-2244568.9998045852,-2333795.2011271836,-2423021.4024497764,-2512247.603772369,-2601473.8050949657,-2690922.374455261,-2780153.137305878,-2869383.900156468,-2958614.663007076,-3047845.42585768,-3137076.188708286,-3226306.951558887,-3315537.7144095004,-3404768.477260095,-3494283.796793659,-3583519.391655433,-3672754.9865172002,-3761990.5813789787,-3851226.176240744,-3940461.771102529,-4029697.365964287,-4118932.960826058,-4208168.555687832,-4297404.150549596,-4386639.745411376,-4475875.340273137,-4565478.598239573,-4654719.303529175,-4743960.008818759,-4833200.714108351,-4922441.419397925],[3910124.705411032,3820271.413960322,3730418.1225096146,3640564.831058903,3550711.5396081912,3460858.2481574765,3371004.9567067684,3281151.6652560565,3191298.3738053488,3101445.0823546355,3011591.7909039245,2921738.499453214,2831885.2080025044,2742031.9165517925,2652178.6251010792,2562325.333650371,2472472.0421996578,2382618.7507489473,2292766.4474303913,2202923.167959111,2113088.1090954803,2023260.97742433,1933440.6103121056,1843625.8294200115,1753815.9584127893,1664010.3428984066,1574209.0600525849,1484411.4095877893,1394616.5071434882,1304823.7609311328,1215034.7427294683,1125247.1664545527,1035461.738662594,945678.2921827869,855896.2162822522,766115.8887152947,676336.9318893217,586558.7388531638,496782.6736469453,407006.6084407279,317230.5432345057,227454.4780282844,137663.19093347667,47884.234107505064,-41894.7227184684,-131706.6186236078,-221488.69452413963,-311270.7704246696,-401105.1060087676,-490890.5338007258,-580675.961592684,-670534.6395055237,-760323.657707185,-850112.6759088552,-939901.6941105174,-1029790.5287113991,-1119583.3818912096,-1209376.2350710207,-1299169.0882508322,-1388961.9414306404,-1478887.5818743748,-1568684.5210095858,-1658481.4601448025,-1748278.3992800196,-1838075.3384152437,-1927872.2775504608,-2017842.253780271,-2107643.536626083,-2197444.819471912,-2287246.1023177425,-2377047.3851635605,-2466848.668009382,-2556649.9508552058,-2646672.723600953,-2736478.6150706178,-2826284.5065402817,-2916090.3980099494,-3005896.2894796273,-3095702.1809492875,-3185508.0724189607,-3275313.963888621,-3365119.8553582923,-3455209.7514118273,-3545020.5239683017,-3634831.2965247706,-3724642.069081259,-3814452.8416377148,-3904263.614194191,-3994074.386750659,-4083885.1593071437,-4173695.9318636097,-4263506.70442008,-4353317.476976551,-4443128.249533027,-4533306.64199084,-4623122.5760506,-4712938.510110362,-4802754.444170134,-4892570.378229892,-4982386.312289663],[3907119.8593458068,3816689.8802664187,3726259.901187025,3635829.9221076337,3545399.9430282484,3454969.9639488547,3364539.9848694634,3274110.0057900706,3183680.026710683,3093250.0476312917,3002820.0685519013,2912390.0894725095,2821960.1103931176,2731530.131313728,2641100.152234336,2550670.1731549473,2460240.1940755546,2369810.214996163,2279382.7029813044,2188964.2887536143,2098555.12813021,2008153.4744176161,1917758.1616078368,1827368.4843466193,1736983.7609701292,1646603.3317151687,1556226.5569902572,1465853.4727573642,1375483.8606485322,1285116.4248508038,1194751.3460560213,1104389.1338351718,1014027.7059170606,923669.6461676478,833311.6056672719,742956.7004418266,652601.7952163839,562249.0671736572,471897.0870523392,381545.1069310163,291193.12680969713,200841.14668838028,110472.36386371683,20117.45863826899,-70269.01107590273,-160627.07082531555,-250985.13057473022,-341394.0508307968,-431755.4998724172,-522116.9489140436,-612478.3979556649,-702915.3086344358,-793280.387429215,-883645.4662240008,-974010.5450187852,-1064477.9258433003,-1154846.8808985515,-1245215.835953807,-1335584.7910090624,-1426085.2226952296,-1516458.3069281261,-1606831.3911610222,-1697204.4753939142,-1787577.5596268075,-1877950.6438596924,-1968495.592743576,-2058873.0658550505,-2149250.5389665137,-2239628.012077981,-2330005.4851894537,-2420382.958300925,-2510760.4314123914,-2601358.4242905304,-2691740.553150135,-2782122.6820097305,-2872504.8108693305,-2962886.9397289166,-3053269.0685885157,-3143651.197448114,-3234033.3263077065,-3324415.4551673057,-3415080.940105764,-3505467.9991480457,-3595855.058190332,-3686242.1172326133,-3776629.1762748985,-3867016.2353171734,-3957403.2943594595,-4047790.353401741,-4138177.4124440346,-4228564.471486302,-4318951.530528589,-4409338.589570874,-4500093.124337585,-4590485.395970625,-4680877.667603654,-4771269.939236681,-4861662.210869721,-4952054.482502767,-5042446.7541358],[3904109.3085518596,3813101.546996862,3722093.7854418675,3631086.023886871,3540078.2623318746,3449070.500776871,3358062.7392218765,3267054.97766688,3176047.2161118854,3085039.454556886,2994031.6930018887,2903023.9314468913,2812016.1698918967,2721008.4083368964,2630000.646781899,2538992.885226902,2447985.1236719047,2356977.3621169077,2265973.5457303766,2174979.902383626,2083995.0213195067,1993017.2721911033,1902045.9188126405,1811080.250546644,1720119.5804256853,1629163.2433386235,1538210.5942773614,1447261.0066402717,1356314.8548984071,1265371.6364402329,1174430.0264582569,1083491.4614645373,992554.3117085837,901619.1734726613,810685.4472637265,719753.4990722865,628822.1860318391,537893.1965056623,446964.2069794908,356035.217453321,265106.2279271432,174161.8099236358,83229.86173219327,-7702.086459242273,-98667.34712247504,-189602.48535839468,-280537.6235943171,-371525.57843496837,-462464.1434286842,-553402.7084224029,-644415.280674554,-735357.514822878,-826299.7489712047,-917241.9831195199,-1008285.0903900936,-1099231.2421319582,-1190177.3938738229,-1281123.5456156875,-1372069.6973575559,-1463150.1038246313,-1554100.428009538,-1645050.7521944437,-1736001.0763793518,-1826951.4005642706,-1917901.72474918,-2009027.0873329728,-2099981.845599761,-2190936.6038665515,-2281891.3621333423,-2372846.1204001303,-2463800.8786669113,-2554755.6369337104,-2645934.555567464,-2736894.016733991,-2827853.4779005134,-2918812.9390670313,-3009772.400233562,-3100731.8614000836,-3191691.3225666117,-3282650.783733135,-3373610.2448996576,-3464857.29638255,-3555821.7368468475,-3646786.177311141,-3737750.61777544,-3828715.058239742,-3919679.4987040414,-4010643.939168336,-4101608.379632622,-4192572.820096921,-4283537.260561226,-4374501.701025521,-4465833.371936127,-4556803.076089689,-4647772.78024323,-4738742.484396801,-4829712.1885503465,-4920681.892703895,-5011651.596857455,-5102621.3010110045],[3901093.124947097,3809506.499871998,3717919.8747968953,3626333.249721792,3534746.6246466874,3443159.999571581,3351573.374496479,3259986.7494213786,3168400.1243462795,3076813.499271174,2985226.874196071,2893640.249120968,2802053.6240458675,2710466.9989707624,2618880.3738956572,2527293.748820557,2435707.123745454,2344120.498670351,2252539.8556940556,2160969.7763933353,2069407.6583055633,1977852.7324642204,1886304.420314987,1794761.694412094,1703223.8198194099,1611690.4945730092,1520160.891148643,1428634.3774533207,1337110.3380350918,1245589.8675658228,1154071.4106059403,1062554.6487494486,971040.6986097249,879527.4010401682,788016.9110150815,696506.8390803838,604998.788483629,513491.70885092183,421985.10018034745,330477.54958550725,238970.4699528017,147446.40747221513,55936.33553751791,-35605.57465537684,-127118.87222493021,-218632.1697944817,-310196.77909790864,-401713.5409543966,-493230.3028108934,-584819.541819531,-676340.0122887981,-767860.4827580638,-859380.9532273291,-951000.788126607,-1042525.2175719887,-1134049.647017375,-1225574.076462749,-1317098.5059081316,-1408755.7551162662,-1500284.40031204,-1591813.0455078194,-1683341.6907036016,-1774870.3358993786,-1866398.9810951534,-1958101.361517022,-2049634.4860321484,-2141167.610547277,-2232700.735062402,-2324233.8595775333,-2415766.9840926635,-2507300.108607798,-2599056.286036781,-2690594.160629466,-2782132.03522214,-2873669.909814829,-2965207.7844075086,-3056745.6590001974,-3148283.533592878,-3239821.4081855686,-3331359.282778248,-3423183.955060904,-3514726.8580846274,-3606269.7611083556,-3697812.664132069,-3789355.567155795,-3880898.4701795178,-3972441.373203243,-4063984.2762269564,-4155527.179250691,-4247070.082274407,-4338612.985298127,-4430522.772266176,-4522070.990087772,-4613619.20790937,-4705167.425730954,-4796715.643552562,-4888263.861374164,-4979812.07919577,-5071360.297017374,-5162908.5148389805],[3898071.380149117,3805904.824254199,3713738.2683592853,3621571.712464367,3529405.1565694516,3437238.600674528,3345072.0447796155,3252905.488884697,3160738.932989782,3068572.377094863,2976405.821199945,2884239.2653050288,2792072.7094101147,2699906.1535151955,2607739.597620277,2515573.0417253627,2423406.485830443,2331239.9299355256,2239082.2582360012,2146933.808427163,2054793.677060379,1962661.3502681958,1870535.1052065897,1778414.2359074994,1686298.058601115,1594185.9098999626,1502077.8763724002,1409973.3706102762,1317871.3621481631,1225771.5881260387,1133675.218909757,1041579.901649772,949487.3776324543,857395.7980746073,765306.5353511665,673217.6572049668,581131.42049462,489045.1837842751,396958.9470739295,304872.7103635841,212770.95901052654,120681.69628708996,28592.43356365245,-63530.38655803213,-155622.91057534842,-247715.43459266517,-339861.1954213539,-431957.22131705284,-524053.24721274525,-616223.8923164136,-708323.666338542,-800423.4403606658,-892523.2143827947,-984724.7645455711,-1076828.5389746404,-1168932.313403715,-1261036.0878327875,-1353139.8622618527,-1445379.1965096416,-1537487.2300372096,-1629595.2635647655,-1721703.2970923334,-1813811.3306198926,-1906091.7079769927,-1998204.266094374,-2090316.8242117702,-2182429.382329153,-2274541.9404465454,-2366654.498563931,-2458767.056681318,-2551101.467793099,-2643218.8231909727,-2735336.178588873,-2827453.53398675,-2919570.889384631,-3011688.2447825186,-3103805.600180405,-3195922.955578286,-3288040.310976173,-3380443.574616366,-3472566.007595705,-3564688.4405750595,-3656810.8735544067,-3748933.3065337576,-3841055.739513117,-3933178.1724924594,-4025300.605471802,-4117423.0384511612,-4209545.47143052,-4301667.904409855,-4394156.77348983,-4486284.572384872,-4578412.371279916,-4670540.170174955,-4762667.969069998,-4854795.767965042,-4946923.566860094,-5039051.365755132,-5131179.164650183,-5223306.963545231],[3895044.1454678103,3802296.60513911,3709549.064810407,3616801.5244817,3524053.9841529964,3431306.4438242954,3338558.903495588,3245811.3631668882,3153063.8228381826,3060316.2825094783,2967568.7421807786,2874821.201852074,2782073.66152337,2689326.121194666,2596578.5808659615,2503831.0405372605,2411083.500208556,2318336.98082024,2225600.2043376667,2132872.7211990464,2040154.2293206183,1947442.606675011,1854737.1179859089,1762037.0520324958,1669341.719747967,1576650.4523779536,1483962.5996947829,1391278.4266796291,1298597.3971332791,1205918.2763206176,1113241.932884598,1020567.6615013042,927894.8575961678,835224.3632764842,742554.8563916569,649887.0925687414,557220.6454813185,464554.58843873814,371887.75130646955,279221.30421904474,186537.8150827242,93868.30819790065,1166.8185447915457,-91505.98536034627,-184178.78926548548,-276903.218655705,-369579.5620917254,-462255.9055277421,-555005.2254734496,-647685.3566037822,-740365.4877341273,-833045.6188644585,-925825.8986992068,-1018510.0717143524,-1111194.2447295072,-1203878.417744663,-1296562.5907598222,-1389380.7636632812,-1482069.239164574,-1574757.7146658655,-1667446.190167143,-1760134.6656684303,-1852823.1411697278,-1945687.0505962274,-2038380.095989673,-2131073.141383114,-2223766.1867765626,-2316459.232170006,-2409152.277563436,-2501845.3229568806,-2594763.773310589,-2687461.663211515,-2780159.553112437,-2872857.443013345,-2965555.3329142677,-3058253.2228151746,-3150951.1127160955,-3243649.002617006,-3336631.814356353,-3429334.8310053116,-3522037.847654282,-3614740.864303259,-3707443.8809522185,-3800146.8976011854,-3892849.914250166,-3985552.93089913,-4078255.9475481035,-4170958.9641970675,-4263661.980846036,-4356364.997495004,-4449439.317987959,-4542147.751678736,-4634856.185369493,-4727564.619060262,-4820273.052751043,-4912981.486441796,-5005689.920132562,-5098398.353823341,-5191106.78751411,-5283815.221204884],[3892011.491898115,3798681.9271469447,3705352.3623957727,3612022.797644602,3518693.2328934316,3425363.6681422596,3332034.10339109,3238704.5386399217,3145374.9738887497,3052045.409137579,2958715.844386409,2865386.2796352375,2772056.714884069,2678727.1501328964,2585397.5853817263,2492068.0206305566,2398738.455879383,2305411.323310813,2212094.024477931,2118787.445611988,2025488.5601309333,1932196.6015247758,1838910.8293098107,1745630.5270378154,1652355.000368956,1559083.575204087,1465815.5958723384,1372550.4233699227,1279288.8951922348,1186029.7327837837,1092772.0319213003,999517.7686951272,906263.9895040826,813013.1986977782,719762.4186802397,626514.7215276593,533267.0243750745,440019.32722249045,346771.63006991055,253508.4538599546,160257.66305364948,67006.87224734854,-26277.591116754804,-119531.71473601693,-212785.83835527906,-306093.48101592157,-399351.18187840004,-492608.8827408855,-585941.6683615576,-679203.1965392507,-772464.7247169395,-865726.2528946232,-959090.3053686763,-1052355.9169548685,-1145621.5285410634,-1238887.1401272495,-1332285.1072442308,-1425555.0647425796,-1518825.0222409042,-1612094.9797392394,-1705364.937237572,-1798634.8947359035,-1892078.7631253172,-1985353.335848845,-2078627.9085723902,-2171902.481295916,-2265177.0540194446,-2358451.62674298,-2451726.1994665195,-2545224.8390859785,-2638504.3035669327,-2731783.768047883,-2825063.2325288304,-2918342.697009784,-3011622.161490743,-3104901.6259716917,-3198181.090452645,-3291460.5549335973,-3385029.0336927343,-3478313.674103489,-3571598.3145142524,-3664882.954925012,-3758167.595335776,-3851452.2357465317,-3944736.876157295,-4038021.5165680572,-4131306.1569788093,-4224590.797389569,-4317875.437800332,-4411530.782327152,-4504820.890913224,-4598110.999499276,-4691401.108085343,-4784691.216671381,-4877981.3252574485,-4971271.433843498,-5064561.542429556,-5157851.651015617,-5251141.759601689,-5344431.868187746],[3888973.4901129864,3795060.874514166,3701148.2589153415,3607235.6433165194,3513323.0277177007,3419410.4121188736,3325497.7965200534,3231585.1809212333,3137672.5653224126,3043759.9497235906,2949847.334124769,2855934.718525948,2762022.1029271246,2668109.4873283026,2574196.8717294815,2480284.256130661,2386371.640531838,2292462.867561112,2198565.3794348156,2104677.3172170315,2010797.0113898087,1916923.8988598282,1823057.1090117116,1729195.4230512239,1635338.2965016016,1541485.687939955,1447636.5566539017,1353790.2580879806,1259946.3064783348,1166106.0778609607,1072267.1835671929,978430.7139595945,884595.923061931,790762.8221249338,696931.0622787983,603101.0889226883,509271.3102875659,415441.14221046935,321611.16885436466,227764.216502944,133931.11556594307,40066.01788361464,-53770.45172398584,-147606.92133157374,-241495.19146549562,-335335.27608726127,-429175.3607090302,-523088.77410492394,-616932.7257145708,-710776.6773242159,-804620.628933867,-898565.3580072559,-992413.434593705,-1086261.5111801452,-1180109.587766596,-1273957.664353039,-1367940.756875223,-1461793.2228370002,-1555645.6887987838,-1649498.1547605628,-1743350.6207223358,-1837375.385834448,-1931232.5123840766,-2025089.638933695,-2118946.7654833174,-2212803.892032948,-2306661.018582562,-2400518.1451321817,-2494597.9076172444,-2588459.9731961326,-2682322.0387750324,-2776184.1043539084,-2870046.169932803,-2963908.235511694,-3057770.301090587,-3151632.3666694798,-3245494.432248368,-3339644.3806002084,-3433511.6713048527,-3527378.9620094933,-3621246.2527141385,-3715113.543418782,-3808980.8341234233,-3902848.12482806,-3996715.4155327138,-4090582.7062373506,-4184449.9969419995,-4278317.287646641,-4372554.581079321,-4466427.391099273,-4560300.2011192255,-4654173.011139177,-4748045.821159133,-4841918.631179092,-4935791.441199037,-5029664.251218982,-5123537.061238953,-5217409.871258881,-5311282.6812788425,-5405155.491298803],[3885930.2104566153,3791433.5310853613,3696936.8517141053,3602440.1723428504,3507943.492971594,3413446.8136003367,3318950.134229078,3224453.454857825,3129956.7754865703,3035460.0961153135,2940963.4167440557,2846466.7373728002,2751970.0580015453,2657473.378630288,2562976.6992590316,2468480.019887776,2373983.3405165197,2279492.4053349495,2185013.2218161286,2090542.662839545,1996080.7511548316,1901625.7758170809,1807176.661647411,1712732.695719312,1618293.1856414163,1523857.4576784149,1429425.90315525,1334997.466987795,1240571.254287288,1146147.773063759,1051726.814124044,957306.9857430209,862890.1727276552,768473.7489386757,674059.9992746634,579646.7370634135,485234.9903176178,390820.21264092205,296406.9504296733,201975.20620479295,107558.78241581097,13108.701932817698,-81311.12644820707,-175730.95482922718,-270204.445289169,-364627.9265126949,-459051.40773621993,-553550.2917052279,-647977.679639875,-742405.0675745276,-836832.455509177,-931362.9599238858,-1025794.5144470497,-1120226.0689702146,-1214657.6234933762,-1309222.4236878627,-1403658.4110855805,-1498094.3984833006,-1592530.385881017,-1686966.373278737,-1781402.3606764572,-1876013.6525947927,-1970454.3459714777,-2064895.039348167,-2159335.732724856,-2253776.4261015416,-2348217.119478225,-2442657.812854916,-2537324.6044910247,-2631770.2841896852,-2726215.963888352,-2820661.643587007,-2915107.323285669,-3009553.0029843235,-3103998.6826829882,-3198444.3623816473,-3293176.6960300216,-3387627.650063593,-3482078.6040971894,-3576529.5581307635,-3670980.5121643525,-3765431.4661979415,-3859882.420231521,-3954333.3742651073,-4048784.3282986823,-4143235.2823322685,-4237686.236365854,-4332506.389979498,-4426962.914474042,-4521419.43896859,-4615875.963463129,-4710332.487957666,-4804789.012452197,-4899245.536946764,-4993702.061441279,-5088158.585935826,-5182615.110430373,-5277071.634924933,-5371528.159419452,-5465984.683914014],[3882881.722937783,3787799.9803053313,3692718.237672891,3597636.495040443,3502554.7524079964,3407473.0097755482,3312391.2671431,3217309.524510654,3122227.7818782106,3027146.0392457633,2932064.296613316,2836982.5539808697,2741900.8113484248,2646819.0687159766,2551737.3260835307,2456655.5834510857,2361573.840818637,2266500.612183297,2171437.8634657646,2076384.8017019886,1981339.801410648,1886301.452670897,1791269.0149111832,1696241.7699846397,1601219.0202079285,1506200.0864540609,1411184.306294892,1316172.4825694752,1221163.1549473815,1126155.4431967312,1031151.2566820823,936147.9839423567,841147.2380037969,746147.3062676492,651149.7559721738,556152.2056767005,461154.6553812295,366157.10508575593,271142.76237243554,176142.0164338732,81141.27049531322,-13894.794979052618,-108898.98149370216,-203955.00367793022,-298962.88091961294,-393970.75816130685,-488978.63540299237,-584063.9919388425,-679075.8156642588,-774087.6393896746,-869200.7127427883,-964216.7447106629,-1059232.7766785338,-1154248.8086464107,-1249264.8406142797,-1344416.7394642849,-1439437.2478408031,-1534457.7562173218,-1629478.2645938476,-1724498.7729703728,-1819692.841653294,-1914718.1014272943,-2009743.3612012877,-2104768.6209752904,-2199793.8807492917,-2294819.1405232865,-2389844.4002972776,-2485094.188471477,-2580124.4818799775,-2675154.7752884775,-2770185.0686969906,-2865215.362105498,-2960245.6555139907,-3055275.9489225084,-3150306.242331004,-3245621.86343175,-3340657.4803966107,-3435693.0973614743,-3530728.714326336,-3625764.331291207,-3720799.94825607,-3815835.5652209297,-3910871.1821857933,-4005906.799150654,-4100942.416115515,-4195978.033080381,-4291381.944622061,-4386423.1831983095,-4481464.421774561,-4576505.660350816,-4671546.898927064,-4766588.137503321,-4861629.376079565,-4956670.614655835,-5051711.853232077,-5146753.091808335,-5241794.330384574,-5336835.568960838,-5431876.807537097,-5526918.046113349],[3879828.0972234146,3784160.305211444,3688492.513199469,3592824.7211874994,3497156.929175529,3401489.137163557,3305821.345151582,3210153.5531396135,3114485.7611276424,3018817.9691156703,2923150.177103698,2827482.385091725,2731814.5930797546,2636146.8010677826,2540479.009055809,2444811.217043839,2349143.8233111287,2253486.9153581713,2157840.294933286,2062203.4030605126,1966573.9880594453,1870951.2797816703,1775334.5325049516,1679723.0228864995,1584116.0479739676,1488512.9232696746,1392912.9808436627,1297315.7352235198,1201722.308175651,1106130.512282086,1010540.9816330089,914953.6699842997,819367.6159567828,723783.6803605512,628200.8561088312,532619.2101451764,437035.2076053936,341452.3833536729,245851.29179166956,150265.23776415316,54645.675142241176,-40943.855506837834,-136533.3861559159,-232176.58155891718,-327769.84087455506,-423363.1001901948,-519031.9315592032,-614629.1771793473,-710226.4227994904,-805823.6684196298,-901524.4656703505,-997125.9612279525,-1092727.4567855513,-1188328.9523431547,-1284064.4180484917,-1379670.4335825676,-1475276.449116652,-1570882.4646507315,-1666488.4801848168,-1762094.4957189,-1857877.035038733,-1953487.847415092,-2049098.6597914505,-2144709.4721678062,-2240320.2845441657,-2335931.0969205205,-2431764.7745516244,-2527380.6678938246,-2622996.5612360197,-2718612.4545782106,-2814228.3479203973,-2909844.2412626036,-3005460.1346047875,-3101076.0279469853,-3196691.921289187,-3292597.0913096173,-3388218.3574409327,-3483839.623572265,-3579460.8897035746,-3675082.155834892,-3770703.421966221,-3866324.688097544,-3961945.954228854,-4057567.220360186,-4153188.4864914985,-4248809.75262282,-4344803.979147389,-4440430.918044457,-4536057.856941546,-4631684.795838605,-4727311.734735678,-4822938.673632754,-4918565.612529831,-5014192.551426913,-5109819.4903239645,-5205446.429221051,-5301073.368118116,-5396700.307015195,-5492327.245912269,-5587954.184809346],[3876769.402632336,3780514.588426124,3684259.774219903,3588004.960013684,3491750.1458074646,3395495.331601247,3299240.517395026,3202985.703188808,3106730.888982594,3010476.074776372,2914221.260570155,2817966.446363935,2721711.6321577183,2625456.8179514995,2529202.00374528,2432947.189539062,2336694.1175043257,2240451.6103350595,2144220.7892555166,2047998.4491793977,1951783.6464325888,1855576.0425147712,1759374.1674669608,1663177.1529363296,1566984.6573633256,1470796.3698123652,1374611.293515815,1278428.7696214947,1182249.15547003,1086072.3045123152,989896.4707007797,893724.135936474,797551.9623873634,701382.891597779,605213.820808189,509044.7500186034,412875.6792290155,316690.127350647,220517.79258634197,124345.45782203972,28137.9821049585,-68037.86538662156,-164265.44265047926,-260445.05680194357,-356624.6709534051,-452804.2851048717,-549061.5129588628,-645245.1532825916,-741428.793606326,-837713.9980936386,-933901.930089721,-1030089.8620858057,-1126277.7940818863,-1222465.7260779664,-1318790.209826821,-1414982.70539982,-1511175.2009728244,-1607367.6965458323,-1703560.1921188422,-1799927.334289059,-1896124.6721743466,-1992322.0100596612,-2088519.3479449525,-2184716.68583025,-2280914.0237155473,-2377111.3616008433,-2473534.936496814,-2569737.4026970766,-2665939.8688973323,-2762142.335097582,-2858344.801297843,-2954547.2674981034,-3050749.7336983643,-3146952.199898623,-3243442.4705647733,-3339650.3587973146,-3435858.247029867,-3532066.1352624055,-3628274.0234949505,-3724481.9117275067,-3820689.7999600405,-3916897.6881925873,-4013105.576425139,-4109313.46465768,-4205521.352890219,-4302101.1436917195,-4398314.755847483,-4494528.368003239,-4590741.980158991,-4686955.592314748,-4783169.204470513,-4879382.81662626,-4975596.428782027,-5071810.040937786,-5168023.653093555,-5264237.265249297,-5360450.877405066,-5456664.489560821,-5552878.101716577,-5649573.279998574],[3873705.7081292663,3776862.9121497287,3680020.116170193,3583177.3201906593,3486334.524211119,3389491.728231583,3292648.9322520485,3195806.1362725124,3098963.3402929804,3002120.5443134434,2905277.7483339077,2808434.9523543715,2711592.1563748377,2614749.3603952997,2517906.5644157636,2421063.7684362284,2324224.0577564817,2227396.1617824016,2130579.008710344,2033770.3552680945,1936970.2474729717,1840176.505129252,1743388.3863820555,1646605.1700809477,1549826.1537740203,1453050.8257191,1356279.657140732,1259511.0645984989,1162744.3961352794,1065981.272318023,969219.4355621133,872459.8606369204,775701.7219075996,678945.4452210998,582189.8922351468,485432.89184810617,388676.61516160774,291902.41108575696,195142.83616056386,98350.03414544929,1586.9103281940334,-95176.21348906634,-191992.86324790632,-288759.7917709984,-385526.7202940942,-482369.24042910244,-579140.2350380411,-675911.229646985,-772682.2242559195,-869557.0475989725,-966332.37565389,-1063107.7037088098,-1159883.0317637296,-1256792.88783227,-1353572.823096044,-1450352.758359816,-1547132.6936235894,-1643912.6288873563,-1740692.5641511423,-1837650.067216686,-1934434.8902869294,-2031219.7133571575,-2128004.536427398,-2224789.3594976305,-2321574.182567866,-2418583.4399196077,-2515373.43867072,-2612163.4374218346,-2708953.4361729487,-2805743.4349240568,-2902533.433675166,-2999323.4324262757,-3096113.4311773907,-3193189.664437092,-3289985.134473133,-3386780.6045091795,-3483576.0745452186,-3580371.544581253,-3677167.0146173015,-3773962.4846533304,-3870757.954689374,-3967553.424725403,-4064348.894761443,-4161144.3647974804,-4258310.577466032,-4355111.822585008,-4451913.06770401,-4548714.312822985,-4645515.557941963,-4742316.803060952,-4839118.048179939,-4935919.293298935,-5032720.538417916,-5129521.7835369175,-5226323.028655893,-5323124.2737748865,-5419925.518893867,-5516726.764012853,-5614009.020504177,-5710816.353174619],[3870637.082318913,3773205.358153553,3675773.633988193,3578341.9098228277,3480910.1856574705,3383478.4614921063,3286046.737326747,3188615.0131613873,3091183.288996026,2993751.564830664,2896319.8406653036,2798888.1164999427,2701456.3923345823,2604024.668169219,2506592.9440038586,2409161.219838499,2311733.9233367844,2214319.8327544,2116915.258877612,2019520.5357097764,1922133.1382117812,1824752.1586881133,1727376.8501413872,1630006.4862070242,1532640.3591173552,1435277.7777124452,1337918.482394957,1240562.8762196188,1143209.2122408622,1045857.8657706829,948509.0817823191,851161.320426906,753816.2726266822,656471.8438378675,559129.4094289283,461782.98626023834,364438.55747142574,267074.75229443656,169726.99093902297,72344.40106888535,-25006.945401294157,-122358.29187147086,-219764.9628816559,-317120.1521550743,-414475.3414284969,-511908.1286373134,-609267.4239548035,-706626.719272289,-804087.7346753841,-901451.4052500823,-998815.0758247795,-1096178.7463994785,-1193542.4169741739,-1291043.1571691288,-1388411.478615007,-1485779.8000608804,-1583148.1215067469,-1680516.442952626,-1778060.3215643293,-1875433.5763339251,-1972806.8311035358,-2070180.0858731396,-2167553.3406427437,-2264926.5954123405,-2362299.850181947,-2459900.865928597,-2557279.3437608015,-2654657.821592996,-2752036.299425202,-2849414.777257393,-2946793.255089597,-3044171.7329218,-3141550.2107539903,-3239218.7758402936,-3336602.774218631,-3433986.772596971,-3531370.770975317,-3628754.7693536496,-3726138.7677319944,-3823522.7661103373,-3920906.764488687,-4018290.7628670065,-4115674.761245355,-4213428.240129082,-4310818.064751575,-4408207.889374049,-4505597.7139965305,-4602987.538619014,-4700377.3632415,-4797767.18786397,-4895157.0124864485,-4992546.837108942,-5089936.661731413,-5187326.486353911,-5284716.310976385,-5382106.135598877,-5479495.960221352,-5577366.134717282,-5674762.099978245,-5772158.065239222],[3867563.5934403436,3769542.0077730357,3671520.422105733,3573498.836438423,3475477.250771112,3377455.6651038053,3279434.0794364954,3181412.4937691875,3083390.908101883,2985369.3224345716,2887347.736767266,2789326.1510999575,2691304.56543265,2593282.9797653398,2495261.3940980323,2397239.8084307257,2299225.0831811028,2201222.772241735,2103230.983034415,2005248.1169293674,1907272.496298993,1809303.3463511837,1711339.914970725,1613381.4706024388,1515427.3001848268,1417476.70712941,1319529.0093410122,1221584.6260256225,1123643.0349098735,1025702.751051992,927765.8700817653,829829.6154052911,731896.1013923597,633962.7378869378,536029.0733664932,438095.55935356184,340144.5842604127,242207.70329018822,144270.82231996022,46297.50890085241,-51642.99346360704,-149636.74038982345,-247581.1237052139,-345525.5070205936,-443545.35073295794,-541493.8800941384,-639442.4094553129,-737390.938816492,-835443.4150011204,-933396.3614672031,-1031349.3079332849,-1129302.2543993667,-1227390.1192347975,-1325347.7602636735,-1423305.4012925439,-1521263.042321417,-1619220.6833502837,-1717178.3243791484,-1815314.4011387257,-1913277.0210306551,-2011239.6409225906,-2109202.260814511,-2207164.8807064467,-2305127.5005983794,-2403315.9387451317,-2501283.829096209,-2599251.719447287,-2697219.609798362,-2795187.5001494433,-2893155.390500513,-2991123.2808515914,-3089091.171202666,-3187347.432338902,-3285320.892505017,-3383294.3526711334,-3481267.812837245,-3579241.2730033575,-3677214.733169471,-3775188.1933355853,-3873161.653501696,-3971135.113667818,-4069108.5738339266,-4167082.0340000354,-4265429.487685532,-4363408.825257637,-4461388.162829743,-4559367.500401865,-4657346.837973969,-4755326.175546071,-4853305.513118169,-4951284.850690278,-5049264.188262391,-5147243.525834501,-5245222.863406605,-5343202.200978698,-5441181.538550828,-5539640.457698441,-5637625.9889898915,-5735611.52028134,-5833597.051572784],[3864485.309361487,3765872.9419012717,3667260.5744410553,3568648.2069808287,3470035.8395206113,3371423.4720603907,3272811.1046001716,3174198.7371399514,3075586.369679733,2976974.0022195117,2878361.634759294,2779749.267299075,2681136.899838855,2582524.5323786335,2483912.1649184143,2385299.797458196,2286696.927048713,2188105.6575138555,2089525.4818219985,1990953.4142193312,1892388.650030112,1793830.8405766247,1695278.3715416258,1596730.4999559456,1498187.3572450802,1399647.8332535666,1301111.2308998946,1202576.87485559,1104046.29637029,1005517.1794336485,906990.2586776537,808465.2117536154,709941.6924072779,611419.5670172109,512894.65371460444,414371.1343682669,315828.73414170067,217301.81338570872,118740.51151729282,20209.933031996246,-78320.64545330685,-176906.23359606136,-275440.73122912785,-373975.2288621934,-472587.1574819386,-571125.8412048598,-669664.5249277791,-768304.9251048234,-866848.0678156861,-965391.2105265502,-1063934.3532374245,-1162477.4959482835,-1261158.0582628222,-1359705.9392562988,-1458253.8202497791,-1556801.7012432637,-1655349.5822367482,-1754073.7544317064,-1852626.6598486109,-1951179.5652655102,-2049732.470682404,-2148285.376099297,-2246838.2815161976,-2345614.968618687,-2444173.1919051656,-2542731.415191623,-2641289.638478101,-2739847.86176456,-2838406.0850510295,-2936964.3083374994,-3035522.531623968,-3134367.3112069005,-3232931.153584093,-3331494.995961289,-3430058.8383384915,-3528622.6807156876,-3627186.523092876,-3725750.3654700704,-3824314.207847273,-3922878.050224471,-4021441.892601664,-4120005.7349788575,-4218942.155310224,-4317511.926255143,-4416081.69720006,-4514651.468144979,-4613221.239089881,-4711791.010034795,-4810360.780979695,-4908930.551924624,-5007500.322869522,-5106070.093814439,-5204639.864759369,-5303209.63570427,-5401779.406649182,-5500349.1775941,-5599403.901760731,-5697979.919499008,-5796555.937237215,-5895131.954975466],[3861402.2975738104,3762198.240982628,3662994.1843914394,3563790.127800251,3464586.0712090624,3365382.0146178748,3266177.958026686,3166973.901435501,3067769.844844313,2968565.788253124,2869361.731661936,2770157.6750707477,2670953.6184795615,2571749.561888371,2472545.505297185,2373341.8483541138,2274149.5243727434,2174969.2658079998,2075798.9178715071,1976637.0579421115,1877483.150965391,1778335.3614305102,1679192.9396819319,1580055.1548221866,1480921.2926163077,1381791.5471585328,1282665.1341528553,1183540.9883449653,1084419.4264549245,985300.5710192341,886182.7032480526,787067.959039585,687953.527190973,588841.7352732569,489724.66349374596,390610.23164513195,291475.49662095495,192357.62884976994,93203.80755451042,-5917.754335531034,-105092.13580399426,-204217.65508662164,-303343.1743692574,-402543.8710814044,-501673.61653917935,-600803.3619969534,-699933.1074547288,-799166.7578896927,-898301.0042526876,-997435.2506156797,-1096569.496978675,-1195838.8837029017,-1294977.9120954825,-1394116.9404880633,-1493255.9688806427,-1592394.9972732286,-1691708.0824423898,-1790852.180838754,-1889996.279235126,-1989140.3776314948,-2088284.4760278608,-2187428.5744242235,-2286572.6728205984,-2385943.787673765,-2485093.25136305,-2584242.7150523285,-2683392.178741607,-2782541.6424308983,-2881691.1061201817,-2980840.569809461,-3079990.033498745,-3179429.8092625625,-3278584.9413241935,-3377740.073385817,-3476895.205447456,-3576050.3375090742,-3675205.469570704,-3774360.6016323324,-3873515.733693964,-3972670.865755582,-4071825.9978172164,-4171352.189866051,-4270513.301656222,-4369674.413446381,-4468835.525236538,-4567996.637026692,-4667157.748816856,-4766318.86060703,-4865479.972397184,-4964641.084187335,-5063802.195977494,-5162963.307767665,-5262124.419557819,-5361285.531347979,-5460446.643138136,-5560091.779173829,-5659259.190823766,-5758426.60247372,-5857594.014123653,-5956761.425773587],[3858314.6251872773,3758517.9850067543,3658721.344826232,3558924.704645711,3459128.0644651866,3359331.4242846626,3259534.7841041395,3159738.1439236174,3059941.503743096,2960144.863562568,2860348.223382048,2760551.5832015253,2660754.943021003,2560958.302840479,2461161.662659955,2361366.698285615,2261583.4964736435,2161813.0051822686,2062051.756645822,1962299.9648234656,1862555.116109097,1762816.4348847796,1663083.166360354,1563354.5744018871,1463629.939403153,1363908.5561949788,1264191.1208015978,1164476.2922783308,1064763.0751924717,965053.3660451223,865344.5618486707,765638.3231962423,665932.7578819313,566225.8458913893,466519.6072389623,366795.11116102454,267085.40201367624,167341.8951872983,67628.45547819277,-32084.984230911825,-131852.97470572544,-231570.41009911243,-331287.84549248917,-431082.3927809526,-530804.0944748507,-630525.7961687539,-730247.497862658,-830075.2947869631,-929801.5393365012,-1029527.7838860373,-1129254.0284355688,-1229117.8738527,-1328848.9442048739,-1428580.0145570496,-1528311.0849092198,-1628042.155261401,-1727950.0735521973,-1827686.2595075406,-1927422.4454628811,-2027158.631418215,-2126894.8173735617,-2226631.0033289082,-2326592.0286013596,-2426333.627284947,-2526075.2259685346,-2625816.8246521107,-2725558.423335691,-2825300.0220192727,-2925041.620702863,-3024783.219386447,-3124813.1686499408,-3224560.484992558,-3324307.8013351755,-3424055.1176777957,-3523802.4340204066,-3623549.7503630277,-3723297.066705644,-3823044.383048254,-3922791.699390865,-4022539.0157334805,-4122655.771632025,-4222409.118862307,-4322162.466092597,-4421915.813322882,-4521669.160553163,-4621422.507783459,-4721175.855013751,-4820929.202244032,-4920682.549474321,-5020435.896704616,-5120189.243934896,-5219942.591165193,-5319695.938395473,-5419449.28562576,-5519685.621179012,-5619445.321327326,-5719205.021475649,-5818964.721623981,-5918724.421772292,-6018484.121920612],[3855222.358925358,3754832.253502649,3654442.148079942,3554052.042657229,3453661.937234522,3353271.831811812,3252881.7263891017,3152491.6209663926,3052101.5155436853,2951711.410120975,2851321.304698264,2750931.1992755556,2650541.093852848,2550150.988430137,2449760.8830074263,2349373.728818409,2248999.6689071367,2148637.1600340367,2048285.1935338178,1947941.523010751,1847604.8508713157,1747274.3964577112,1646949.39986905,1546629.1197575768,1446312.831166646,1345999.8234051862,1245689.5902365332,1145383.1988065084,1045078.4343048902,944776.0022148108,844475.6944404077,744176.7674759408,643879.7320156128,543578.913546998,443279.9865825281,342961.409674319,242658.97758423444,142321.20733696735,42015.00819000928,-58291.190956947394,-158653.67412459478,-258963.907293248,-359274.1404618984,-459663.4218703965,-559977.9615039239,-660292.5011374517,-760710.7430898738,-861029.8675615056,-961348.992033131,-1061668.1165047633,-1162122.4341569622,-1262446.4282293194,-1362770.422301677,-1463094.4163740329,-1563418.4104463998,-1663916.883501601,-1764246.038794525,-1864575.194087449,-1964904.349380368,-2065233.5046732882,-2165562.659966208,-2265891.815259138,-2366448.9987655533,-2466783.614233099,-2567118.2297006496,-2667452.8451681873,-2767787.460635729,-2868122.076103275,-2968456.6915708147,-3068791.3070383593,-3169417.9802811854,-3269758.3626986714,-3370098.7451161616,-3470439.127533648,-3570779.5099511407,-3671119.892368627,-3771460.2747861203,-3871800.6572036,-3972141.0396211008,-4072481.422038586,-4173195.603102169,-4273542.067564041,-4373888.532025911,-4474234.996487785,-4574581.460949641,-4674927.925411533,-4775274.389873398,-4875620.854335272,-4975967.31879713,-5076313.783259004,-5176660.247720876,-5277006.712182752,-5377353.176644611,-5478181.486520925,-5578534.356950224,-5678887.227379526,-5779240.097808822,-5879592.968238119,-5979945.83866743,-6080298.709096738],[3852125.565120337,3751141.12553303,3650156.685945722,3549172.2463584146,3448187.806771099,3347203.3671837905,3246218.9275964834,3145234.488009175,3044250.0484218677,2943265.6088345577,2842281.1692472454,2741296.7296599383,2640312.2900726288,2539327.8504853197,2438343.4108980093,2337363.1972409915,2236397.4090372073,2135442.739833333,2034498.459010569,1933562.0406468788,1832632.6761015677,1731709.7732878216,1630792.0232419404,1529879.1498365444,1428970.3395465147,1328064.8393953005,1227161.945149153,1126262.1176661411,1025364.938547913,924468.9146680019,823576.2364839693,722683.7524175972,621794.3803616115,520898.784284855,420006.30021848064,319092.7713885342,218196.74750861758,117263.8428360289,16364.015353018884,-84589.75901918951,-185493.658905942,-286397.5587926889,-387378.09842291847,-488286.34497677395,-589194.5915306341,-690102.8380844914,-791116.9384875731,-892029.8118929709,-992942.6852983613,-1093855.5587037615,-1194906.0447304826,-1295823.8315587016,-1396741.6183869196,-1497659.4052151297,-1598577.1920433491,-1699672.205527498,-1800595.1992107108,-1901518.1928939228,-2002441.1865771324,-2103364.180260348,-2204287.1739435606,-2305435.8777872464,-2406364.3791016014,-2507292.880415966,-2608221.3817303223,-2709149.883044687,-2810078.3843590477,-2911006.885673409,-3011935.3869877704,-3113153.837283398,-3214088.1548419935,-3315022.4724005964,-3415956.789959207,-3516891.107517815,-3617825.425076415,-3718759.742635018,-3819694.060193627,-3920628.377752236,-4021562.6953108273,-4122869.0371373435,-4223809.487893862,-4324749.938650391,-4425690.389406921,-4526630.840163437,-4627571.290919967,-4728511.741676494,-4829452.192432997,-4930392.643189517,-5031333.093946063,-5132273.544702573,-5233213.9954591,-5334154.446215621,-5435575.492456119,-5536522.402219981,-5637469.311983839,-5738416.221747711,-5839363.1315115485,-5940310.04127542,-6041256.951039274,-6142203.860803127],[3849024.3097086865,3747444.679688836,3645865.049668991,3544285.419649143,3442705.789629293,3341126.159609444,3239546.529589591,3137966.8995697442,3036387.2695498983,2934807.6395300487,2833228.0095101995,2731648.379490352,2630068.7494705026,2528489.1194506553,2426909.489430805,2325336.2927990844,2223776.9852623455,2122229.792455144,2020691.846746334,1919162.1543089477,1817640.0046561435,1716123.7062896083,1614612.5001486447,1513105.6437746156,1411602.8339666743,1310103.986203165,1208607.7682236643,1107113.4796966729,1005623.0078234747,904133.5304457773,802646.6331301965,701160.7571808742,599672.8384990301,498185.9411834474,396680.1765842817,295189.70471108146,193664.64749837015,92170.33542583883,-9323.976646695752,-110873.9344223151,-212372.35732413642,-313870.7802259545,-415447.742576282,-516950.5523842238,-618453.3621921623,-720059.5100164195,-821566.9887192538,-923074.4674220816,-1024581.9461249104,-1126224.5008905353,-1227736.9368612552,-1329249.3728319742,-1430761.808802694,-1532274.244773408,-1633786.680744133,-1735479.0920111942,-1836996.780487401,-1938514.4689636244,-2040032.1574398158,-2141549.8459160393,-2243067.534392245,-2344814.0757624134,-2446337.319335552,-2547860.5629086844,-2649383.806481812,-2750907.050054935,-2852430.293628067,-2953953.5372011852,-3055764.522560821,-3157293.6316750627,-3258822.7407892956,-3360351.849903535,-3461880.9590177652,-3563410.0681320056,-3664939.177246252,-3766468.286360489,-3867997.39547473,-3969526.5045889663,-4071425.7609607596,-4172961.054422549,-4274496.347884331,-4376031.641346126,-4477566.934807898,-4579102.228269689,-4680637.521731477,-4782172.815193256,-4883708.108655048,-4985243.402116827,-5086778.695578625,-5188313.989040387,-5289849.282502171,-5391384.575963972,-5493405.6199620655,-5594947.425460972,-5696489.230959878,-5798031.036458749,-5899572.841957656,-6001114.647456551,-6102656.452955466,-6204198.258454349],[3845918.658226728,3743742.99408408,3641567.329941434,3539391.665798782,3437216.0016561328,3335040.337513483,3232864.6733708344,3130689.009228188,3028513.3450855394,2926337.680942889,2824162.0168002415,2721986.352657592,2619810.688514943,2517635.0243722945,2415459.360229646,2313292.6335950824,2211139.385432051,2108998.1032983577,2006865.9221369305,1904742.7014452058,1802626.1625434088,1700515.5224739918,1598410.0170213217,1496308.8984364837,1394211.4331948487,1292117.6435501585,1190027.2594333054,1087938.8229905665,985853.0594876024,883769.4809771667,781687.3268309638,679607.3861970252,577523.0185385591,475440.8643923581,373338.4784698202,271252.714966855,169130.85595898936,67041.21561155748,-35048.424735873006,-137195.4114754973,-239289.20112018334,-341459.0044229664,-443557.22124849167,-545655.438074016,-647753.6548995408,-749957.3222088097,-852060.2500014259,-954163.1777940416,-1056266.105586649,-1158506.4874971327,-1260614.4164246721,-1362722.345352213,-1464830.2742797518,-1566938.2032072875,-1669223.5588102,-1771336.7859088304,-1873450.0130074676,-1975563.2401061063,-2077676.4672047445,-2179789.694303371,-2282129.315011421,-2384248.1446811003,-2486366.974350764,-2588485.8040204416,-2690604.6336901146,-2792723.463359786,-2894842.2930294545,-2996961.1226991345,-3099371.303333309,-3201496.047842702,-3303620.7923521223,-3405745.5368615137,-3507870.281370925,-3609995.0258803247,-3712119.770389729,-3814244.514899133,-3916369.259408544,-4018494.00391795,-4120993.151487382,-4223124.131489336,-4325255.111491282,-4427386.091493239,-4529517.071495179,-4631648.051497149,-4733779.031499086,-4835910.011501051,-4938040.991503001,-5040171.971504947,-5142302.9515069155,-5244433.931508853,-5346564.911510812,-5449180.231000178,-5551317.776058253,-5653455.321116337,-5755592.866174423,-5857730.411232505,-5959867.956290609,-6062005.501348698,-6164143.046406783,-6266280.591464875],[3842808.6758064074,3740036.1463507684,3637263.6168951304,3534491.087439492,3431718.557983851,3328946.0285282102,3226173.4990725713,3123400.969616934,3020628.440161296,2917855.910705654,2815083.381250015,2712310.8517943765,2609538.3223387366,2506765.7928831,2403993.2634274587,2301232.1742205094,2198484.6383306123,2095747.9499666528,1993021.6152348043,1890303.1213745032,1787591.3620709805,1684885.549336232,1582184.9138228055,1479488.7025131774,1376796.1764402704,1274106.6084332494,1171420.8086161283,1068737.3946030852,966055.5083233847,863376.9993803245,760698.7857755255,658023.9230618852,555342.2731709138,452664.03110111086,349964.190645193,247282.3043654929,144562.81044490915,41877.0106277857,-60863.65029118396,-163553.63791485783,-266243.6255385331,-369011.4895361434,-471705.94464999624,-574400.3997638505,-677094.8548776931,-779896.8743391484,-882596.0825201077,-985295.2907010568,-1087994.4988820138,-1190833.540372421,-1293537.7935762974,-1396242.0467801779,-1498946.299984055,-1601650.5531879338,-1704534.9377903272,-1807244.5348450774,-1909954.1318998197,-2012663.7289545592,-2115373.326009306,-2218082.9230640503,-2321022.010027946,-2423737.2571353093,-2526452.504242691,-2629167.751350059,-2731882.998457447,-2834598.2455648235,-2937313.492672207,-3040317.735852745,-3143038.9470994184,-3245760.158346101,-3348481.369592797,-3451202.5808394784,-3553923.792086167,-3656645.003332847,-3759366.214579532,-3862087.425826219,-3964808.637072905,-4067902.220105635,-4170629.7179845283,-4273357.215863424,-4376084.713742315,-4478812.211621212,-4581539.7095001135,-4684267.207379008,-4786994.7052579075,-4889722.203136808,-4992449.701015697,-5095177.198894587,-5197904.69677349,-5300632.194652388,-5403842.5138497725,-5506576.6297924705,-5609310.745735179,-5712044.861677864,-5814778.9776205355,-5917513.093563234,-6020247.209505926,-6122981.325448626,-6225715.441391315,-6328449.557333993],[3839694.427171212,3736324.2136340663,3632954.0000969265,3529583.7865597853,3426213.5730226412,3322843.3594854954,3219473.145948356,3116102.9324112143,3012732.718874075,2909362.505336928,2805992.291799786,2702622.0782626458,2599251.864725502,2495881.6511883605,2392512.2632940034,2289155.2763243574,2185812.5221952144,2082480.488046904,1979158.3006980177,1875843.7147501768,1772535.9162886522,1669234.1123190809,1565937.5283854238,1462645.4062234121,1359357.0014434168,1256071.581236734,1152788.8031674637,1049509.594307098,946231.6588902811,842956.5102140785,739682.5533959323,636406.2128616772,533131.0641854729,429836.6275541594,326557.7997620015,223243.8005440235,119961.02247475134,16678.244405482896,-86661.06051175762,-189948.0649371203,-293235.0693624923,-396601.814428709,-499893.32668716135,-603184.83894561,-706581.2355662216,-809877.5430185697,-913173.850470922,-1016470.1579232654,-1119903.5900286539,-1223204.986411911,-1326506.3827951583,-1429807.7791784075,-1533109.175561659,-1636588.0193060474,-1739894.8052331577,-1843201.591160277,-1946508.3770873952,-2049815.1630145032,-2153121.948941619,-2256655.6239676913,-2359968.1074356353,-2463280.5909035862,-2566593.0743715256,-2669905.557839478,-2773218.041307418,-2876530.5247753724,-2979843.0082433177,-3083448.047744959,-3186766.5446519586,-3290085.0415589605,-3393403.5384659534,-3496722.0353729594,-3600040.532279961,-3703359.0291869603,-3806677.5260939626,-3909996.0230009593,-4013684.757650365,-4117009.592323199,-4220334.426996039,-4323659.261668863,-4426984.096341704,-4530308.931014544,-4633633.765687363,-4736958.600360215,-4840283.43503304,-4943608.269705893,-5046933.1043787105,-5150257.939051548,-5253582.773724383,-5357388.80440425,-5460720.310136594,-5564051.815868896,-5667383.321601236,-5770714.827333552,-5874046.333065868,-5977377.8387982175,-6080709.344530533,-6184040.850262864,-6287372.355995181,-6390703.86172751],[3836575.976632373,3732607.2725877822,3628638.5685431845,3524669.8644985934,3420701.160453997,3316732.4564094036,3212763.752364807,3108795.048320212,3004826.3442756208,2900857.6402310217,2796888.9361864273,2692920.232141832,2588951.5280972384,2484982.824052642,2381016.1537984973,2277063.1561185634,2173123.2988014626,2069195.4089816096,1965276.264944702,1861364.917630828,1757460.880921997,1653562.4096616234,1549668.7334808598,1445779.357556995,1341894.2684988356,1238012.1907878653,1134132.3950395046,1030255.8193314376,926381.2969697185,822508.4353355323,718637.930982531,614762.7120671507,510889.8504329594,406996.36508303834,303119.7893749671,199206.5630824617,95326.00031191576,-8608.491800898686,-112493.31951626157,-216378.14723162167,-320340.0331487092,-424229.4090725775,-528118.784996442,-632008.1609203056,-736004.4955555787,-839898.7088258881,-943792.9220962035,-1047687.1353665199,-1151720.810667316,-1255620.1567954998,-1359519.5029236935,-1463418.8490518872,-1567318.1951800715,-1671397.6491994024,-1775302.430576757,-1879207.2119541233,-1983111.9933314826,-2087016.7747088452,-2190921.556086195,-2295056.2761744196,-2398966.802586577,-2502877.328998721,-2606787.85541088,-2710698.381823026,-2814608.9082351695,-2918519.434647314,-3022720.0142716,-3126636.603421942,-3230553.192572285,-3334469.7817226313,-3438386.370872964,-3542302.960023313,-3646219.5491736433,-3750136.1383239813,-3854052.7274743207,-3957969.316624659,-4062260.295559722,-4166183.2736027893,-4270106.251645869,-4374029.229688935,-4477952.2077320125,-4581875.185775088,-4685798.163818161,-4789721.141861243,-4893644.119904298,-4997567.097947378,-5101490.075990456,-5205413.054033513,-5309336.032076611,-5413745.197696025,-5517674.899781722,-5621604.601867422,-5725534.303953113,-5829464.006038826,-5933393.708124533,-6037323.410210224,-6141253.112295939,-6245182.814381629,-6349112.516467323,-6453042.218553029],[3833453.388085097,3728885.3993698107,3624317.4106545267,3519749.4219392426,3415181.4332239544,3310613.4445086685,3206045.4557933854,3101477.4670781004,2996909.4783628173,2892341.489647529,2787773.5009322446,2683205.5122169624,2578637.5235016756,2474069.5347863897,2369504.7873129123,2264954.9834969705,2160418.147495612,2055892.6877529058,1951375.9785467857,1846867.9021580697,1742366.2282219294,1637870.1647305042,1533378.9361084532,1428891.7808109648,1324408.3355719792,1219928.8072747712,1115451.5821121093,1010976.4643372272,906504.5625671521,802033.2399217994,697565.3498808087,593090.4564903404,488619.087798072,384125.7576879391,279650.63991306187,175137.39060161868,70658.24893429922,-33876.452534838114,-138359.89777382836,-242843.34301281348,-347405.6732703694,-451893.70712474687,-556381.7409791271,-660973.929588289,-765466.8429664145,-869959.7563445405,-974452.6697226702,-1079082.2066653604,-1183580.2968463711,-1288078.3870273852,-1392576.4772083908,-1497074.5673894081,-1601749.9456891883,-1706253.5168360826,-1810757.0879829652,-1915260.659129846,-2019764.2302767383,-2124267.8014236293,-2228998.5686749723,-2333507.9323555212,-2438017.296036061,-2542526.6597166024,-2647036.0233971477,-2751545.3870776924,-2856054.750758235,-2960564.114438788,-3065367.041851189,-3169882.5175676965,-3274397.9932841896,-3378913.4690006804,-3483428.9447171856,-3587944.4204336926,-3692459.896150183,-3796975.3718666798,-3901490.847583183,-4006378.424513193,-4110900.340241922,-4215422.255970656,-4319944.171699377,-4424466.087428107,-4528988.0031568315,-4633509.918885551,-4738031.834614279,-4842553.750343005,-4947075.66607173,-5051597.581800473,-5156119.49752918,-5260641.413257904,-5365647.729881817,-5470176.422623165,-5574705.115364526,-5679233.808105874,-5783762.500847233,-5888291.19358859,-5992819.886329949,-6097348.5790712945,-6201877.271812634,-6306405.964553996,-6410934.657295352,-6515463.3500367105],[3830326.725005068,3725158.669638064,3619990.6142710545,3514822.558904048,3409654.5035370383,3304486.448170029,3199318.3928030226,3094150.337436014,2988982.2820690083,2883814.2267020005,2778646.171334992,2673478.1159679866,2568310.060600977,2463142.00523397,2357978.4194759503,2252831.0046605146,2147696.6512268144,2042572.5948941945,1937458.4963681707,1832351.9506372267,1727251.857597387,1622157.4201919227,1517067.8576612857,1411982.403101956,1306900.3010435565,1201821.7984571916,1096746.3632171252,991672.5175715934,886601.8598922819,781532.3844238308,676460.5445336595,571389.8868543501,466299.71229460393,361225.27047418524,256115.2751799135,151036.77259354806,45958.27000718517,-59177.424405409954,-164260.2692273655,-269343.1140493164,-374506.6720829806,-479594.1459576553,-584681.6198323392,-689875.2829966028,-794967.6785960798,-900060.0741955526,-1005152.4697950399,-1110383.7842427008,-1215481.4006071505,-1320579.0169716026,-1425676.633336044,-1530774.2497005016,-1636051.7702612672,-1741154.9133187956,-1846258.05637633,-1951361.1994338674,-2056464.3424913846,-2161567.48554892,-2266900.827397237,-2372009.8104913747,-2477118.7935855095,-2582227.776679645,-2687336.7597737787,-2792445.7428679033,-2897554.725962047,-3002954.6218247893,-3108069.7662505144,-3213184.9106762423,-3318300.055101977,-3423415.1995276995,-3528530.343953426,-3633645.4883791422,-3738760.6328048804,-3843875.777230612,-3948990.9216563404,-4054482.266969776,-4159603.9025191627,-4264725.538068537,-4369847.173617911,-4474968.8091673,-4580090.44471667,-4685212.080266043,-4790333.715815429,-4895455.3513648,-5000576.986914179,-5105698.622463551,-5210820.258012938,-5316424.400572055,-5421552.866090349,-5526681.331608656,-5631809.797126927,-5736938.26264524,-5842066.728163524,-5947195.193681826,-6052323.65920011,-6157452.124718418,-6262580.590236709,-6367709.055754999,-6472837.5212732935,-6577965.986791595],[3827196.050445076,3721427.158546355,3615658.266647632,3509889.37474891,3404120.482850185,3298351.5909514613,3192582.699052739,3086813.8071540166,2981044.915255295,2875276.0233565704,2769507.1314578475,2663738.239559127,2557969.347660402,2452200.4557616776,2346438.4057809217,2240691.818911348,2134958.822397922,2029236.2498706593,1923523.1409861722,1817817.355944651,1712118.0739994417,1606424.4930684206,1500735.8272302556,1395051.3042435467,1289370.1630820427,1183691.6514790738,1078017.1157255336,972344.3069691197,866673.5904639151,761005.1942010543,655332.1574535067,549661.440948304,443969.9012042931,338295.36545075243,232583.82872707234,126905.19529157598,21172.11742002284,-84510.89695102023,-190193.91132205585,-295954.7989086164,-401642.4827991384,-507330.1666896548,-613017.8505801759,-718813.7593924473,-824506.4072315698,-930199.0550707043,-1035891.7029098277,-1141725.5657232068,-1247423.4783055326,-1353121.390887848,-1458819.3034701636,-1564694.1644465039,-1670397.6494587278,-1776101.1344709415,-1881804.6194831715,-1987508.104495396,-2093211.5895076212,-2199142.3886602963,-2304851.761215327,-2410561.133770368,-2516270.506325408,-2621979.8788804375,-2727689.2514354745,-2833398.6239905204,-2939107.9965455583,-3045111.7427405217,-3150827.325919918,-3256542.9090993316,-3362258.4922787235,-3467974.0754581224,-3573689.6586375292,-3679405.241816928,-3785120.8249963233,-3890836.4081757264,-3996925.7489907164,-4102647.874396464,-4208369.999802213,-4314092.125207968,-4419814.250613704,-4525536.3760194555,-4631258.501425204,-4736980.6268309485,-4842702.752236709,-4948424.877642457,-5054147.003048205,-5159869.128453945,-5265591.253859685,-5371800.768086254,-5477529.776402934,-5583258.784719603,-5688987.793036297,-5794716.801352971,-5900445.809669654,-6006174.817986318,-6111903.826303012,-6217632.834619673,-6323361.842936353,-6429090.851253037,-6534819.859569726,-6640548.867886399],[3824061.4270318183,3717690.9407406566,3611320.4544494925,3504949.968158324,3398579.4818671583,3292208.995575991,3185838.5092848265,3079468.02299366,2973097.536702495,2866727.0504113273,2760356.5641201623,2653986.0778289954,2547615.5915378286,2441245.1052466645,2334883.831854917,2228538.105764581,2122204.9159191046,2015883.3529405016,1909570.1913024052,1803264.408978574,1696965.3563251393,1590671.8235732834,1484383.1716839622,1378098.8230884243,1271817.8858917193,1165539.6020249631,1059264.214460224,952991.6862512243,846720.1530897459,740452.0800351205,634177.0867667892,527905.5536053095,421611.8890717882,315336.5015070506,209022.66384996986,102743.14164579473,-3592.4212823286653,-109876.36315700086,-216160.30503167538,-322523.91043450404,-428812.5623238287,-535101.2142131496,-641495.1215124396,-747788.7795959669,-854082.4376794924,-960376.0957630188,-1066807.9570040447,-1173106.923824184,-1279405.8906443259,-1385704.8574644732,-1492003.8242846103,-1598482.3108807886,-1704786.8958763964,-1811091.4808719894,-1917396.065867607,-2023700.6508632074,-2130005.2358588134,-2236540.0905337925,-2342850.610580864,-2449161.1306279525,-2555471.6506750365,-2661782.170722109,-2768092.6907692,-2874403.2108162814,-2981005.305208418,-3087322.085169033,-3193638.8651296524,-3299955.645090259,-3406272.4250508775,-3512589.2050114824,-3618905.9849721007,-3725222.7649327293,-3831539.5448933393,-3938227.5362990657,-4044550.909579345,-4150874.2828596225,-4257197.65613992,-4363521.02942022,-4469844.402700498,-4576167.775980796,-4682491.149261079,-4788814.522541367,-4895137.8958216645,-5001461.269101943,-5107784.642382239,-5214108.015662524,-5320916.722369545,-5427247.031487963,-5533577.340606363,-5639907.649724779,-5746237.958843198,-5852568.267961608,-5958898.577080016,-6065228.886198429,-6171559.195316842,-6277889.504435247,-6384219.813553677,-6490550.122672061,-6596880.431790481,-6703210.740908892],[3820922.9169628387,3713950.0903554093,3606977.2637479734,3500004.4371405374,3393031.6105331024,3286058.7839256646,3179085.9573182315,3072113.130710797,2965140.304103362,2858167.477495925,2751194.6508884896,2644221.8242810545,2537248.9976736214,2430276.1710661855,2323314.9270096654,2216369.314610862,2109436.1320000165,2002513.8848929962,1895599.9243080977,1788694.2445208027,1681794.7628381052,1574900.5815933985,1468010.9115724852,1361125.296168949,1254243.817904463,1147365.0170043313,1040488.1352905331,933615.0395686482,826742.9337982836,719873.4498949719,612995.7524029906,506103.10504385596,399226.11971710925,292313.4052275724,185432.24826347316,78551.09129937273,-28387.704077014234,-135273.31948054163,-242158.93488407647,-349126.0053403354,-456016.3712810995,-562906.737221865,-669904.353992336,-776799.7683937154,-883695.1827951036,-990590.5971964868,-1097626.4684133092,-1204527.2355590141,-1311428.0027047233,-1418328.7698504268,-1525405.9646680295,-1632312.3957426408,-1739218.8268172476,-1846125.2578918613,-1953031.6889664708,-2059938.120041071,-2167071.7938753227,-2273984.2075117365,-2380896.621148132,-2487809.034784546,-2594721.448420941,-2701633.8620573482,-2808546.275693765,-2915458.689330166,-3022666.0883422326,-3129584.811177023,-3236503.534011812,-3343422.2568466016,-3450340.979681402,-3557259.702516187,-3664178.4253509836,-3771097.1481857733,-3878015.8710205615,-3985309.8005537214,-4092235.167791496,-4199160.535029286,-4306085.902267061,-4413011.269504851,-4519936.63674263,-4626862.003980413,-4733787.371218212,-4840712.738455993,-4947638.105693783,-5054563.47293156,-5161488.840169351,-5268897.378199109,-5375829.734186825,-5482762.090174564,-5589694.446162287,-5696626.802150014,-5803559.1581377415,-5910491.514125474,-6017423.870113206,-6124356.226100917,-6231288.582088654,-6338220.938076373,-6445153.294064103,-6552085.650051833,-6659018.006039558,-6765950.362027284],[3817780.58200364,3710204.6810101005,3602628.780016557,3495052.8790230136,3387476.978029467,3279901.0770359216,3172325.176042377,3064749.2750488357,2957173.374055293,2849597.4730617465,2742021.572068203,2634445.6710746596,2526869.7700811164,2419294.2059096307,2311732.192056074,2204185.6850117086,2096651.9499047499,1989128.108960899,1881613.623242429,1774106.5656042667,1666605.6591523392,1559110.0957104529,1451619.080792679,1344131.8310034801,1236648.305133718,1129168.254245339,1021690.1407488013,914214.7484058472,806741.4560190586,699263.9637199419,591788.5713769884,484292.35107489,376813.03387981746,269296.57230584184,161813.04643607605,54329.5205663112,-53213.2427628655,-160701.26587397745,-268267.74564434635,-375760.5598416948,-483253.37403903436,-590746.1882363763,-698348.249621008,-805846.1545652472,-913344.0595094925,-1020979.2788466746,-1128482.5805563307,-1235985.8822659785,-1343489.1839756314,-1450992.4856852815,-1558674.7415260077,-1666183.7529250234,-1773692.7643240336,-1881201.7757230476,-1988710.7871220596,-2096219.7985210759,-2203958.9608649462,-2311474.002336943,-2418989.0438089543,-2526504.0852809595,-2634019.126752965,-2741534.168224965,-2849049.2096969807,-2956856.288749142,-3064377.6886993577,-3171899.08864958,-3279420.488599795,-3386941.888550009,-3494463.2885002284,-3601984.6884504473,-3709506.088400671,-3817027.4883508785,-3924921.39394559,-4032449.4893714916,-4139977.584797389,-4247505.68022329,-4355033.775649183,-4462561.871075094,-4570089.966500981,-4677618.0619268855,-4785146.157352787,-4892674.252778688,-5000202.348204583,-5107730.443630481,-5215258.539056376,-5323274.577043233,-5430809.714114993,-5538344.851186753,-5645879.988258524,-5753415.125330285,-5860950.2624020465,-5968485.399473826,-6076020.5365455765,-6183555.673617355,-6291090.81068911,-6398625.947760878,-6506161.084832638,-6613696.221904401,-6721231.358976174,-6828766.496047933],[3814634.4834849522,3706454.7858060086,3598275.0881270627,3490095.390448111,3381915.6927691624,3273735.995090212,3165556.2974112644,3057376.5997323156,2949196.9020533706,2841017.2043744177,2732837.506695471,2624657.809016523,2516478.111337575,2408299.891115055,2300136.4322270798,2191987.894019594,2083852.44401679,1975726.928510804,1867610.6476578477,1759501.3952513235,1651398.3418723834,1543300.6742693712,1435207.5927061508,1327118.308318078,1219032.040467815,1110949.669059263,1002869.6028285476,894791.1915640349,786715.9919405868,678634.3690350088,570555.957770498,462455.44060795545,354373.06919940305,246252.13672822504,138165.51956924144,30022.600661385804,-78068.55257282779,-186159.70580704557,-294331.0689438055,-402427.05383869447,-510523.038733589,-618725.1350145531,-726826.2529615457,-834927.3709085495,-943028.4888555435,-1051269.1321438649,-1159375.690889835,-1267482.2496357993,-1375588.8083817745,-1483695.367127739,-1591983.406805431,-1700095.721007415,-1808208.035209394,-1916320.34941136,-2024432.6636133487,-2132771.959367243,-2240890.3511535064,-2349008.7429397698,-2457127.13472604,-2565245.5265123015,-2673363.9182985662,-2781482.3100848356,-2889600.7018710924,-2998014.4922499605,-3106139.2917885357,-3214264.091327125,-3322388.8908656985,-3430513.6904042684,-3538638.489942853,-3646763.2894814406,-3754888.0890200175,-3863012.8885585964,-3971514.1362226075,-4079645.68229831,-4187777.2283740155,-4295908.774449719,-4404040.320525431,-4512171.866601133,-4620303.412676829,-4728434.958752529,-4836566.504828233,-4944698.0509039415,-5052829.596979645,-5160961.143055351,-5269578.307454036,-5377716.948055105,-5485855.588656171,-5593994.229257236,-5702132.869858289,-5810271.510459359,-5918410.1510604285,-6026548.791661504,-6134687.432262572,-6242826.072863628,-6350964.713464689,-6459103.35406576,-6567241.994666842,-6675380.63526789,-6783519.275868973,-6891657.91647005],[3811484.6823001946,3702700.4773231624,3593916.272346125,3485132.0673690913,3376347.862392057,3267563.657415024,3158779.45243799,3049995.2474609558,2941211.0424839235,2832426.837506889,2723642.6325298543,2614858.4275528206,2506074.222575789,2397292.6350286724,2288527.019244366,2179776.4052309203,2071037.8621818363,1962310.413349153,1853591.174133831,1744879.0162930537,1636173.1054891415,1527472.6234178843,1418776.7651146362,1310084.736265502,1201395.750515548,1092709.614225748,984026.8859556324,875345.3578798531,766666.929308872,657980.4635250682,549298.322714882,440592.79648694675,331906.6601971458,223180.54473163746,114490.1255782512,5741.841899456456,-102953.15219401754,-211648.14628749387,-320425.10458450625,-429124.97093759384,-537824.8372906703,-646632.7708067982,-755337.8125351723,-864042.8542635525,-972747.8959919303,-1081594.6745921066,-1190305.2011646111,-1299015.7277371166,-1407726.2543096235,-1516614.9876822275,-1625331.315482745,-1734047.6432832656,-1842763.971083778,-1951480.2988842963,-2060196.6266848124,-2169142.7966774916,-2277865.2495729397,-2386587.7024683733,-2495310.155363817,-2604032.6082592597,-2712755.0611546906,-2821477.5140501256,-2930492.269093363,-3039221.1790088136,-3147950.088924255,-3256678.9988397006,-3365407.908755145,-3474136.818670583,-3582865.72858603,-3691594.6385014807,-3800323.548416921,-3909426.0504372544,-4018161.757939431,-4126897.4654415967,-4235633.172943757,-4344368.880445923,-4453104.587948089,-4561840.295450251,-4670576.002952429,-4779311.710454591,-4888047.417956746,-4996783.125458919,-5105518.8329610815,-5214737.727295205,-5323480.582185289,-5432223.437075374,-5540966.291965467,-5649709.146855551,-5758452.001745646,-5867194.856635734,-5975937.711525831,-6084680.5664159125,-6193423.421306027,-6302166.2761960775,-6410909.131086186,-6519651.985976277,-6628394.840866362,-6737137.695756456,-6845880.550646541,-6954623.40553665],[3808331.238903018,3698941.8276174376,3589552.41633185,3480163.005046269,3370773.5937606855,3261384.1824751017,3151994.771189518,3042605.3599039353,2933215.948618354,2823826.5373327704,2714437.1260471875,2605047.7147616055,2495658.303476021,2386272.648932923,2276904.175986471,2167550.786080054,2058209.2578163808,1948878.3686871324,1839555.471881163,1730239.907059488,1620930.6940701194,1511626.447620193,1402326.913526377,1293031.4419213943,1183739.0402243738,1074448.9213139787,965162.3519261165,855877.4766483556,746594.6530718207,637302.6437625312,527995.4387500896,418704.8385043761,309377.15959158307,200082.23933271738,90787.31907384563,-18567.029693256132,-127866.56378707476,-237244.9045196902,-346549.3514955612,-455853.79847143125,-565158.245447306,-674572.7161519094,-783882.3808433404,-893192.0455347626,-1002640.130266638,-1111955.323858067,-1221270.5174494982,-1330585.7110409308,-1439900.9046323635,-1549396.7865943592,-1658717.827190335,-1768038.86778632,-1877359.9083823054,-1986680.948978288,-2096228.5197175466,-2205555.7329176725,-2314882.9461177965,-2424210.1593179195,-2533537.3725180537,-2642864.5857181745,-2752191.7989183106,-2861519.0121184345,-2971141.838297632,-3080475.557778365,-3189809.2772591,-3299142.9967398415,-3408476.716220579,-3517810.4357013213,-3627144.155182064,-3736477.8746628,-3845811.5941435285,-3955522.7952976413,-4064863.3634022567,-4174203.9315068647,-4283544.499611489,-4392885.067716095,-4502225.635820729,-4611566.203925329,-4720906.77202994,-4830247.340134558,-4939587.908239167,-5048928.476343797,-5158269.044448398,-5268097.460880746,-5377445.229218407,-5486792.997556054,-5596140.765893712,-5705488.5342313675,-5814836.302569032,-5924184.0709067,-6033531.839244358,-6142879.607582015,-6252227.375919662,-6361575.144257331,-6470922.912594983,-6580270.680932641,-6689618.4492703,-6798966.21760798,-6908313.985945621,-7017661.754283306],[3805174.2133050156,3695178.9082177957,3585183.6031305776,3475188.2980433553,3365192.992956137,3255197.6878689164,3145202.3827816937,3035207.077694476,2925211.7726072567,2815216.4675200377,2705221.162432818,2595225.8573455983,2485230.552258378,2375240.7042242773,2265268.499987513,2155311.269318618,2045366.3427062116,1935431.0502703723,1825504.301956619,1715585.0180618446,1605671.5622675386,1495763.1128689102,1385858.858937407,1275958.7499713544,1166062.2457605388,1056168.045382495,946276.3598923716,836387.4514234941,726492.988912123,616601.3034220014,506687.73515314143,396791.9833852418,286857.7692768844,176957.6603108314,67001.21044533141,-42903.55127956066,-152808.31300445134,-262793.59701587027,-372703.31226783665,-482613.0275197993,-592629.463740868,-702544.4390647998,-812459.4143887428,-922374.389712682,-1032429.9535748363,-1142350.5018645092,-1252271.0501541775,-1362191.5984438574,-1472289.4240921387,-1582215.8651666464,-1692142.3062411407,-1802068.7473156443,-1911995.1883901455,-2021921.6294646412,-2132077.413601086,-2242010.0747868055,-2351942.7359725167,-2461875.3971582376,-2571808.0583439553,-2681740.7195296613,-2791673.3807153786,-2901898.4096570686,-3011837.626376259,-3121776.843095435,-3231716.059814623,-3341655.2765338104,-3451594.493252985,-3561533.709972169,-3671472.9266913566,-3781412.1434105355,-3891725.830731008,-4001671.9470981965,-4111618.0634653894,-4221564.179832588,-4331510.296199773,-4441456.412566958,-4551402.528934152,-4661348.645301342,-4771294.761668532,-4881240.878035735,-4991186.994402923,-5101133.110770117,-5211564.482318347,-5321517.851745743,-5431471.2211731775,-5541424.590600599,-5651377.960028014,-5761331.32945544,-5871284.69888285,-5981238.068310251,-6091191.437737686,-6201144.80716509,-6311098.176592527,-6421051.546019942,-6531004.9154473385,-6640958.284874771,-6750911.654302179,-6860865.023729601,-6970818.393157023,-7081415.294125774]],"type":"heatmap","xaxis":"x","yaxis":"y","hovertemplate":"Home Price: %{x}\u003cbr\u003eMortgage Rate: %{y}\u003cbr\u003eBreak Even: %{z}\u003cextra\u003e\u003c\u002fextra\u003e"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"autorange":"reversed","title":{"text":"Mortgage Rate"}},"coloraxis":{"colorbar":{"title":{"text":"Break Even"}},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [17]:
# A little magic to automatically write my blog :)
import subprocess

subprocess.run(
    [
        "jupyter",
        "nbconvert",
        "--to",
        "markdown",
        "--output",
        "~/jakee417.github.io/_includes/rent_vs_buy_blog_post.md",
        "rent_vs_buy_blog_post.ipynb",
    ]
)

[NbConvertApp] Converting notebook rent_vs_buy_blog_post.ipynb to markdown
/Users/jakeetaylor/opt/anaconda3/lib/python3.9/site-packages/nbconvert/filters/datatypefilter.py:39: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  warn("Your element with mimetype(s) {mimetypes}"
[NbConvertApp] Writing 478897 bytes to /Users/jakeetaylor/jakee417.github.io/_includes/rent_vs_buy_blog_post.md


CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'markdown', '--output', '~/jakee417.github.io/_includes/rent_vs_buy_blog_post.md', 'rent_vs_buy_blog_post.ipynb'], returncode=0)